In [4]:
import os# define the arguments for the model:
from src.functions import get_model_files, get_single_model_file_with_path, prolog_reasoning

In [52]:
prob_ec_cached = """
% 1. 在时间0时, aux为true
holdsAt_(aux = true, 0).
% 2. 状态从Tprev延续到T, 没有被打断
holdsAt(F = V, T) :-                            % holdsAt(sequence0 = false, 13).   (GIVEN)
  \+ sdFluent(F),                               % not sdFluent( aux ). ==> F != aux
  T @> 0,                                       % 13 > 0
  allTimeStamps(Timestamps),                    % allTimeStamps([...]),
  previousTimeStamp(T, Timestamps, Tprev),      % previousTimeStamp(13, [...], 12),
  holdsAt_(F = V, Tprev),                       % holdsAt_(sequence0 = false, 12).
  \+ broken(F = V, Tprev, T).                   % not broken(sequence0 = false, 12, 13).
% 3. 状态在Tprev时被初始化
holdsAt(F = V, T) :-                            % holdsAt(sequence0 = false, 13).   (GIVEN)
  \+ sdFluent(F),                               % not sdFluent( aux ). ==> F != aux
  T @> 0,                                       % 13 > 0
  allTimeStamps(Timestamps),                    % allTimeStamps([...]),
  previousTimeStamp(T, Timestamps, Tprev),      % previousTimeStamp(16, [...], 12),
  initAtDigit(F = V, Tprev).                    % initiatedAt(sequence0 = false, 12). => recurrent

%===== broken/3 谓词：状态被中断的判断 =====% 
% 1. T1到T2时间内, 状态F从V1变化成了V2
broken(F = V1, T1, T2):-                        % broken(sequence0 = false, 12, 16). 
  allTimeStamps(Timestamps),                    % allTimeStamps([...]),
  previousTimeStamp(T2, Timestamps, T2prev),    % previousTimeStamp(16, [...], 12),
  initAtDigit(F = V2, T2prev),                  % initiatedAt(sequence0 = true, 12),
  V1 \= V2.                                     % false \= true.
% 2. T1到T2时间内, 若存在其他时间戳, 递归
broken(F = V, T1, T2) :-                        % broken(sequence0 = false, 12, 16). 
  allTimeStamps(Timestamps),                    % allTimeStamps([...]),
  previousTimeStamp(T2, Timestamps, T2prev),    % previousTimeStamp(16, [...], 12),
  T2prev > T1,                                  % 12 > 12,
  broken(F = V, T1, T2prev).                    % broken(sequence0 = false, 12, 12).

previousTimeStamp(T, Timestamps, Tprev):- Tprev is T - 1.
nextTimeStamp(T, Timestamps, Tnext):- Tnext is T + 1.

sdFluent( aux ).

"""

event_defs_digit = """
%=====================consider more timestamps:======================%
givenRemaining(3).

% In the window of size Remaining, if two (not necessarily consecutive) events occur,
% then initAtDigit(sequence = EventID, T) is true with event EventID and timestamp T.
initAtDigit(sequence = EventID, T) :-                   % 
    givenRemaining(Remaining),                          % 
    sequenceEndingAt([EventID, EventID], Remaining, T). % 


% An empty sequence will always be within
sequenceWithin([], _, _).

% Case1: A sequence can be within Remaining of T if it end at T:
sequenceWithin(L, Remaining, T) :-
    sequenceEndingAt(L, Remaining, T).

% Case2: If Remaining>0, check with recursion: sequenceWithin of last timestamp.
sequenceWithin(L, Remaining, T) :-
    Remaining > 0,
    T >= 0,
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    sequenceWithin(L, NextRemaining, Tprev).


% A sequence will start at T and be within Remaining if X happens at T and the rest of the sequence is within NextRemaining of Tprev
sequenceEndingAt([X | L], Remaining, T) :-
    Remaining > 0,
    T >= 0,
    happensAt(Y, T),
    digit(Y, X),
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    sequenceWithin(L, NextRemaining, Tprev).

happdigit(X, T):-
    happensAt(Y, T),
    digit(Y, X).

previousTimeStamp(T, Timestamps, Tprev):- Tprev is T - 1.
nextTimeStamp(T, Timestamps, Tnext):- Tnext is T + 1.


"""

event_defs_sound = """
initiatedAtNoise(X, Y) :- initiatedAt(X, Y).

% Number of timestamps to look at (should be min the length of the sequence)
givenRemaining(2).

% 
initiatedAt(sequence0 = true, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([air_conditioner, air_conditioner], Remaining, T). % 
initiatedAt(sequence0 = false, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([car_horn, car_horn], Remaining, T).

initiatedAt(sequence1 = true, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([children_playing, children_playing], Remaining, T). % 
initiatedAt(sequence1 = false, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([dog_bark, dog_bark], Remaining, T). % 

initiatedAt(sequence2 = true, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([drilling, drilling], Remaining, T).
initiatedAt(sequence2 = false, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([engine_idling, engine_idling], Remaining, T).

initiatedAt(sequence3 = true, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([gun_shot, gun_shot], Remaining, T).
initiatedAt(sequence3 = false, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([jackhammer, jackhammer], Remaining, T).

initiatedAt(sequence4 = true, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([siren, siren], Remaining, T).
initiatedAt(sequence4 = false, T) :-
    givenRemaining(Remaining),
    sequenceEndingAt([street_music, street_music], Remaining, T).

% An empty sequence will always be within
sequenceWithin([], _, _).

% A sequence can be within Remaining of T if it starts at T
sequenceWithin(L, Remaining, T) :-
    sequenceEndingAt(L, Remaining, T).

% A sequence can be within Remaining of T if it is within NextRemaining of Tprev
sequenceWithin(L, Remaining, T) :-
    Remaining > 0,
    T >= 0,
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    sequenceWithin(L, NextRemaining, Tprev).

% A sequence will start at T and be within Remaining if X happens at T and the rest of the sequence is within NextRemaining of Tprev
% example: sequenceEndingAt([car_horn, car_horn], 2, 153) :-
%   2 >= 0;
%   153 >= 0; 
%   happensAt(6310, 153), % 153 是一个事件发生的时间
%   sound(6310, car_horn), % car_horn 是一个发生的声音
%   NextRemaining is 2 - 1,
%   allTimeStamps(Timestamps),
%   previousTimeStamp(153, Timestamps, 151),
%   sequenceWithin(car_horn, 1, 151). % 递归检查

sequenceEndingAt([X | L], Remaining, T) :-
    Remaining > 0, 
    T >= 0,
    happensAt(Y, T),
    sound(Y, X),
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    sequenceWithin(L, NextRemaining, Tprev).

sdFluent( aux ).    % aux 被定义为状态依赖型fluent
"""

In [138]:
my_event_defs = """
:- use_module(library(lists), [ member/2,
                                append/2 as list_concat
                              ]).
%===%============================================================%===%
%============given window size and number of identicals:=============%
givenRemaining(4).
requiredOccurrences(3).

%===%=============================2==============================%===%
%=====================consider more timestamps:======================%

% In the window of size Remaining, if ??? (not necessarily consecutive) events occur,
% then initAtDigit(sequence = EventID, T) is true with event EventID and timestamp T.
initAtEvent(sequence = EventID, T) :-
    givenRemaining(Remaining),
    requiredOccurrences(N),
    occursNTimes(EventID, N, Remaining, T).

% if requiredOccurrences reduce to 0, then correct
occursNTimes(_, 0, Remaining, _):- Remaining >= 0.

occursNTimes(EventID, N, Remaining, T) :-
    Remaining > 0,
    N > 0,
    T >= 0,
    happensAt(Y, T),
    digit(Y, EventID),
    N1 is N - 1,
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    occursNTimes(EventID, N1, NextRemaining, Tprev).

occursNTimes(EventID, N, Remaining, T) :-
    Remaining > 0,
    T >= 0,
    happensAt(Y, T),
    digit(Y, EventID2),  
    EventID2 \= EventID,
    NextRemaining is Remaining - 1,
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    occursNTimes(EventID, N, NextRemaining, Tprev).

%===%============================================================%===%
%==================define previous/next timestamps:==================%
previousTimeStamp(T, Timestamps, Tprev):- Tprev is T - 1, Tprev > 0, member(Tprev, Timestamps).
nextTimeStamp(T, Timestamps, Tnext):- T > 0, Tnext is T + 1, member(Tprev, Timestamps).

"""

In [16]:
model = """
% Define timestamps
allTimeStamps([0, 8, 16, 24, 32]).

% Initial state
holdsAt_(location = kitchen, 0).
holdsAt_(light = off, 0).

% Events
happens(move(living_room), 8).
happens(switch_on, 16).

% Event effects
initiatedAt(location = L, T) :- 
  happens(move(L), T).

initiatedAt(light = on, T) :- 
  happens(switch_on, T),
  holdsAt(location = L, T),
  has_light(L).

% Domain knowledge
has_light(kitchen).
has_light(living_room).
query(holdsAt(location = L, 16)).
query(holdsAt(light = on, 24)).
"""
prolog_reasoning(event_defs_sound+prob_ec_cached+model)

ProbLog Inference Result：
holdsAt(location=living_room,16) = 1.0000
holdsAt(light=on,24) = 1.0000


In [108]:
query_list = '\n'.join([f"query(initAtEvent(sequence=1, {i}))." for i in range(81)])
all_time_stamps = f"allTimeStamps([{','.join([str(i) for i in range(81)])}])."
print(query_list, all_time_stamps)

query(initAtEvent(sequence=1, 0)).
query(initAtEvent(sequence=1, 1)).
query(initAtEvent(sequence=1, 2)).
query(initAtEvent(sequence=1, 3)).
query(initAtEvent(sequence=1, 4)).
query(initAtEvent(sequence=1, 5)).
query(initAtEvent(sequence=1, 6)).
query(initAtEvent(sequence=1, 7)).
query(initAtEvent(sequence=1, 8)).
query(initAtEvent(sequence=1, 9)).
query(initAtEvent(sequence=1, 10)).
query(initAtEvent(sequence=1, 11)).
query(initAtEvent(sequence=1, 12)).
query(initAtEvent(sequence=1, 13)).
query(initAtEvent(sequence=1, 14)).
query(initAtEvent(sequence=1, 15)).
query(initAtEvent(sequence=1, 16)).
query(initAtEvent(sequence=1, 17)).
query(initAtEvent(sequence=1, 18)).
query(initAtEvent(sequence=1, 19)).
query(initAtEvent(sequence=1, 20)).
query(initAtEvent(sequence=1, 21)).
query(initAtEvent(sequence=1, 22)).
query(initAtEvent(sequence=1, 23)).
query(initAtEvent(sequence=1, 24)).
query(initAtEvent(sequence=1, 25)).
query(initAtEvent(sequence=1, 26)).
query(initAtEvent(sequence=1, 27)).
qu

In [83]:
happenat = """
happensAt(14564, 0).
happensAt(40314, 1).
happensAt(451, 2).
happensAt(56610, 3).
happensAt(49840, 4).
happensAt(27805, 5).
happensAt(42002, 6).
happensAt(51978, 7).
happensAt(18488, 8).
happensAt(44482, 9).
happensAt(53328, 10).
happensAt(24133, 11).
happensAt(13474, 12).
happensAt(43053, 13).
happensAt(7281, 14).
happensAt(95, 15).
happensAt(21903, 16).
happensAt(14859, 17).
happensAt(48896, 18).
happensAt(3560, 19).
happensAt(2708, 20).
happensAt(39603, 21).
happensAt(7599, 22).
happensAt(22248, 23).
happensAt(51052, 24).
happensAt(40058, 25).
happensAt(1354, 26).
happensAt(3743, 27).
happensAt(46507, 28).
happensAt(6847, 29).
happensAt(40348, 30).
happensAt(9983, 31).
happensAt(17535, 32).
happensAt(205, 33).
happensAt(17669, 34).
happensAt(53326, 35).
happensAt(609, 36).
happensAt(36265, 37).
happensAt(27018, 38).
happensAt(53489, 39).
happensAt(7022, 40).
happensAt(31154, 41).
happensAt(58887, 42).
happensAt(46916, 43).
happensAt(19446, 44).
happensAt(22213, 45).
happensAt(10068, 46).
happensAt(8489, 47).
happensAt(47346, 48).
happensAt(47721, 49).
happensAt(32151, 50).
happensAt(48227, 51).
happensAt(42180, 52).
happensAt(29623, 53).
happensAt(28593, 54).
happensAt(5393, 55).
happensAt(21364, 56).
happensAt(7685, 57).
happensAt(18603, 58).
happensAt(9003, 59).
happensAt(50328, 60).
happensAt(49770, 61).
happensAt(41333, 62).
happensAt(45690, 63).
happensAt(42276, 64).
happensAt(31272, 65).
happensAt(4790, 66).
happensAt(1131, 67).
happensAt(16325, 68).
happensAt(25729, 69).
happensAt(33079, 70).
happensAt(42981, 71).
happensAt(4013, 72).
happensAt(39975, 73).
happensAt(6208, 74).
happensAt(30606, 75).
happensAt(47647, 76).
happensAt(42871, 77).
happensAt(51831, 78).
happensAt(39207, 79).
happensAt(58276, 80).
"""
digit = """
digit(14564, 0).
digit(40314, 1).
digit(451, 0).
digit(56610, 1).
digit(49840, 1).
digit(27805, 0).
digit(42002, 1).
digit(51978, 0).
digit(18488, 1).
digit(44482, 1).
digit(53328, 0).
digit(24133, 1).
digit(13474, 1).
digit(43053, 1).
digit(7281, 0).
digit(95, 0).
digit(21903, 0).
digit(14859, 1).
digit(48896, 1).
digit(3560, 1).
digit(2708, 1).
digit(39603, 1).
digit(7599, 1).
digit(22248, 1).
digit(51052, 0).
digit(40058, 1).
digit(1354, 1).
digit(3743, 0).
digit(46507, 1).
digit(6847, 0).
digit(40348, 1).
digit(9983, 0).
digit(17535, 0).
digit(205, 1).
digit(17669, 1).
digit(53326, 1).
digit(609, 1).
digit(36265, 1).
digit(27018, 1).
digit(53489, 1).
digit(7022, 1).
digit(31154, 0).
digit(58887, 1).
digit(46916, 1).
digit(19446, 1).
digit(22213, 0).
digit(10068, 1).
digit(8489, 0).
digit(47346, 0).
digit(47721, 1).
digit(32151, 0).
digit(48227, 0).
digit(42180, 0).
digit(29623, 0).
digit(28593, 0).
digit(5393, 0).
digit(21364, 1).
digit(7685, 0).
digit(18603, 0).
digit(9003, 0).
digit(50328, 0).
digit(49770, 0).
digit(41333, 1).
digit(45690, 1).
digit(42276, 1).
digit(31272, 0).
digit(4790, 1).
digit(1131, 1).
digit(16325, 1).
digit(25729, 0).
digit(33079, 0).
digit(42981, 0).
digit(4013, 1).
digit(39975, 0).
digit(6208, 1).
digit(30606, 1).
digit(47647, 0).
digit(42871, 0).
digit(51831, 0).
digit(39207, 0).
digit(58276, 1).
digit(7858, 0).
digit(45853, 0).
digit(59487, 0).
digit(33675, 1).
digit(31470, 0).
digit(2197, 1).
digit(51250, 1).
digit(40428, 0).
digit(29469, 0).
digit(7951, 1).
digit(52325, 1).
digit(54900, 0).
digit(21266, 0).
digit(28291, 1).
digit(53297, 0).
digit(745, 0).
digit(29388, 0).
digit(45754, 1).
digit(1605, 0).
digit(49488, 1).
digit(30170, 0).
digit(53917, 0).
digit(21741, 1).
digit(9048, 0).
digit(30384, 0).
digit(43933, 1).
digit(34264, 1).
digit(14157, 0).
digit(32184, 0).
digit(9297, 0).
digit(32019, 1).
digit(31771, 1).
digit(3213, 0).
digit(14321, 1).
digit(17695, 1).
digit(16595, 0).
digit(46539, 1).
digit(40790, 1).
digit(50205, 0).
digit(7410, 1).
digit(27637, 0).
digit(49186, 1).
digit(7716, 1).
digit(9628, 1).
digit(39897, 0).
digit(13657, 1).
digit(28187, 1).
digit(3468, 1).
digit(48755, 0).
digit(14371, 1).
digit(15311, 0).
digit(21376, 1).
digit(54061, 1).
digit(34999, 0).
digit(59012, 0).
digit(6003, 1).
digit(52569, 1).
digit(41649, 1).
digit(43615, 1).
digit(174, 1).
digit(31250, 1).
digit(35908, 1).
digit(4889, 0).
digit(52817, 0).
digit(6737, 0).
digit(40122, 0).
digit(59154, 0).
digit(5090, 1).
digit(31916, 1).
digit(23493, 1).
digit(12387, 0).
digit(19517, 1).
digit(57727, 0).
digit(29055, 0).
digit(40115, 0).
digit(1860, 1).
digit(18132, 0).
digit(11860, 0).
digit(40861, 1).
digit(678, 1).
digit(46596, 0).
digit(326, 0).
digit(37335, 0).
digit(34356, 0).
digit(14509, 1).
digit(51206, 1).
digit(16518, 0).
digit(22640, 0).
digit(43671, 1).
digit(55727, 0).
digit(12662, 0).
digit(10348, 0).
digit(27629, 1).
digit(47150, 0).
digit(40843, 0).
digit(3937, 0).
digit(32306, 1).
digit(5571, 1).
digit(28305, 0).
digit(37738, 0).
digit(1051, 1).
digit(35739, 0).
digit(38946, 0).
digit(57888, 1).
digit(6611, 0).
digit(23505, 1).
digit(21202, 1).
digit(2373, 0).
digit(31478, 1).
digit(9130, 1).
digit(9378, 1).
digit(1787, 1).
digit(2582, 0).
digit(10364, 1).
digit(50424, 0).
digit(10456, 0).
digit(8095, 1).
digit(7671, 1).
digit(53429, 0).
digit(34451, 0).
digit(48142, 0).
digit(59233, 1).
digit(51249, 1).
digit(30709, 0).
digit(32158, 1).
digit(21479, 1).
digit(6423, 1).
digit(9675, 0).
digit(19547, 0).
digit(19735, 1).
digit(27578, 1).
digit(8332, 1).
digit(55140, 1).
digit(19786, 0).
digit(39550, 1).
digit(36165, 1).
digit(17476, 0).
digit(28946, 0).
digit(53966, 1).
digit(40859, 0).
digit(14353, 1).
digit(4267, 1).
digit(11105, 0).
digit(59911, 0).
digit(35359, 1).
digit(22879, 1).
digit(6565, 1).
digit(9256, 1).
digit(52780, 1).
digit(18159, 0).
digit(22453, 0).
digit(32335, 1).
digit(43188, 0).
digit(59882, 1).
digit(33023, 0).
digit(5963, 1).
digit(31243, 0).
digit(18424, 0).
digit(3490, 0).
digit(45401, 0).
digit(11562, 0).
digit(52885, 0).
digit(21253, 1).
digit(14021, 1).
digit(26438, 0).
digit(51369, 1).
digit(42759, 0).
digit(31350, 0).
digit(53027, 0).
digit(43085, 1).
digit(7472, 1).
digit(28331, 1).
digit(5747, 0).
digit(15999, 1).
digit(49878, 1).
digit(42179, 1).
digit(26598, 1).
digit(49397, 1).
digit(35481, 0).
digit(36351, 0).
digit(4507, 1).
digit(3545, 1).
digit(14698, 1).
digit(40879, 1).
digit(41555, 1).
digit(5665, 0).
digit(8418, 1).
digit(59488, 1).
digit(36022, 0).
digit(54783, 0).
digit(19550, 0).
digit(5006, 1).
digit(27123, 1).
digit(26881, 0).
digit(779, 1).
digit(10098, 1).
digit(46332, 0).
digit(47241, 0).
digit(51672, 1).
digit(48136, 0).
digit(8330, 1).
digit(27639, 1).
digit(53124, 0).
digit(31780, 1).
digit(57595, 0).
digit(30009, 0).
digit(59116, 1).
digit(4985, 0).
digit(52526, 0).
digit(47054, 1).
digit(40002, 0).
digit(53899, 1).
digit(25626, 0).
digit(8373, 1).
digit(53471, 1).
digit(30089, 1).
digit(52075, 1).
digit(56541, 1).
digit(40825, 0).
digit(35936, 0).
digit(33048, 1).
digit(13450, 0).
digit(20556, 1).
digit(27082, 0).
digit(9963, 0).
digit(30625, 0).
digit(7668, 1).
digit(24696, 0).
digit(53924, 0).
digit(46892, 0).
digit(40565, 0).
digit(18371, 0).
digit(5147, 0).
digit(13514, 1).
digit(26986, 1).
digit(9085, 1).
digit(15539, 1).
digit(16305, 0).
digit(7491, 1).
digit(309, 1).
digit(6647, 1).
digit(57491, 0).
digit(17365, 0).
digit(19614, 1).
digit(29934, 0).
digit(19440, 1).
digit(37261, 0).
digit(21709, 1).
digit(14516, 1).
digit(51024, 0).
digit(3440, 1).
digit(21970, 0).
digit(58492, 0).
digit(47993, 1).
digit(51483, 1).
digit(11902, 0).
digit(46792, 1).
digit(18695, 1).
digit(48831, 1).
digit(56991, 1).
digit(59820, 1).
digit(41959, 1).
digit(24681, 0).
digit(46196, 1).
digit(55588, 0).
digit(54101, 0).
digit(46185, 1).
digit(2806, 0).
digit(52429, 0).
digit(10416, 1).
digit(35327, 0).
digit(31245, 1).
digit(25124, 1).
digit(33536, 0).
digit(45076, 1).
digit(30471, 1).
digit(20533, 0).
digit(28965, 0).
digit(17213, 1).
digit(40362, 1).
digit(38629, 1).
digit(47668, 1).
digit(44893, 0).
digit(16363, 0).
digit(19006, 0).
digit(49354, 0).
digit(59337, 0).
digit(54393, 1).
digit(49199, 0).
digit(11440, 1).
digit(16546, 1).
digit(29428, 1).
digit(21889, 1).
digit(15542, 0).
digit(56159, 0).
digit(14651, 1).
digit(44208, 0).
digit(17928, 1).
digit(7305, 0).
digit(23250, 0).
digit(46294, 0).
digit(39701, 0).
digit(11379, 0).
digit(19782, 1).
digit(49576, 0).
digit(37047, 0).
digit(1879, 1).
digit(54083, 0).
digit(26780, 1).
digit(59619, 1).
digit(44900, 0).
digit(19091, 1).
digit(6970, 0).
digit(49246, 1).
digit(57749, 1).
digit(21910, 0).
digit(16367, 1).
digit(50151, 0).
digit(49374, 0).
digit(4333, 1).
digit(54293, 1).
digit(18892, 0).
digit(8725, 1).
digit(50862, 0).
digit(30548, 0).
digit(15854, 0).
digit(52995, 0).
digit(24328, 1).
digit(33292, 0).
digit(9315, 1).
digit(51434, 0).
digit(20332, 1).
digit(59773, 1).
digit(39797, 1).
digit(28516, 1).
digit(32079, 1).
digit(38164, 1).
digit(39095, 1).
digit(51714, 1).
digit(31145, 0).
digit(6380, 1).
digit(44335, 0).
digit(41352, 0).
digit(20887, 1).
digit(7471, 0).
digit(5123, 1).
digit(23348, 1).
digit(54753, 1).
digit(7876, 1).
digit(18137, 0).
digit(28063, 1).
digit(23115, 1).
digit(34402, 1).
digit(12500, 0).
digit(45579, 1).
digit(48176, 0).
digit(21477, 1).
digit(49042, 1).
digit(10705, 0).
digit(4710, 1).
digit(23795, 1).
digit(55080, 1).
digit(35565, 1).
digit(53897, 0).
digit(15946, 0).
digit(7037, 0).
digit(30894, 1).
digit(36849, 0).
digit(42666, 1).
digit(59112, 1).
digit(56424, 0).
digit(26087, 1).
digit(14998, 1).
digit(56744, 1).
digit(34493, 1).
digit(52965, 1).
digit(54329, 1).
digit(4607, 0).
digit(8664, 1).
digit(19640, 1).
digit(2436, 0).
digit(34017, 1).
digit(32789, 0).
digit(38599, 0).
digit(40153, 1).
digit(4976, 1).
digit(51651, 1).
digit(43847, 0).
digit(2393, 1).
digit(39173, 1).
digit(38128, 0).
digit(41332, 1).
digit(52036, 0).
digit(7948, 0).
digit(52745, 1).
digit(16136, 0).
digit(57843, 0).
digit(4676, 1).
digit(56459, 0).
digit(58362, 0).
digit(33634, 1).
digit(59353, 0).
digit(42848, 0).
digit(8705, 1).
digit(51308, 0).
digit(49996, 0).
digit(11533, 1).
digit(46334, 1).
digit(19816, 1).
digit(48175, 1).
digit(18360, 1).
digit(12839, 0).
digit(45349, 0).
digit(5246, 0).
digit(53498, 1).
digit(14756, 1).
digit(43316, 1).
digit(52720, 0).
digit(44531, 0).
digit(12743, 1).
digit(54683, 1).
digit(8648, 1).
digit(35907, 1).
digit(29895, 0).
digit(11093, 1).
digit(14685, 1).
digit(37953, 0).
digit(23910, 1).
digit(45723, 1).
digit(32773, 1).
digit(44079, 1).
digit(25779, 1).
digit(40954, 1).
digit(41826, 1).
digit(50070, 0).
digit(53577, 1).
digit(29341, 0).
digit(13371, 0).
digit(25036, 1).
digit(42433, 0).
digit(59498, 1).
digit(59052, 1).
digit(959, 1).
digit(52444, 1).
digit(57552, 1).
digit(9506, 0).
digit(50709, 1).
digit(42584, 1).
digit(46077, 1).
digit(53354, 0).
digit(16887, 1).
digit(42005, 0).
digit(24715, 0).
digit(31323, 1).
digit(28130, 1).
digit(19845, 0).
digit(39049, 1).
digit(20688, 0).
digit(3715, 0).
digit(17376, 0).
digit(50829, 1).
digit(46823, 1).
digit(5204, 1).
digit(39623, 0).
digit(31265, 1).
digit(2864, 0).
digit(35285, 1).
digit(15865, 1).
digit(26487, 1).
digit(4128, 1).
digit(54010, 0).
digit(6209, 1).
digit(46954, 1).
digit(18583, 1).
digit(42776, 0).
digit(7932, 0).
digit(22294, 0).
digit(16080, 1).
digit(5398, 0).
digit(45816, 1).
digit(42243, 1).
digit(50293, 1).
digit(57326, 0).
digit(48273, 1).
digit(20817, 1).
digit(8509, 0).
digit(15138, 1).
digit(52639, 0).
digit(23009, 0).
digit(23823, 0).
digit(2375, 1).
digit(9525, 1).
digit(36288, 1).
digit(13589, 0).
digit(40074, 0).
digit(37977, 0).
digit(15224, 1).
digit(33142, 0).
digit(37043, 0).
digit(41877, 0).
digit(19917, 1).
digit(24839, 1).
digit(59836, 1).
digit(59165, 0).
digit(29918, 0).
digit(11116, 0).
digit(1046, 0).
digit(2051, 0).
digit(18963, 1).
digit(13076, 0).
digit(1403, 0).
digit(13075, 0).
digit(39462, 1).
digit(11026, 0).
digit(11041, 0).
digit(47224, 1).
digit(52916, 1).
digit(34869, 1).
digit(19526, 1).
digit(15700, 0).
digit(31001, 1).
digit(44788, 0).
digit(54675, 1).
digit(4729, 0).
digit(14857, 0).
digit(44276, 0).
digit(23647, 1).
digit(29403, 1).
digit(35999, 0).
digit(3113, 1).
digit(50225, 1).
digit(41401, 0).
digit(42058, 1).
digit(27401, 0).
digit(25024, 0).
digit(11933, 0).
digit(53018, 1).
digit(351, 1).
digit(35294, 0).
digit(51067, 1).
digit(32411, 1).
digit(33268, 1).
digit(29074, 0).
digit(19325, 0).
digit(30721, 1).
digit(47867, 0).
digit(5256, 1).
digit(21288, 1).
digit(56336, 1).
digit(14559, 0).
digit(28238, 1).
digit(5808, 0).
digit(21220, 1).
digit(20465, 1).
digit(41922, 0).
digit(11308, 1).
digit(949, 0).
digit(41756, 0).
digit(22432, 0).
digit(54865, 0).
digit(22170, 1).
digit(19774, 0).
digit(11248, 1).
digit(56668, 1).
digit(29119, 0).
digit(27484, 0).
digit(34334, 1).
digit(50407, 0).
digit(47875, 1).
digit(35236, 1).
digit(34511, 0).
digit(22343, 1).
digit(49065, 0).
digit(104, 1).
digit(10340, 0).
digit(20371, 0).
digit(45589, 0).
digit(33741, 1).
digit(47482, 0).
digit(40443, 1).
digit(59849, 1).
digit(22981, 0).
digit(16686, 1).
digit(46326, 1).
digit(31180, 0).
digit(29926, 0).
digit(58266, 1).
digit(3396, 0).
digit(8289, 0).
digit(51833, 1).
digit(45559, 1).
digit(57053, 0).
digit(53632, 0).
digit(25112, 1).
digit(55217, 0).
digit(7710, 0).
digit(55819, 1).
digit(51365, 0).
digit(14016, 1).
digit(42806, 1).
digit(58051, 1).
digit(42915, 0).
digit(38163, 1).
digit(31082, 0).
digit(7060, 1).
digit(40310, 1).
digit(9674, 1).
digit(33464, 0).
digit(58677, 1).
digit(19997, 1).
digit(56606, 0).
digit(41713, 0).
digit(43624, 0).
digit(53178, 1).
digit(35661, 1).
digit(10686, 1).
digit(38585, 1).
digit(14945, 1).
digit(43347, 0).
digit(50024, 0).
digit(25085, 0).
digit(31179, 1).
digit(28650, 1).
digit(27276, 1).
digit(5608, 0).
digit(9415, 1).
digit(13719, 0).
digit(41454, 1).
digit(5189, 1).
digit(20699, 1).
digit(25199, 0).
digit(22375, 0).
digit(58072, 1).
digit(56158, 0).
digit(17503, 0).
digit(21697, 1).
digit(2709, 0).
digit(1211, 1).
digit(17488, 1).
digit(702, 0).
digit(6570, 1).
digit(14445, 0).
digit(6422, 0).
digit(7012, 0).
digit(45608, 0).
digit(49628, 1).
digit(10409, 1).
digit(1876, 0).
digit(3612, 0).
digit(44103, 0).
digit(43932, 0).
digit(20536, 1).
digit(11409, 0).
digit(54468, 1).
digit(42260, 1).
digit(31556, 1).
digit(33705, 1).
digit(10261, 0).
digit(5927, 0).
digit(46601, 1).
digit(20662, 0).
digit(7561, 1).
digit(4265, 0).
digit(10664, 0).
digit(22225, 0).
digit(37968, 0).
digit(1606, 0).
digit(871, 1).
digit(37045, 1).
digit(30800, 0).
digit(15397, 1).
digit(3587, 1).
digit(22441, 0).
digit(14393, 0).
digit(44967, 1).
digit(20713, 0).
digit(58160, 0).
digit(56714, 1).
digit(2826, 0).
digit(8885, 0).
digit(57855, 1).
digit(50192, 0).
digit(14879, 1).
digit(42486, 1).
digit(18486, 0).
digit(49924, 0).
digit(11501, 0).
digit(41056, 0).
digit(17618, 1).
digit(45189, 0).
digit(9710, 0).
digit(18219, 0).
digit(15191, 0).
digit(18794, 0).
digit(5646, 0).
digit(16817, 0).
digit(25285, 0).
digit(41986, 1).
digit(27572, 1).
digit(36255, 0).
digit(46058, 1).
digit(42818, 0).
digit(57558, 0).
digit(6851, 1).
digit(2100, 0).
digit(6070, 1).
digit(2493, 0).
digit(20212, 1).
digit(55646, 0).
digit(24189, 0).
digit(34261, 1).
digit(58206, 0).
digit(16021, 1).
digit(7566, 0).
digit(9047, 0).
digit(4186, 1).
digit(9485, 0).
digit(15160, 0).
digit(1463, 1).
digit(49265, 1).
digit(21168, 1).
digit(58952, 0).
digit(47197, 0).
digit(45146, 0).
digit(59433, 0).
digit(9973, 1).
digit(55150, 1).
digit(39874, 0).
digit(59840, 0).
digit(22405, 0).
digit(39785, 1).
digit(34555, 0).
digit(14070, 1).
digit(2316, 1).
digit(44770, 1).
digit(47172, 0).
digit(36168, 0).
digit(15288, 0).
digit(52508, 0).
digit(22529, 0).
digit(114, 0).
digit(42932, 1).
digit(50631, 1).
digit(30189, 1).
digit(19399, 0).
digit(17124, 0).
digit(12764, 0).
digit(55169, 1).
digit(45942, 0).
digit(47190, 0).
digit(8723, 0).
digit(56715, 0).
digit(6146, 0).
digit(32986, 1).
digit(4962, 0).
digit(58685, 1).
digit(17941, 0).
digit(57073, 0).
digit(40171, 0).
digit(48611, 1).
digit(12521, 1).
digit(3546, 1).
digit(56559, 1).
digit(2889, 1).
digit(26535, 1).
digit(52318, 0).
digit(36042, 1).
digit(4455, 0).
digit(31597, 1).
digit(57673, 1).
digit(51554, 1).
digit(30711, 1).
digit(54947, 1).
digit(33433, 1).
digit(33776, 0).
digit(39620, 0).
digit(872, 0).
digit(54764, 0).
digit(17845, 1).
digit(44724, 1).
digit(59060, 1).
digit(49949, 0).
digit(4106, 0).
digit(21932, 0).
digit(22705, 1).
digit(17888, 1).
digit(2974, 0).
digit(5169, 1).
digit(6880, 1).
digit(40273, 1).
digit(17433, 1).
digit(37663, 0).
digit(31633, 1).
digit(11156, 1).
digit(156, 0).
digit(7905, 0).
digit(23589, 0).
digit(58557, 1).
digit(30404, 1).
digit(47078, 0).
digit(18568, 1).
digit(21362, 1).
digit(3145, 1).
digit(18507, 0).
digit(7574, 1).
digit(45276, 0).
digit(440, 0).
digit(31701, 1).
digit(29679, 1).
digit(10859, 0).
digit(54711, 0).
digit(32036, 0).
digit(55259, 0).
digit(12412, 1).
digit(30848, 1).
digit(46375, 0).
digit(19452, 1).
digit(23521, 0).
digit(26991, 1).
digit(18858, 0).
digit(833, 1).
digit(54391, 0).
digit(3567, 1).
digit(35831, 1).
digit(40200, 0).
digit(34679, 1).
digit(36317, 1).
digit(45307, 0).
digit(39334, 0).
digit(13711, 1).
digit(47373, 1).
digit(21301, 1).
digit(56239, 0).
digit(24640, 0).
digit(44650, 1).
digit(47773, 1).
digit(32399, 0).
digit(36442, 1).
digit(38835, 0).
digit(48419, 0).
digit(40042, 0).
digit(44225, 0).
digit(59256, 0).
digit(17943, 1).
digit(8319, 1).
digit(5729, 0).
digit(18781, 1).
digit(7746, 1).
digit(8219, 1).
digit(24755, 0).
digit(59918, 0).
digit(41570, 0).
digit(50585, 1).
digit(3585, 0).
digit(128, 1).
digit(12209, 1).
digit(52342, 1).
digit(11214, 1).
digit(18658, 0).
digit(14041, 1).
digit(20863, 0).
digit(51666, 1).
digit(23289, 0).
digit(52228, 0).
digit(56929, 0).
digit(38824, 1).
digit(15649, 1).
digit(49097, 0).
digit(5732, 1).
digit(648, 1).
digit(1915, 1).
digit(34623, 0).
digit(3831, 1).
digit(25364, 1).
digit(53539, 1).
digit(16691, 1).
digit(22658, 0).
digit(21526, 0).
digit(46139, 0).
digit(20691, 1).
digit(59847, 0).
digit(53573, 0).
digit(1090, 0).
digit(54651, 1).
digit(29587, 1).
digit(24115, 1).
digit(13748, 1).
digit(59272, 0).
digit(5955, 1).
digit(25433, 0).
digit(11848, 0).
digit(56980, 1).
digit(11683, 0).
digit(42157, 0).
digit(28044, 0).
digit(2269, 0).
digit(43423, 0).
digit(47670, 1).
digit(42113, 1).
digit(35596, 1).
digit(40821, 0).
digit(17607, 1).
digit(51395, 0).
digit(57682, 0).
digit(9507, 1).
digit(32463, 0).
digit(54876, 1).
digit(27550, 0).
digit(55499, 1).
digit(41728, 1).
digit(45218, 0).
digit(18970, 0).
digit(26485, 1).
digit(11975, 1).
digit(51892, 0).
digit(33480, 1).
digit(28888, 0).
digit(33923, 0).
digit(780, 1).
digit(52741, 1).
digit(15926, 0).
digit(15992, 0).
digit(12845, 1).
digit(6785, 1).
digit(29242, 1).
digit(33066, 0).
digit(18993, 1).
digit(48441, 1).
digit(40769, 1).
digit(22706, 0).
digit(44899, 1).
digit(6987, 0).
digit(38523, 0).
digit(25039, 0).
digit(13344, 0).
digit(24509, 1).
digit(17465, 1).
digit(45371, 0).
digit(38559, 0).
digit(54380, 1).
digit(26944, 1).
digit(1837, 0).
digit(25322, 1).
digit(53575, 0).
digit(12310, 1).
digit(17485, 0).
digit(54414, 0).
digit(35558, 1).
digit(41614, 0).
digit(3605, 1).
digit(23613, 0).
digit(24838, 1).
digit(58599, 0).
digit(26277, 0).
digit(42578, 1).
digit(4685, 1).
digit(52715, 1).
digit(8185, 1).
digit(14498, 1).
digit(41795, 0).
digit(34273, 0).
digit(1699, 1).
digit(14039, 0).
digit(7105, 1).
digit(31625, 0).
digit(28755, 0).
digit(12872, 0).
digit(52731, 1).
digit(13751, 0).
digit(25443, 1).
digit(9703, 0).
digit(43984, 1).
digit(39936, 1).
digit(45975, 0).
digit(43197, 1).
digit(58427, 1).
digit(662, 0).
digit(3321, 1).
digit(618, 1).
digit(42786, 1).
digit(1080, 1).
digit(58745, 1).
digit(28568, 1).
digit(36626, 1).
digit(6763, 0).
digit(2121, 0).
digit(52354, 0).
digit(9365, 1).
digit(2526, 0).
digit(7330, 1).
digit(46436, 0).
digit(27011, 1).
digit(29887, 0).
digit(18073, 0).
digit(40249, 0).
digit(5888, 0).
digit(44411, 0).
digit(45650, 1).
digit(39857, 1).
digit(38868, 1).
digit(4316, 0).
digit(52821, 0).
digit(18256, 1).
digit(52758, 1).
digit(25054, 1).
digit(59796, 1).
digit(53789, 0).
digit(27137, 0).
digit(35994, 0).
digit(24782, 1).
digit(7406, 1).
digit(55095, 0).
digit(51826, 0).
digit(17466, 0).
digit(14964, 1).
digit(59606, 1).
digit(16392, 1).
digit(3131, 0).
digit(32315, 1).
digit(22644, 1).
digit(34641, 1).
digit(18606, 0).
digit(36848, 0).
digit(43336, 1).
digit(18896, 0).
digit(36223, 0).
digit(4918, 0).
digit(59940, 0).
digit(27380, 0).
digit(19394, 1).
digit(40454, 0).
digit(49370, 0).
digit(16233, 1).
digit(47716, 0).
digit(26728, 0).
digit(2040, 1).
digit(7548, 0).
digit(2245, 0).
digit(23889, 1).
digit(49807, 1).
digit(11604, 1).
digit(1230, 1).
digit(40400, 0).
digit(15002, 1).
digit(32486, 1).
digit(39656, 0).
digit(56103, 0).
digit(48719, 0).
digit(34199, 1).
digit(17285, 0).
digit(905, 1).
digit(58763, 1).
digit(20365, 1).
digit(656, 0).
digit(5981, 0).
digit(54809, 1).
digit(31570, 1).
digit(35600, 1).
digit(46860, 0).
digit(36764, 0).
digit(4764, 1).
digit(42537, 1).
digit(15901, 0).
digit(30963, 1).
digit(58935, 1).
digit(4069, 1).
digit(20010, 0).
digit(51587, 0).
digit(21090, 1).
digit(31865, 0).
digit(42224, 0).
digit(1857, 0).
digit(28349, 1).
digit(34070, 0).
digit(4950, 1).
digit(12421, 0).
digit(17519, 0).
digit(57410, 1).
digit(40779, 1).
digit(28546, 0).
digit(46360, 1).
digit(48363, 1).
digit(41409, 1).
digit(3943, 0).
digit(25980, 0).
digit(18893, 0).
digit(13542, 1).
digit(47488, 0).
digit(6034, 0).
digit(18461, 0).
digit(23318, 1).
digit(40456, 0).
digit(42038, 0).
digit(3337, 0).
digit(50493, 1).
digit(8063, 1).
digit(27343, 0).
digit(37059, 0).
digit(42446, 0).
digit(32890, 1).
digit(16905, 0).
digit(17429, 1).
digit(13674, 1).
digit(54835, 0).
digit(35778, 0).
digit(9196, 0).
digit(19241, 0).
digit(54342, 1).
digit(50127, 0).
digit(19490, 0).
digit(32472, 1).
digit(56110, 1).
digit(41288, 0).
digit(39037, 0).
digit(58286, 1).
digit(48189, 1).
digit(22583, 0).
digit(15428, 1).
digit(2333, 0).
digit(46880, 1).
digit(58486, 1).
digit(15435, 1).
digit(52470, 0).
digit(10048, 1).
digit(49441, 1).
digit(35067, 0).
digit(54089, 0).
digit(31318, 1).
digit(55541, 0).
digit(30891, 0).
digit(56556, 0).
digit(39403, 1).
digit(27247, 0).
digit(11630, 1).
digit(26099, 1).
digit(47096, 0).
digit(39033, 1).
digit(38455, 1).
digit(19859, 0).
digit(2134, 1).
digit(36788, 0).
digit(48741, 1).
digit(57575, 0).
digit(41620, 0).
digit(46166, 0).
digit(26537, 1).
digit(46746, 0).
digit(55567, 1).
digit(11249, 1).
digit(50918, 1).
digit(15633, 1).
digit(47820, 0).
digit(23536, 0).
digit(46459, 0).
digit(50723, 0).
digit(7089, 1).
digit(40516, 1).
digit(44448, 0).
digit(57887, 0).
digit(57969, 1).
digit(58809, 0).
digit(9342, 0).
digit(51836, 1).
digit(48444, 1).
digit(37495, 0).
digit(38965, 1).
digit(19104, 0).
digit(22208, 0).
digit(4763, 1).
digit(49284, 1).
digit(5373, 0).
digit(26261, 0).
digit(52831, 0).
digit(43315, 1).
digit(39710, 0).
digit(20947, 0).
digit(23342, 1).
digit(34958, 0).
digit(29368, 0).
digit(16498, 0).
digit(48434, 0).
digit(24940, 0).
digit(24346, 0).
digit(31174, 1).
digit(35116, 1).
digit(38718, 1).
digit(57390, 0).
digit(43553, 1).
digit(36240, 1).
digit(45140, 1).
digit(11157, 1).
digit(8193, 0).
digit(39233, 0).
digit(19649, 0).
digit(46159, 1).
digit(2410, 1).
digit(46840, 1).
digit(290, 1).
digit(40359, 1).
digit(40820, 0).
digit(32070, 1).
digit(57385, 1).
digit(54103, 1).
digit(4623, 1).
digit(41524, 1).
digit(49932, 1).
digit(19643, 1).
digit(24624, 1).
digit(53399, 0).
digit(39072, 1).
digit(39782, 1).
digit(27792, 1).
digit(15758, 1).
digit(38335, 0).
digit(28945, 0).
digit(45666, 1).
digit(9840, 0).
digit(31908, 1).
digit(41320, 0).
digit(3004, 1).
digit(13347, 0).
digit(29976, 1).
digit(23279, 0).
digit(52418, 1).
digit(38332, 0).
digit(15265, 1).
digit(31494, 1).
digit(34625, 1).
digit(21141, 0).
digit(26238, 1).
digit(34158, 1).
digit(4920, 1).
digit(53351, 1).
digit(15993, 0).
digit(20461, 1).
digit(45695, 0).
digit(52613, 0).
digit(34705, 1).
digit(8046, 0).
digit(489, 0).
digit(3377, 0).
digit(28894, 1).
digit(57685, 0).
digit(47222, 1).
digit(51801, 1).
digit(577, 0).
digit(37141, 0).
digit(27707, 1).
digit(10272, 0).
digit(48631, 0).
digit(22196, 0).
digit(50219, 1).
digit(18758, 1).
digit(9837, 1).
digit(53980, 1).
digit(57650, 0).
digit(13785, 1).
digit(16345, 1).
digit(47855, 0).
digit(50051, 1).
digit(22916, 1).
digit(24738, 1).
digit(3309, 0).
digit(16412, 0).
digit(524, 0).
digit(13005, 0).
digit(54485, 1).
digit(24388, 0).
digit(58672, 1).
digit(53087, 1).
digit(28420, 1).
digit(14575, 1).
digit(56219, 0).
digit(41259, 0).
digit(50018, 0).
digit(30667, 1).
digit(633, 0).
digit(22659, 1).
digit(55222, 1).
digit(29809, 0).
digit(10020, 1).
digit(41222, 1).
digit(40335, 0).
digit(17693, 0).
digit(22158, 1).
digit(25921, 0).
digit(52941, 0).
digit(11780, 0).
digit(50682, 1).
digit(492, 1).
digit(1310, 0).
digit(2894, 1).
digit(25269, 1).
digit(8113, 0).
digit(16265, 1).
digit(46818, 0).
digit(49342, 0).
digit(21878, 1).
digit(39106, 1).
digit(11561, 1).
digit(34777, 1).
digit(39737, 1).
digit(37463, 1).
digit(50472, 1).
digit(31769, 0).
digit(15878, 1).
digit(27270, 1).
digit(30854, 1).
digit(42098, 1).
digit(40664, 1).
digit(33750, 1).
digit(5903, 1).
digit(46484, 0).
digit(12693, 1).
digit(36146, 0).
digit(27262, 1).
digit(31134, 1).
digit(38099, 0).
digit(26495, 1).
digit(17704, 1).
digit(44513, 0).
digit(16961, 0).
digit(4655, 0).
digit(46141, 1).
digit(11342, 1).
digit(209, 0).
digit(47194, 1).
digit(32153, 1).
digit(22159, 0).
digit(8223, 0).
digit(41852, 1).
digit(13066, 0).
digit(56573, 0).
digit(29812, 1).
digit(43549, 0).
digit(46089, 1).
digit(29421, 0).
digit(36388, 1).
digit(16950, 1).
digit(34352, 0).
digit(408, 1).
digit(43444, 1).
digit(38516, 1).
digit(35710, 1).
digit(12394, 1).
digit(6411, 1).
digit(22831, 1).
digit(7050, 0).
digit(20951, 1).
digit(1963, 0).
digit(7057, 1).
digit(58973, 0).
digit(43980, 1).
digit(14464, 1).
digit(8021, 0).
digit(51019, 0).
digit(55253, 0).
digit(18482, 1).
digit(37213, 1).
digit(12671, 1).
digit(40976, 1).
digit(192, 0).
digit(16307, 1).
digit(50857, 1).
digit(56278, 1).
digit(28769, 1).
digit(24872, 0).
digit(26350, 1).
digit(22545, 1).
digit(47465, 1).
digit(12686, 0).
digit(8703, 0).
digit(57197, 1).
digit(48274, 0).
digit(6113, 0).
digit(2340, 0).
digit(38787, 1).
digit(5487, 0).
digit(37425, 0).
digit(23238, 1).
digit(41358, 0).
digit(58335, 1).
digit(37382, 0).
digit(39309, 0).
digit(39306, 0).
digit(31729, 1).
digit(22917, 0).
digit(13497, 0).
digit(21769, 0).
digit(33580, 1).
digit(42634, 0).
digit(13214, 0).
digit(12339, 0).
digit(17938, 1).
digit(43134, 0).
digit(54980, 0).
digit(58440, 1).
digit(21460, 1).
digit(12516, 0).
digit(19421, 1).
digit(56362, 0).
digit(15805, 1).
digit(53531, 1).
digit(8089, 0).
digit(6461, 0).
digit(12142, 0).
digit(18232, 0).
digit(37192, 0).
digit(54087, 1).
digit(47736, 0).
digit(32690, 0).
digit(59248, 1).
digit(41150, 1).
digit(19410, 1).
digit(48318, 1).
digit(43045, 0).
digit(9296, 0).
digit(58561, 1).
digit(15628, 0).
digit(33628, 1).
digit(13609, 0).
digit(49786, 1).
digit(44796, 0).
digit(53936, 0).
digit(13335, 0).
digit(4535, 0).
digit(30408, 1).
digit(34867, 0).
digit(20135, 1).
digit(49110, 0).
digit(23656, 1).
digit(25978, 0).
digit(1527, 1).
digit(26427, 0).
digit(37377, 0).
digit(18926, 0).
digit(5131, 0).
digit(34977, 0).
digit(28402, 0).
digit(52032, 1).
digit(9221, 1).
digit(38235, 1).
digit(23444, 1).
digit(45596, 0).
digit(12681, 1).
digit(37946, 0).
digit(45112, 0).
digit(2066, 0).
digit(37433, 1).
digit(11948, 0).
digit(23417, 1).
digit(28896, 0).
digit(3561, 1).
digit(52107, 1).
digit(47756, 0).
digit(32334, 1).
digit(27477, 0).
digit(4066, 1).
digit(34782, 1).
digit(31209, 1).
digit(5921, 0).
digit(55297, 0).
digit(11743, 1).
digit(42896, 0).
digit(46952, 0).
digit(49639, 0).
digit(5864, 1).
digit(15382, 0).
digit(36533, 0).
digit(40613, 0).
digit(6783, 1).
digit(58754, 0).
digit(11779, 0).
digit(40277, 0).
digit(6045, 0).
digit(7896, 1).
digit(12665, 0).
digit(43662, 0).
digit(28191, 1).
digit(41839, 1).
digit(10662, 1).
digit(57571, 1).
digit(40630, 0).
digit(58322, 1).
digit(45503, 0).
digit(7857, 1).
digit(15730, 1).
digit(31017, 1).
digit(16610, 1).
digit(14276, 0).
digit(14310, 1).
digit(20604, 1).
digit(54173, 1).
digit(45442, 0).
digit(40644, 1).
digit(17012, 1).
digit(18545, 1).
digit(19674, 1).
digit(28627, 0).
digit(15578, 1).
digit(25290, 0).
digit(13721, 1).
digit(7138, 1).
digit(52161, 0).
digit(7267, 0).
digit(43186, 0).
digit(31963, 1).
digit(22833, 1).
digit(28725, 0).
digit(18074, 1).
digit(46962, 0).
digit(53290, 0).
digit(4629, 0).
digit(11657, 0).
digit(20384, 1).
digit(59241, 0).
digit(10363, 0).
digit(48537, 1).
digit(14846, 1).
digit(19899, 0).
digit(38200, 0).
digit(7892, 0).
digit(5692, 0).
digit(38212, 0).
digit(7129, 0).
digit(15130, 0).
digit(50055, 1).
digit(40075, 0).
digit(37981, 0).
digit(55278, 0).
digit(9312, 1).
digit(56868, 0).
digit(41338, 0).
digit(11029, 0).
digit(26686, 0).
digit(24529, 0).
digit(34237, 0).
digit(34008, 1).
digit(28816, 0).
digit(25703, 1).
digit(28804, 1).
digit(12095, 0).
digit(38482, 1).
digit(53077, 0).
digit(5876, 1).
digit(40201, 1).
digit(42531, 1).
digit(55417, 1).
digit(5096, 0).
digit(54501, 1).
digit(14417, 1).
digit(6311, 1).
digit(4566, 1).
digit(35507, 1).
digit(26089, 1).
digit(26339, 0).
digit(320, 0).
digit(7275, 1).
digit(57661, 1).
digit(36737, 0).
digit(25436, 1).
digit(44065, 1).
digit(19504, 0).
digit(2159, 1).
digit(43849, 1).
digit(36260, 0).
digit(18692, 1).
digit(54579, 1).
digit(52381, 1).
digit(53314, 1).
digit(48807, 0).
digit(8394, 1).
digit(17905, 0).
digit(3901, 1).
digit(28250, 1).
digit(21427, 1).
digit(27028, 1).
digit(54214, 0).
digit(33983, 0).
digit(57095, 1).
digit(50830, 1).
digit(49535, 0).
digit(12798, 1).
digit(34215, 1).
digit(41384, 0).
digit(12878, 1).
digit(38106, 0).
digit(54720, 1).
digit(57322, 1).
digit(34553, 0).
digit(5840, 1).
digit(49717, 1).
digit(5915, 1).
digit(50482, 1).
digit(52136, 1).
digit(56181, 0).
digit(20845, 1).
digit(23454, 0).
digit(57778, 1).
digit(16087, 1).
digit(10025, 0).
digit(53842, 1).
digit(40610, 1).
digit(34083, 1).
digit(41748, 1).
digit(21028, 0).
digit(5054, 1).
digit(17479, 0).
digit(28394, 1).
digit(55023, 0).
digit(53375, 1).
digit(29897, 1).
digit(41209, 1).
digit(17846, 1).
digit(23943, 0).
digit(16523, 0).
digit(57192, 1).
digit(20747, 0).
digit(46571, 0).
digit(53792, 1).
digit(51960, 0).
digit(6591, 0).
digit(26292, 0).
digit(19273, 1).
digit(12113, 1).
digit(40263, 1).
digit(50526, 0).
digit(21321, 1).
digit(32536, 1).
digit(47213, 1).
digit(51252, 1).
digit(22502, 1).
digit(19768, 1).
digit(59143, 0).
digit(9907, 1).
digit(11537, 1).
digit(32336, 0).
digit(56530, 0).
digit(27148, 0).
digit(3155, 1).
digit(8050, 0).
digit(27461, 1).
digit(54489, 1).
digit(44288, 1).
digit(40714, 1).
digit(37739, 0).
digit(49268, 0).
digit(23470, 1).
digit(40577, 1).
digit(33847, 0).
digit(38484, 0).
digit(22008, 0).
digit(58819, 0).
digit(24487, 1).
digit(56857, 0).
digit(20394, 1).
digit(48268, 0).
digit(13623, 0).
digit(18801, 1).
digit(11712, 0).
digit(38532, 1).
digit(7451, 0).
digit(30137, 1).
digit(44897, 0).
digit(23093, 0).
digit(59558, 0).
digit(58207, 1).
digit(30586, 0).
digit(54165, 1).
digit(57222, 1).
digit(14723, 0).
digit(6998, 1).
digit(11432, 1).
digit(54210, 1).
digit(12512, 1).
digit(7838, 1).
digit(22899, 1).
digit(21780, 0).
digit(4751, 1).
digit(18881, 0).
digit(27531, 1).
digit(59207, 1).
digit(42819, 1).
digit(31785, 0).
digit(57170, 1).
digit(28041, 0).
digit(5176, 1).
digit(13007, 1).
digit(59419, 1).
digit(41647, 0).
digit(15564, 0).
digit(8385, 0).
digit(4012, 1).
digit(34302, 0).
digit(464, 0).
digit(30274, 1).
digit(13231, 0).
digit(9720, 0).
digit(17748, 1).
digit(4064, 1).
digit(43746, 1).
digit(34467, 1).
digit(32723, 0).
digit(25957, 0).
digit(20913, 1).
digit(16318, 0).
digit(27280, 0).
digit(31689, 1).
digit(33157, 0).
digit(17942, 1).
digit(48023, 0).
digit(28583, 1).
digit(56511, 1).
digit(11462, 0).
digit(23333, 1).
digit(49357, 1).
digit(35064, 0).
digit(43951, 1).
digit(33687, 0).
digit(51533, 0).
digit(33501, 0).
digit(44277, 1).
digit(19093, 1).
digit(11887, 0).
digit(25051, 0).
digit(30335, 1).
digit(54102, 0).
digit(48184, 1).
digit(32887, 1).
digit(41811, 0).
digit(7062, 0).
digit(4505, 0).
digit(33377, 1).
digit(58768, 0).
digit(5355, 1).
digit(7234, 1).
digit(44313, 0).
digit(7524, 1).
digit(57061, 0).
digit(34706, 0).
digit(18524, 1).
digit(57816, 1).
digit(12415, 1).
digit(9131, 0).
digit(9487, 1).
digit(58014, 0).
digit(39220, 0).
digit(35395, 1).
digit(22349, 1).
digit(55361, 0).
digit(46240, 1).
digit(39054, 0).
digit(51464, 0).
digit(22582, 1).
digit(30068, 1).
digit(17838, 0).
digit(10283, 0).
digit(49145, 1).
digit(19554, 0).
digit(34169, 1).
digit(14056, 1).
digit(35487, 1).
digit(36956, 1).
digit(5075, 1).
digit(40918, 1).
digit(10120, 0).
digit(51999, 0).
digit(50847, 1).
digit(3241, 0).
digit(49200, 0).
digit(41683, 0).
digit(28428, 0).
digit(29810, 1).
digit(10868, 1).
digit(59166, 1).
digit(573, 1).
digit(19940, 1).
digit(29069, 1).
digit(6702, 1).
digit(34943, 0).
digit(27446, 0).
digit(24856, 0).
digit(33354, 1).
digit(56986, 1).
digit(33191, 1).
digit(29087, 1).
digit(53243, 0).
digit(28862, 1).
digit(2584, 0).
digit(53361, 1).
digit(10747, 0).
digit(19598, 1).
digit(50804, 0).
digit(27429, 0).
digit(24800, 1).
digit(38384, 1).
digit(32392, 1).
digit(8506, 1).
digit(56169, 1).
digit(35427, 1).
digit(52854, 1).
digit(27388, 1).
digit(5439, 1).
digit(15121, 1).
digit(470, 1).
digit(34945, 1).
digit(48862, 1).
digit(26892, 1).
digit(12591, 1).
digit(50600, 1).
digit(6793, 1).
digit(30434, 0).
digit(50236, 0).
digit(20958, 0).
digit(25159, 0).
digit(51299, 0).
digit(52382, 1).
digit(9653, 1).
digit(59912, 1).
digit(9857, 0).
digit(34014, 0).
digit(42463, 1).
digit(12497, 0).
digit(35408, 0).
digit(58033, 1).
digit(11729, 0).
digit(25758, 1).
digit(42673, 1).
digit(45908, 0).
digit(7788, 1).
digit(58309, 1).
digit(3272, 1).
digit(11240, 0).
digit(37856, 0).
digit(49306, 0).
digit(57397, 0).
digit(22475, 1).
digit(1645, 0).
digit(11133, 0).
digit(36084, 0).
digit(49606, 1).
digit(35900, 1).
digit(42835, 0).
digit(12383, 1).
digit(36749, 0).
digit(15814, 0).
digit(53257, 0).
digit(45255, 0).
digit(9906, 1).
digit(34057, 0).
digit(14323, 0).
digit(22066, 1).
digit(9473, 0).
digit(37305, 1).
digit(7020, 1).
digit(43016, 1).
digit(51900, 1).
digit(47752, 0).
digit(28513, 1).
digit(19216, 0).
digit(10325, 1).
digit(56941, 0).
digit(52247, 0).
digit(37190, 1).
digit(18836, 0).
digit(17108, 0).
digit(16306, 0).
digit(15904, 0).
digit(12553, 0).
digit(20828, 1).
digit(27831, 1).
digit(48044, 1).
digit(11403, 0).
digit(49723, 1).
digit(9298, 1).
digit(39441, 1).
digit(21634, 1).
digit(5830, 0).
digit(3021, 0).
digit(462, 0).
digit(32494, 0).
digit(27630, 1).
digit(43083, 0).
digit(38545, 1).
digit(30564, 1).
digit(20123, 0).
digit(26867, 1).
digit(44453, 1).
digit(58121, 0).
digit(31993, 1).
digit(25918, 1).
digit(13729, 0).
digit(17875, 1).
digit(4435, 1).
digit(49899, 1).
digit(17854, 1).
digit(57917, 0).
digit(47391, 1).
digit(41195, 1).
digit(49961, 1).
digit(43006, 1).
digit(1703, 1).
digit(7499, 0).
digit(1907, 0).
digit(9423, 1).
digit(41388, 0).
digit(39262, 1).
digit(41744, 1).
digit(57536, 0).
digit(3106, 0).
digit(13033, 0).
digit(22347, 1).
digit(57516, 1).
digit(16139, 0).
digit(42348, 1).
digit(39950, 1).
digit(46826, 0).
digit(527, 0).
digit(10293, 1).
digit(54979, 1).
digit(13510, 1).
digit(15888, 1).
digit(43017, 1).
digit(49134, 1).
digit(5288, 0).
digit(24495, 0).
digit(55398, 0).
digit(54611, 1).
digit(2202, 1).
digit(22364, 0).
digit(28837, 1).
digit(7324, 1).
digit(9457, 1).
digit(57834, 1).
digit(507, 1).
digit(44864, 1).
digit(41858, 1).
digit(30668, 0).
digit(22511, 0).
digit(57407, 0).
digit(29860, 0).
digit(17812, 1).
digit(31739, 1).
digit(3441, 0).
digit(42268, 1).
digit(17508, 0).
digit(29214, 1).
digit(57916, 0).
digit(57494, 0).
digit(37011, 1).
digit(7243, 0).
digit(11433, 1).
digit(11914, 0).
digit(2719, 1).
digit(58356, 0).
digit(34097, 0).
digit(21921, 1).
digit(7748, 1).
digit(51277, 1).
digit(34403, 1).
digit(47158, 1).
digit(39346, 0).
digit(612, 0).
digit(29805, 0).
digit(6110, 1).
digit(54974, 1).
digit(46775, 0).
digit(56903, 1).
digit(2688, 1).
digit(29562, 0).
digit(34502, 0).
digit(19360, 0).
digit(68, 0).
digit(39152, 1).
digit(39288, 1).
digit(44131, 1).
digit(30455, 0).
digit(28647, 0).
digit(58662, 1).
digit(14557, 0).
digit(40844, 0).
digit(2623, 1).
digit(47291, 1).
digit(14102, 0).
digit(24678, 1).
digit(35942, 1).
digit(10365, 1).
digit(36457, 1).
digit(30340, 0).
digit(11052, 0).
digit(45516, 1).
digit(11387, 1).
digit(39196, 1).
digit(2419, 1).
digit(45782, 1).
digit(4708, 1).
digit(41138, 0).
digit(40612, 0).
digit(52492, 1).
digit(26072, 1).
digit(34515, 1).
digit(49205, 1).
digit(22847, 1).
digit(38968, 1).
digit(48715, 0).
digit(52205, 1).
digit(39661, 1).
digit(33381, 0).
digit(8529, 1).
digit(23753, 0).
digit(8811, 1).
digit(18282, 1).
digit(36382, 1).
digit(32461, 0).
digit(56050, 0).
digit(23971, 0).
digit(43387, 0).
digit(54864, 1).
digit(46747, 1).
digit(22860, 0).
digit(2101, 0).
digit(56397, 0).
digit(19979, 0).
digit(22785, 1).
digit(17402, 1).
digit(40457, 0).
digit(18196, 1).
digit(55505, 1).
digit(26338, 0).
digit(16181, 1).
digit(49384, 1).
digit(49872, 0).
digit(24288, 1).
digit(16300, 1).
digit(2667, 1).
digit(20351, 0).
digit(3406, 1).
digit(26667, 1).
digit(53599, 1).
digit(3568, 0).
digit(44583, 0).
digit(40831, 1).
digit(5512, 0).
digit(5074, 0).
digit(40004, 1).
digit(49338, 1).
digit(3015, 0).
digit(45302, 1).
digit(31361, 0).
digit(19227, 1).
digit(46165, 1).
digit(34580, 0).
digit(54715, 0).
digit(6372, 1).
digit(18779, 0).
digit(18301, 0).
digit(57505, 1).
digit(46748, 0).
digit(37531, 1).
digit(59864, 1).
digit(13233, 0).
digit(2310, 0).
digit(30849, 0).
digit(49185, 0).
digit(49087, 1).
digit(3427, 1).
digit(12932, 1).
digit(47156, 0).
digit(22710, 0).
digit(22303, 1).
digit(29434, 1).
digit(17028, 1).
digit(35667, 0).
digit(38889, 0).
digit(23372, 0).
digit(58234, 1).
digit(46012, 1).
digit(30366, 1).
digit(28895, 0).
digit(44815, 0).
digit(58171, 1).
digit(10971, 1).
digit(7728, 0).
digit(43684, 1).
digit(28296, 0).
digit(30053, 1).
digit(18113, 0).
digit(53761, 1).
digit(33739, 0).
digit(3049, 0).
digit(8926, 0).
digit(59762, 0).
digit(9005, 0).
digit(44730, 0).
digit(7435, 1).
digit(54875, 1).
digit(39606, 0).
digit(52990, 1).
digit(54824, 1).
digit(43939, 0).
digit(58038, 0).
digit(31894, 1).
digit(41596, 1).
digit(38736, 1).
digit(14319, 0).
digit(58189, 1).
digit(1252, 1).
digit(13113, 1).
digit(55227, 0).
digit(17603, 0).
digit(56, 0).
digit(54803, 1).
digit(9453, 1).
digit(53119, 1).
digit(27603, 0).
digit(49220, 0).
digit(43088, 0).
digit(40322, 0).
digit(15903, 1).
digit(39624, 0).
digit(2830, 1).
digit(27020, 1).
digit(56783, 1).
digit(50559, 0).
digit(50799, 1).
digit(32496, 1).
digit(30789, 1).
digit(8515, 0).
digit(48977, 1).
digit(6008, 0).
digit(12746, 1).
digit(1003, 1).
digit(59071, 1).
digit(152, 1).
digit(33176, 1).
digit(24081, 0).
digit(40965, 0).
digit(6708, 0).
digit(41526, 1).
digit(47580, 1).
digit(6991, 0).
digit(56435, 1).
digit(20625, 0).
digit(46292, 0).
digit(34197, 0).
digit(13211, 0).
digit(55939, 1).
digit(16323, 0).
digit(54027, 1).
digit(43195, 0).
digit(43011, 0).
digit(20563, 1).
digit(33298, 1).
digit(37869, 1).
digit(58091, 1).
digit(27879, 1).
digit(36145, 1).
digit(22882, 0).
digit(52313, 0).
digit(11358, 1).
digit(14291, 0).
digit(45939, 0).
digit(33043, 0).
digit(4311, 1).
digit(23132, 1).
digit(14414, 1).
digit(29994, 0).
digit(46936, 1).
digit(21971, 0).
digit(23875, 0).
digit(35654, 1).
digit(19090, 1).
digit(49902, 1).
digit(58459, 1).
digit(12705, 1).
digit(30707, 0).
digit(41371, 0).
digit(15524, 0).
digit(20988, 1).
digit(35816, 0).
digit(24320, 0).
digit(52291, 1).
digit(23501, 1).
digit(34173, 1).
digit(38125, 1).
digit(54595, 0).
digit(4160, 1).
digit(10469, 0).
digit(59342, 0).
digit(21391, 1).
digit(3971, 1).
digit(43339, 1).
digit(48024, 1).
digit(26104, 1).
digit(12653, 0).
digit(32950, 0).
digit(21747, 0).
digit(56602, 1).
digit(41372, 1).
digit(2122, 1).
digit(58953, 1).
digit(13648, 1).
digit(22207, 1).
digit(26058, 0).
digit(59965, 1).
digit(46173, 1).
digit(7747, 1).
digit(12318, 1).
digit(40140, 1).
digit(70, 1).
digit(16475, 1).
digit(3691, 0).
digit(58139, 1).
digit(58521, 0).
digit(53135, 1).
digit(48989, 1).
digit(11079, 1).
digit(20283, 0).
digit(34512, 1).
digit(2218, 0).
digit(3964, 0).
digit(2843, 1).
digit(22634, 0).
digit(56674, 0).
digit(38228, 0).
digit(15439, 1).
digit(41646, 0).
digit(16402, 0).
digit(19985, 0).
digit(17956, 0).
digit(8545, 0).
digit(27328, 0).
digit(29451, 1).
digit(4484, 1).
digit(15471, 0).
digit(49059, 0).
digit(31278, 1).
digit(59557, 0).
digit(41238, 1).
digit(48751, 1).
digit(32925, 0).
digit(9287, 0).
digit(37378, 1).
digit(23056, 1).
digit(53319, 0).
digit(55707, 1).
digit(35107, 0).
digit(3231, 0).
digit(23401, 0).
digit(6926, 1).
digit(5353, 0).
digit(58934, 0).
digit(16460, 0).
digit(6985, 1).
digit(54428, 1).
digit(4044, 0).
digit(27951, 0).
digit(33646, 1).
digit(33652, 1).
digit(49664, 1).
digit(10479, 0).
digit(4738, 0).
digit(27492, 0).
digit(21574, 1).
digit(15423, 1).
digit(44859, 0).
digit(28667, 0).
digit(41019, 1).
digit(47275, 0).
digit(34213, 0).
digit(27245, 1).
digit(26914, 1).
digit(42010, 0).
digit(15795, 0).
digit(22370, 1).
digit(5272, 0).
digit(55622, 0).
digit(10556, 1).
digit(23203, 1).
digit(56635, 0).
digit(25268, 1).
digit(12195, 0).
digit(50929, 0).
digit(20443, 1).
digit(21271, 0).
digit(4470, 1).
digit(11129, 0).
digit(45145, 0).
digit(48207, 0).
digit(28836, 1).
digit(3720, 0).
digit(58965, 1).
digit(43664, 1).
digit(6521, 1).
digit(6133, 0).
digit(17625, 0).
digit(9422, 0).
digit(16083, 0).
digit(46984, 1).
digit(40099, 0).
digit(27820, 0).
digit(33330, 0).
digit(21847, 1).
digit(7358, 0).
digit(48511, 0).
digit(34156, 1).
digit(6654, 1).
digit(15548, 1).
digit(28656, 1).
digit(29599, 0).
digit(45651, 1).
digit(19565, 0).
digit(56969, 0).
digit(21702, 1).
digit(20846, 0).
digit(41553, 0).
digit(58659, 0).
digit(49837, 1).
digit(48209, 1).
digit(20816, 0).
digit(14967, 1).
digit(50150, 1).
digit(52084, 1).
digit(58501, 1).
digit(55412, 1).
digit(11072, 0).
digit(33516, 1).
digit(5859, 1).
digit(55335, 1).
digit(8349, 0).
digit(38432, 1).
digit(42406, 1).
digit(51015, 1).
digit(47480, 0).
digit(13956, 1).
digit(10413, 1).
digit(15955, 0).
digit(7462, 0).
digit(32474, 1).
digit(34519, 0).
digit(55955, 1).
digit(59296, 0).
digit(5582, 1).
digit(18179, 1).
digit(1796, 0).
digit(38029, 1).
digit(25629, 0).
digit(31263, 0).
digit(37759, 0).
digit(33725, 1).
digit(4181, 0).
digit(41894, 1).
digit(16679, 0).
digit(13641, 0).
digit(7656, 1).
digit(40228, 0).
digit(10157, 1).
digit(54353, 1).
digit(49380, 1).
digit(20407, 0).
digit(52521, 1).
digit(23172, 0).
digit(42610, 0).
digit(7349, 1).
digit(5500, 1).
digit(9803, 1).
digit(26191, 0).
digit(24070, 1).
digit(28607, 0).
digit(41205, 1).
digit(19012, 0).
digit(19068, 1).
digit(1815, 1).
digit(22788, 1).
digit(15009, 1).
digit(46335, 1).
digit(10755, 0).
digit(23598, 0).
digit(36682, 1).
digit(29211, 0).
digit(14518, 0).
digit(12193, 0).
digit(19871, 0).
digit(38840, 1).
digit(535, 1).
digit(8687, 0).
digit(46199, 1).
digit(16903, 0).
digit(35389, 1).
digit(37897, 1).
digit(1916, 0).
digit(10152, 1).
digit(55500, 1).
digit(15582, 0).
digit(20237, 1).
digit(28456, 0).
digit(21933, 0).
digit(27967, 1).
digit(10140, 0).
digit(41282, 1).
digit(4203, 0).
digit(48479, 0).
digit(18538, 0).
digit(52143, 1).
digit(49322, 0).
digit(52082, 1).
digit(29578, 0).
digit(3771, 1).
digit(9671, 0).
digit(37911, 1).
digit(25087, 1).
digit(11508, 1).
digit(56048, 1).
digit(57266, 1).
digit(58198, 1).
digit(25264, 1).
digit(38522, 1).
digit(8160, 1).
digit(36744, 1).
digit(22639, 1).
digit(44627, 1).
digit(41187, 0).
digit(11091, 0).
digit(41987, 0).
digit(29456, 0).
digit(41805, 1).
digit(12041, 1).
digit(27414, 0).
digit(2994, 1).
digit(28246, 1).
digit(15182, 1).
digit(56374, 1).
digit(43286, 1).
digit(22657, 0).
digit(2817, 0).
digit(19822, 0).
digit(31901, 1).
digit(12128, 1).
digit(44792, 1).
digit(8131, 1).
digit(16614, 1).
digit(11362, 1).
digit(59372, 0).
digit(1304, 0).
digit(39984, 1).
digit(25576, 0).
digit(5319, 0).
digit(23756, 1).
digit(41824, 1).
digit(26279, 1).
digit(42899, 1).
digit(23835, 1).
digit(33456, 1).
digit(7003, 0).
digit(30212, 0).
digit(40180, 0).
digit(49681, 1).
digit(21848, 0).
digit(43261, 1).
digit(24619, 1).
digit(19051, 1).
digit(8112, 1).
digit(22886, 0).
digit(2717, 1).
digit(6613, 1).
digit(58617, 0).
digit(31296, 1).
digit(37790, 0).
digit(24958, 1).
digit(15175, 1).
digit(53160, 1).
digit(16987, 0).
digit(29177, 0).
digit(43267, 1).
digit(22209, 0).
digit(35000, 1).
digit(38528, 1).
digit(12132, 1).
digit(26578, 1).
digit(49500, 0).
digit(37252, 0).
digit(25214, 0).
digit(33389, 0).
digit(41732, 1).
digit(4105, 1).
digit(24216, 1).
digit(47817, 1).
digit(36106, 1).
digit(34177, 0).
digit(44979, 0).
digit(33998, 1).
digit(33508, 1).
digit(54816, 1).
digit(50525, 1).
digit(43844, 1).
digit(36448, 1).
digit(8, 1).
digit(17210, 0).
digit(41898, 1).
digit(58022, 1).
digit(31425, 0).
digit(59805, 0).
digit(24352, 1).
digit(48099, 1).
digit(45893, 1).
digit(26753, 0).
digit(50661, 1).
digit(7116, 1).
digit(29245, 1).
digit(8512, 1).
digit(34566, 1).
digit(1716, 1).
digit(9480, 1).
digit(42074, 0).
digit(47186, 0).
digit(55065, 0).
digit(42230, 0).
digit(50437, 1).
digit(12499, 1).
digit(33418, 1).
digit(38845, 1).
digit(51422, 0).
digit(9865, 0).
digit(43915, 0).
digit(8809, 0).
digit(43266, 0).
digit(59530, 1).
digit(26745, 1).
digit(4639, 1).
digit(10852, 1).
digit(13504, 1).
digit(10669, 1).
digit(26501, 0).
digit(16872, 1).
digit(7894, 0).
digit(29480, 1).
digit(17095, 0).
digit(6877, 1).
digit(58737, 1).
digit(30391, 0).
digit(39581, 0).
digit(47316, 0).
digit(57813, 1).
digit(11702, 1).
digit(17965, 1).
digit(36272, 0).
digit(26230, 0).
digit(40148, 1).
digit(8937, 1).
digit(18515, 0).
digit(23495, 0).
digit(42552, 0).
digit(24706, 0).
digit(53707, 1).
digit(40248, 1).
digit(47153, 1).
digit(27806, 0).
digit(2996, 0).
digit(34064, 0).
digit(14759, 1).
digit(14196, 1).
digit(16217, 0).
digit(21216, 0).
digit(5824, 1).
digit(26188, 1).
digit(39028, 0).
digit(7996, 1).
digit(3996, 1).
digit(1192, 1).
digit(1443, 0).
digit(25814, 0).
digit(7629, 0).
digit(9775, 1).
digit(12723, 0).
digit(16543, 0).
digit(50518, 0).
digit(11082, 1).
digit(50404, 1).
digit(39202, 0).
digit(26924, 1).
digit(473, 0).
digit(49497, 0).
digit(28100, 1).
digit(35514, 1).
digit(44743, 0).
digit(3393, 1).
digit(59787, 1).
digit(50062, 1).
digit(52872, 1).
digit(26639, 0).
digit(34585, 1).
digit(51529, 1).
digit(5368, 0).
digit(24647, 1).
digit(2557, 0).
digit(33706, 1).
digit(58782, 0).
digit(23762, 1).
digit(27250, 1).
digit(9895, 1).
digit(25650, 0).
digit(12356, 1).
digit(9960, 1).
digit(25344, 0).
digit(6857, 0).
digit(3878, 1).
digit(45961, 1).
digit(58597, 1).
digit(7828, 1).
digit(17129, 1).
digit(40504, 1).
digit(36290, 0).
digit(24183, 1).
digit(43701, 0).
digit(1597, 1).
digit(52961, 1).
digit(52489, 1).
digit(12362, 1).
digit(34796, 1).
digit(11574, 0).
digit(44876, 1).
digit(38729, 0).
digit(733, 0).
digit(25702, 1).
digit(35761, 1).
digit(48119, 1).
digit(27677, 1).
digit(47229, 1).
digit(49229, 0).
digit(36633, 1).
digit(2770, 0).
digit(31286, 1).
digit(24605, 0).
digit(5052, 0).
digit(46029, 1).
digit(2500, 0).
digit(14541, 1).
digit(40038, 0).
digit(26746, 1).
digit(5779, 1).
digit(8187, 0).
digit(33427, 0).
digit(11584, 0).
digit(24918, 1).
digit(59189, 1).
digit(34640, 1).
digit(59414, 1).
digit(14794, 1).
digit(2642, 0).
digit(44130, 0).
digit(965, 0).
digit(56137, 0).
digit(7240, 1).
digit(16362, 1).
digit(54417, 1).
digit(39138, 1).
digit(28423, 0).
digit(13884, 1).
digit(22859, 1).
digit(22844, 1).
digit(55521, 0).
digit(58275, 0).
digit(28505, 0).
digit(32009, 0).
digit(25983, 1).
digit(22727, 1).
digit(44158, 1).
digit(50353, 0).
digit(8080, 1).
digit(29907, 0).
digit(37822, 0).
digit(29540, 0).
digit(15152, 1).
digit(2617, 0).
digit(42965, 0).
digit(8486, 1).
digit(35651, 0).
digit(8088, 0).
digit(13001, 1).
digit(38054, 0).
digit(39980, 1).
digit(47466, 0).
digit(25103, 1).
digit(33677, 0).
digit(5428, 1).
digit(59639, 0).
digit(34074, 1).
digit(2195, 0).
digit(40973, 1).
digit(57700, 0).
digit(54418, 1).
digit(33503, 1).
digit(39622, 1).
digit(49592, 1).
digit(45577, 0).
digit(11291, 1).
digit(19388, 1).
digit(23142, 1).
digit(12226, 1).
digit(18541, 1).
digit(21454, 0).
digit(5643, 1).
digit(10239, 1).
digit(44548, 1).
digit(11354, 0).
digit(29230, 0).
digit(32905, 0).
digit(58594, 0).
digit(31980, 1).
digit(53610, 1).
digit(34291, 1).
digit(9622, 1).
digit(19639, 1).
digit(48210, 0).
digit(16286, 0).
digit(246, 0).
digit(20117, 0).
digit(55142, 1).
digit(43577, 1).
digit(31144, 1).
digit(6303, 1).
digit(27064, 1).
digit(57176, 0).
digit(19420, 1).
digit(33327, 0).
digit(12811, 1).
digit(26366, 0).
digit(31497, 0).
digit(33403, 0).
digit(13733, 1).
digit(15169, 0).
digit(36975, 0).
digit(53444, 0).
digit(51197, 1).
digit(59373, 1).
digit(27744, 1).
digit(26799, 0).
digit(52793, 0).
digit(58341, 1).
digit(6446, 0).
digit(13366, 0).
digit(39402, 1).
digit(19439, 0).
digit(30486, 1).
digit(357, 1).
digit(58560, 0).
digit(5015, 0).
digit(54512, 0).
digit(52268, 1).
digit(19898, 1).
digit(34714, 0).
digit(45803, 1).
digit(26422, 0).
digit(23862, 1).
digit(57303, 1).
digit(55302, 0).
digit(7898, 1).
digit(8574, 1).
digit(3412, 1).
digit(42862, 0).
digit(14309, 1).
digit(35740, 0).
digit(22143, 0).
digit(6332, 0).
digit(53233, 1).
digit(18914, 0).
digit(49131, 1).
digit(3479, 0).
digit(13273, 1).
digit(49941, 1).
digit(10580, 1).
digit(17462, 0).
digit(21996, 0).
digit(27455, 1).
digit(24085, 0).
digit(17771, 1).
digit(40056, 1).
digit(25764, 1).
digit(42380, 1).
digit(4524, 1).
digit(21681, 0).
digit(5192, 0).
digit(56662, 0).
digit(19990, 0).
digit(9199, 1).
digit(42374, 0).
digit(12703, 0).
digit(34699, 0).
digit(44012, 1).
digit(25101, 0).
digit(53014, 0).
digit(29901, 1).
digit(46107, 1).
digit(25858, 1).
digit(26308, 0).
digit(1797, 0).
digit(13030, 0).
digit(7027, 1).
digit(52614, 1).
digit(35060, 0).
digit(18707, 1).
digit(25749, 1).
digit(24861, 0).
digit(44693, 1).
digit(43539, 1).
digit(38574, 0).
digit(24688, 1).
digit(16085, 0).
digit(36368, 1).
digit(37914, 0).
digit(35963, 0).
digit(2082, 0).
digit(10195, 0).
digit(21759, 0).
digit(53199, 1).
digit(23973, 1).
digit(35343, 0).
digit(765, 1).
digit(17413, 1).
digit(26647, 1).
digit(50489, 1).
digit(604, 1).
digit(14258, 0).
digit(42834, 1).
digit(16222, 1).
digit(29479, 1).
digit(14812, 1).
digit(54388, 1).
digit(30158, 1).
digit(19136, 1).
digit(38929, 0).
digit(3954, 1).
digit(33385, 1).
digit(21594, 1).
digit(23854, 1).
digit(15442, 0).
digit(10380, 0).
digit(46040, 1).
digit(27301, 1).
digit(48612, 0).
digit(40692, 1).
digit(30438, 0).
digit(6704, 0).
digit(31518, 0).
digit(53301, 1).
digit(51901, 1).
digit(5961, 0).
digit(21972, 1).
digit(32705, 1).
digit(35621, 1).
digit(56960, 1).
digit(14511, 0).
digit(22749, 0).
digit(20979, 0).
digit(30906, 1).
digit(5304, 1).
digit(8402, 1).
digit(51789, 0).
digit(37948, 1).
digit(41210, 1).
digit(17423, 1).
digit(28970, 1).
digit(21553, 0).
digit(46254, 1).
digit(12133, 1).
digit(13816, 0).
digit(50249, 1).
digit(47729, 1).
digit(4286, 1).
digit(43151, 1).
digit(39281, 1).
digit(6832, 0).
digit(9616, 1).
digit(39546, 1).
digit(49852, 1).
digit(38860, 1).
digit(12987, 1).
digit(22538, 1).
digit(37758, 0).
digit(56683, 1).
digit(29420, 0).
digit(39740, 0).
digit(35749, 1).
digit(10425, 0).
digit(41635, 0).
digit(33446, 1).
digit(24877, 0).
digit(55000, 1).
digit(54070, 1).
digit(30100, 1).
digit(27900, 0).
digit(48022, 1).
digit(37978, 1).
digit(1095, 1).
digit(43715, 1).
digit(5051, 1).
digit(38487, 1).
digit(11262, 1).
digit(34483, 1).
digit(36376, 0).
digit(44561, 1).
digit(55922, 0).
digit(15759, 0).
digit(17262, 1).
digit(6137, 0).
digit(44108, 1).
digit(45315, 0).
digit(36124, 1).
digit(59153, 1).
digit(6758, 0).
digit(20589, 1).
digit(26545, 0).
digit(33924, 1).
digit(18425, 0).
digit(18212, 0).
digit(38614, 0).
digit(53143, 1).
digit(37116, 1).
digit(54182, 1).
digit(13738, 0).
digit(38161, 0).
digit(1367, 0).
digit(57241, 0).
digit(57995, 0).
digit(8873, 1).
digit(50157, 0).
digit(38081, 1).
digit(9402, 0).
digit(56221, 1).
digit(39265, 1).
digit(2505, 1).
digit(26178, 0).
digit(17025, 0).
digit(58451, 1).
digit(52629, 0).
digit(668, 0).
digit(21379, 1).
digit(30223, 0).
digit(55523, 1).
digit(17049, 0).
digit(9072, 1).
digit(31264, 1).
digit(603, 0).
digit(17662, 1).
digit(8538, 1).
digit(4944, 1).
digit(27109, 0).
digit(11636, 0).
digit(18665, 0).
digit(45063, 1).
digit(57607, 1).
digit(17438, 1).
digit(52051, 1).
digit(53928, 0).
digit(32246, 1).
digit(16997, 0).
digit(21372, 0).
digit(45730, 1).
digit(17381, 0).
digit(48860, 1).
digit(48361, 1).
digit(18546, 1).
digit(33394, 1).
digit(25394, 1).
digit(30737, 1).
digit(23954, 1).
digit(51235, 0).
digit(47233, 1).
digit(26002, 1).
digit(59472, 0).
digit(1877, 0).
digit(22735, 1).
digit(4688, 0).
digit(3677, 0).
digit(16921, 0).
digit(458, 0).
digit(30992, 0).
digit(53321, 1).
digit(46868, 0).
digit(21409, 1).
digit(14059, 0).
digit(54506, 1).
digit(28877, 1).
digit(22040, 0).
digit(36333, 1).
digit(22868, 1).
digit(9806, 0).
digit(44094, 0).
digit(52963, 1).
digit(32224, 1).
digit(26459, 1).
digit(112, 1).
digit(33398, 1).
digit(31079, 0).
digit(57722, 1).
digit(55177, 0).
digit(43836, 1).
digit(49208, 0).
digit(30901, 1).
digit(49359, 1).
digit(1771, 0).
digit(25022, 0).
digit(14165, 0).
digit(26175, 1).
digit(21109, 1).
digit(1544, 1).
digit(42071, 1).
digit(6768, 0).
digit(4527, 0).
digit(31465, 1).
digit(57261, 0).
digit(40611, 0).
digit(32693, 0).
digit(59149, 1).
digit(53168, 1).
digit(30700, 0).
digit(16670, 1).
digit(28184, 1).
digit(11760, 1).
digit(24771, 1).
digit(29396, 1).
digit(26258, 0).
digit(39039, 0).
digit(40329, 0).
digit(21203, 0).
digit(55410, 0).
digit(34725, 1).
digit(49796, 1).
digit(54773, 1).
digit(3160, 0).
digit(2483, 0).
digit(1152, 0).
digit(45760, 0).
digit(6730, 0).
digit(30104, 1).
digit(17738, 0).
digit(41117, 0).
digit(27933, 1).
digit(32245, 1).
digit(13874, 0).
digit(28772, 0).
digit(4951, 0).
digit(36391, 0).
digit(24011, 0).
digit(54209, 0).
digit(16396, 0).
digit(26117, 0).
digit(59697, 1).
digit(6502, 0).
digit(7912, 1).
digit(52309, 1).
digit(4657, 1).
digit(23530, 1).
digit(8253, 0).
digit(20045, 1).
digit(21248, 1).
digit(2112, 0).
digit(30829, 1).
digit(43989, 1).
digit(56651, 0).
digit(1501, 0).
digit(4813, 1).
digit(36318, 0).
digit(20238, 0).
digit(33244, 0).
digit(13054, 0).
digit(19686, 0).
digit(24970, 1).
digit(17349, 0).
digit(10342, 1).
digit(31095, 0).
digit(3344, 1).
digit(2907, 1).
digit(37674, 1).
digit(53536, 0).
digit(42506, 1).
digit(35815, 0).
digit(40043, 0).
digit(30457, 1).
digit(46434, 1).
digit(52360, 1).
digit(37635, 1).
digit(56609, 0).
digit(17214, 0).
digit(26243, 1).
digit(42888, 1).
digit(55870, 1).
digit(42102, 1).
digit(18357, 0).
digit(1029, 0).
digit(6330, 0).
digit(5461, 0).
digit(47124, 0).
digit(9288, 1).
digit(8124, 1).
digit(34619, 0).
digit(25750, 0).
digit(33350, 0).
digit(13685, 0).
digit(45126, 0).
digit(13807, 0).
digit(5494, 1).
digit(24020, 1).
digit(37923, 1).
digit(19381, 1).
digit(31808, 0).
digit(13334, 1).
digit(7764, 1).
digit(14503, 1).
digit(13107, 1).
digit(28724, 1).
digit(24966, 1).
digit(358, 1).
digit(30424, 0).
digit(29603, 1).
digit(29825, 1).
digit(58053, 0).
digit(36864, 1).
digit(11451, 0).
digit(42623, 0).
digit(51649, 0).
digit(9609, 1).
digit(41600, 1).
digit(23295, 1).
digit(45221, 1).
digit(14773, 0).
digit(58511, 1).
digit(13544, 0).
digit(17900, 1).
digit(43102, 0).
digit(1118, 1).
digit(41211, 0).
digit(20244, 0).
digit(42131, 0).
digit(25803, 0).
digit(41247, 1).
digit(25416, 1).
digit(1015, 0).
digit(43445, 0).
digit(1394, 1).
digit(12930, 1).
digit(5714, 1).
digit(6135, 1).
digit(52809, 0).
digit(50477, 1).
digit(53934, 0).
digit(59156, 1).
digit(7371, 0).
digit(15722, 1).
digit(22473, 0).
digit(40693, 1).
digit(35821, 0).
digit(53791, 1).
digit(32880, 0).
digit(42713, 1).
digit(26479, 1).
digit(47166, 0).
digit(36164, 1).
digit(10505, 0).
digit(1405, 1).
digit(31122, 1).
digit(7846, 0).
digit(29807, 1).
digit(6366, 1).
digit(32597, 1).
digit(38847, 1).
digit(40561, 1).
digit(200, 1).
digit(5773, 0).
digit(31920, 1).
digit(46328, 0).
digit(41137, 0).
digit(15001, 1).
digit(30565, 0).
digit(41076, 1).
digit(31310, 0).
digit(54284, 0).
digit(7588, 1).
digit(33148, 0).
digit(41011, 1).
digit(46481, 1).
digit(55262, 1).
digit(25003, 1).
digit(47372, 0).
digit(24689, 0).
digit(32333, 0).
digit(20083, 1).
digit(26968, 1).
digit(40848, 1).
digit(7128, 0).
digit(7040, 1).
digit(36019, 0).
digit(32005, 1).
digit(17314, 0).
digit(21031, 1).
digit(22910, 0).
digit(45953, 0).
digit(43613, 0).
digit(37686, 0).
digit(46696, 1).
digit(44369, 1).
digit(15252, 1).
digit(47908, 1).
digit(24424, 1).
digit(1625, 0).
digit(20551, 0).
digit(30639, 1).
digit(57543, 0).
digit(44000, 0).
digit(41296, 0).
digit(28105, 0).
digit(21716, 1).
digit(5221, 1).
digit(5266, 0).
digit(36831, 1).
digit(52836, 0).
digit(26617, 1).
digit(1377, 0).
digit(15890, 0).
digit(31843, 1).
digit(11920, 0).
digit(1956, 0).
digit(12215, 1).
digit(20602, 0).
digit(45769, 0).
digit(24069, 0).
digit(11612, 0).
digit(56252, 1).
digit(39695, 1).
digit(15798, 0).
digit(53299, 0).
digit(39757, 1).
digit(43044, 1).
digit(9755, 1).
digit(58146, 0).
digit(54354, 0).
digit(12504, 0).
digit(18304, 1).
digit(25745, 1).
digit(49394, 0).
digit(15318, 0).
digit(666, 0).
digit(52615, 1).
digit(50324, 0).
digit(17304, 0).
digit(33390, 0).
digit(33005, 1).
digit(1909, 1).
digit(42245, 0).
digit(43813, 1).
digit(41896, 0).
digit(10080, 0).
digit(18253, 0).
digit(57707, 0).
digit(19400, 1).
digit(56332, 0).
digit(44682, 1).
digit(24245, 0).
digit(28017, 1).
digit(22776, 1).
digit(19203, 0).
digit(22025, 1).
digit(54600, 0).
digit(44614, 0).
digit(18188, 1).
digit(18070, 1).
digit(57453, 1).
digit(29283, 0).
digit(36108, 1).
digit(57454, 1).
digit(32526, 0).
digit(37408, 0).
digit(5966, 1).
digit(2594, 1).
digit(48040, 1).
digit(3881, 0).
digit(15965, 0).
digit(35857, 0).
digit(3247, 0).
digit(24676, 0).
digit(10926, 1).
digit(10022, 0).
digit(43297, 1).
digit(2746, 0).
digit(8531, 0).
digit(52451, 1).
digit(10713, 1).
digit(49989, 0).
digit(17664, 1).
digit(232, 0).
digit(13230, 0).
digit(23709, 0).
digit(47777, 1).
digit(29628, 1).
digit(28646, 1).
digit(2645, 1).
digit(24078, 0).
digit(26197, 0).
digit(10985, 1).
digit(18235, 1).
digit(7149, 1).
digit(57160, 1).
digit(23586, 0).
digit(39360, 0).
digit(26260, 0).
digit(30649, 1).
digit(5004, 1).
digit(50072, 0).
digit(50643, 0).
digit(14821, 1).
digit(43422, 0).
digit(21353, 0).
digit(28521, 1).
digit(35048, 1).
digit(9222, 0).
digit(12486, 1).
digit(45840, 0).
digit(44785, 1).
digit(27875, 1).
digit(50617, 0).
digit(38289, 0).
digit(44838, 0).
digit(9017, 1).
digit(48468, 1).
digit(1081, 1).
digit(63, 0).
digit(35586, 1).
digit(40648, 0).
digit(36315, 1).
digit(33942, 1).
digit(25989, 1).
digit(38893, 0).
digit(55152, 0).
digit(36040, 0).
digit(52847, 0).
digit(4556, 0).
digit(49011, 0).
digit(24163, 1).
digit(47388, 1).
digit(25836, 0).
digit(15677, 1).
digit(12949, 1).
digit(49600, 1).
digit(53881, 0).
digit(17775, 1).
digit(26183, 0).
digit(44844, 0).
digit(54079, 1).
digit(38494, 0).
digit(14753, 1).
digit(30628, 1).
digit(43366, 0).
digit(9406, 0).
digit(20205, 0).
digit(43976, 0).
digit(7237, 1).
digit(26502, 1).
digit(19080, 0).
digit(30920, 1).
digit(54023, 1).
digit(46837, 0).
digit(1786, 1).
digit(47259, 1).
digit(53004, 1).
digit(25961, 1).
digit(49014, 0).
digit(31445, 0).
digit(23543, 1).
digit(23788, 1).
digit(44619, 0).
digit(38666, 1).
digit(23844, 1).
digit(22410, 1).
digit(51226, 0).
digit(45541, 1).
digit(40396, 0).
digit(533, 1).
digit(6122, 1).
digit(1359, 0).
digit(56688, 1).
digit(54356, 0).
digit(3429, 0).
digit(5731, 1).
digit(28909, 1).
digit(30364, 1).
digit(36276, 1).
digit(35217, 0).
digit(34662, 0).
digit(16385, 1).
digit(22440, 0).
digit(22487, 0).
digit(37618, 1).
digit(52622, 1).
digit(22794, 1).
digit(18640, 1).
digit(1128, 0).
digit(24930, 1).
digit(5167, 0).
digit(46210, 1).
digit(28232, 1).
digit(38429, 0).
digit(48830, 0).
digit(23331, 0).
digit(49482, 1).
digit(55254, 0).
digit(46407, 1).
digit(26559, 1).
digit(1627, 1).
digit(31127, 0).
digit(47063, 1).
digit(16381, 1).
digit(52223, 1).
digit(31932, 1).
digit(32229, 1).
digit(3622, 0).
digit(9910, 1).
digit(33752, 1).
digit(14329, 0).
digit(43714, 1).
digit(36447, 0).
digit(49884, 1).
digit(11027, 0).
digit(14644, 0).
digit(52021, 0).
digit(39988, 1).
digit(16960, 1).
digit(49561, 0).
digit(20060, 0).
digit(8589, 1).
digit(44511, 1).
digit(35655, 0).
digit(34613, 1).
digit(10659, 1).
digit(32635, 1).
digit(57968, 1).
digit(59625, 1).
digit(40864, 1).
digit(9978, 0).
digit(47344, 0).
digit(59225, 0).
digit(56124, 0).
digit(39905, 1).
digit(17003, 0).
digit(46069, 0).
digit(7835, 0).
digit(2854, 0).
digit(2563, 1).
digit(47394, 1).
digit(9666, 0).
digit(39775, 0).
digit(58500, 0).
digit(26910, 1).
digit(42720, 0).
digit(58223, 0).
digit(5168, 1).
digit(54340, 1).
digit(10545, 1).
digit(52397, 0).
digit(38570, 1).
digit(29474, 0).
digit(39119, 0).
digit(35448, 1).
digit(15675, 0).
digit(9570, 1).
digit(42000, 1).
digit(43554, 0).
digit(57521, 1).
digit(40694, 1).
digit(3549, 1).
digit(30171, 1).
digit(37014, 1).
digit(5933, 1).
digit(52120, 1).
digit(19691, 1).
digit(21771, 1).
digit(3488, 1).
digit(37061, 1).
digit(45229, 1).
digit(55784, 1).
digit(24811, 1).
digit(25842, 1).
digit(24707, 1).
digit(43296, 1).
digit(26371, 0).
digit(59686, 1).
digit(2791, 1).
digit(4718, 0).
digit(55036, 1).
digit(50367, 1).
digit(17322, 0).
digit(17885, 1).
digit(11993, 0).
digit(41836, 1).
digit(11022, 0).
digit(7708, 0).
digit(52819, 1).
digit(28317, 0).
digit(51781, 1).
digit(6883, 0).
digit(26300, 0).
digit(44493, 1).
digit(18918, 1).
digit(12883, 1).
digit(38206, 1).
digit(17587, 1).
digit(47271, 0).
digit(52294, 0).
digit(41784, 1).
digit(2970, 1).
digit(54576, 1).
digit(21600, 0).
digit(38985, 1).
digit(45676, 0).
digit(13102, 1).
digit(57847, 0).
digit(54832, 0).
digit(53861, 0).
digit(8274, 0).
digit(56942, 0).
digit(22691, 0).
digit(13453, 1).
digit(37881, 1).
digit(8738, 1).
digit(37023, 1).
digit(48112, 1).
digit(52238, 0).
digit(7429, 1).
digit(49815, 0).
digit(36520, 1).
digit(21413, 0).
digit(34632, 1).
digit(2084, 0).
digit(19485, 0).
digit(6681, 0).
digit(4900, 1).
digit(5785, 1).
digit(78, 1).
digit(30509, 1).
digit(16501, 0).
digit(4534, 1).
digit(50136, 1).
digit(29676, 0).
digit(31109, 0).
digit(216, 0).
digit(21978, 1).
digit(19924, 0).
digit(9562, 1).
digit(19158, 1).
digit(41999, 0).
digit(34638, 1).
digit(27749, 0).
digit(58434, 1).
digit(54693, 1).
digit(11036, 0).
digit(13372, 1).
digit(43236, 0).
digit(52028, 0).
digit(13528, 0).
digit(42412, 1).
digit(14271, 1).
digit(52868, 1).
digit(44296, 1).
digit(49695, 0).
digit(35990, 1).
digit(31839, 1).
digit(56388, 0).
digit(33442, 0).
digit(10188, 0).
digit(53073, 0).
digit(33567, 1).
digit(7325, 0).
digit(809, 1).
digit(16735, 1).
digit(1512, 0).
digit(48911, 0).
digit(55721, 0).
digit(14521, 1).
digit(38724, 1).
digit(43583, 1).
digit(7254, 0).
digit(38056, 1).
digit(21402, 1).
digit(46371, 0).
digit(28792, 1).
digit(51556, 1).
digit(13558, 1).
digit(53355, 0).
digit(7664, 0).
digit(7860, 1).
digit(25600, 1).
digit(23464, 0).
digit(48412, 0).
digit(43567, 1).
digit(21403, 0).
digit(46949, 1).
digit(28814, 0).
digit(21508, 0).
digit(35157, 0).
digit(36594, 1).
digit(38157, 0).
digit(20085, 0).
digit(15308, 1).
digit(818, 0).
digit(31181, 1).
digit(11663, 1).
digit(36689, 0).
digit(19738, 0).
digit(44245, 1).
digit(42716, 1).
digit(42674, 0).
digit(1729, 0).
digit(12217, 0).
digit(13025, 1).
digit(15619, 0).
digit(52575, 1).
digit(52449, 0).
digit(15828, 0).
digit(4911, 0).
digit(38663, 0).
digit(55873, 0).
digit(14820, 0).
digit(16598, 0).
digit(14148, 0).
digit(50356, 1).
digit(38981, 1).
digit(8348, 1).
digit(22605, 0).
digit(9257, 0).
digit(5694, 0).
digit(40389, 0).
digit(50222, 1).
digit(24175, 1).
digit(14953, 1).
digit(38691, 0).
digit(4855, 0).
digit(27094, 0).
digit(3702, 0).
digit(55888, 1).
digit(24431, 1).
digit(29713, 0).
digit(32879, 1).
digit(16326, 0).
digit(53118, 0).
digit(7632, 1).
digit(7973, 1).
digit(6328, 1).
digit(7552, 0).
digit(37155, 1).
digit(27063, 1).
digit(57811, 0).
digit(15695, 0).
digit(10599, 0).
digit(58618, 1).
digit(32855, 1).
digit(48645, 1).
digit(55967, 0).
digit(53424, 0).
digit(43149, 0).
digit(7247, 0).
digit(34244, 0).
digit(33669, 0).
digit(18314, 1).
digit(23492, 1).
digit(20929, 0).
digit(25202, 1).
digit(30007, 1).
digit(43169, 0).
digit(10439, 0).
digit(27788, 0).
digit(1489, 0).
digit(14615, 1).
digit(46344, 0).
digit(14079, 1).
digit(58148, 1).
digit(47928, 1).
digit(27297, 1).
digit(46547, 1).
digit(44339, 1).
digit(44599, 0).
digit(54223, 1).
digit(29080, 1).
digit(3962, 1).
digit(29520, 0).
digit(36712, 0).
digit(42502, 1).
digit(18112, 0).
digit(4279, 0).
digit(46552, 1).
digit(8894, 0).
digit(7557, 0).
digit(47400, 1).
digit(41696, 0).
digit(38802, 0).
digit(27159, 1).
digit(22430, 0).
digit(59044, 1).
digit(12207, 1).
digit(22669, 0).
digit(27751, 1).
digit(20474, 1).
digit(51348, 0).
digit(41637, 1).
digit(55609, 1).
digit(15813, 1).
digit(56724, 0).
digit(38211, 0).
digit(31038, 1).
digit(2253, 0).
digit(25052, 1).
digit(38837, 1).
digit(44935, 1).
digit(48931, 1).
digit(16718, 1).
digit(23136, 0).
digit(13213, 1).
digit(20131, 1).
digit(8092, 1).
digit(5929, 0).
digit(59336, 0).
digit(27336, 1).
digit(41616, 0).
digit(16192, 0).
digit(54931, 1).
digit(35595, 1).
digit(46006, 1).
digit(17815, 1).
digit(13782, 1).
digit(32741, 1).
digit(36649, 1).
digit(47601, 1).
digit(23504, 0).
digit(1910, 1).
digit(48791, 0).
digit(35592, 1).
digit(22560, 1).
digit(2355, 1).
digit(40315, 1).
digit(43404, 0).
digit(4906, 0).
digit(57711, 1).
digit(39755, 0).
digit(57629, 0).
digit(24366, 1).
digit(59934, 0).
digit(29614, 0).
digit(53887, 1).
digit(57420, 0).
digit(36921, 1).
digit(3565, 0).
digit(14585, 1).
digit(17225, 1).
digit(12060, 1).
digit(11077, 0).
digit(47619, 0).
digit(19796, 0).
digit(20334, 0).
digit(3610, 0).
digit(8288, 0).
digit(54384, 1).
digit(40380, 1).
digit(46488, 0).
digit(20577, 0).
digit(2538, 0).
digit(12918, 1).
digit(33538, 0).
digit(31321, 0).
digit(6791, 0).
digit(11058, 1).
digit(42602, 1).
digit(14980, 1).
digit(6245, 0).
digit(32574, 0).
digit(22153, 1).
digit(55338, 1).
digit(32138, 1).
digit(6775, 1).
digit(24247, 1).
digit(2552, 1).
digit(15120, 0).
digit(49512, 1).
digit(23585, 1).
digit(17637, 1).
digit(11559, 0).
digit(46439, 0).
digit(52305, 0).
digit(15028, 0).
digit(38138, 0).
digit(4776, 0).
digit(48913, 1).
digit(47812, 0).
digit(394, 1).
digit(10420, 1).
digit(46680, 0).
digit(23013, 0).
digit(28981, 1).
digit(29961, 1).
digit(34653, 0).
digit(33105, 1).
digit(58903, 1).
digit(48526, 0).
digit(44290, 1).
digit(17923, 0).
digit(5058, 1).
digit(16428, 0).
digit(31432, 1).
digit(15385, 1).
digit(37559, 0).
digit(15928, 0).
digit(30281, 1).
digit(41536, 1).
digit(16928, 0).
digit(17472, 1).
digit(17960, 1).
digit(29133, 1).
digit(49085, 0).
digit(58074, 0).
digit(49903, 0).
digit(1712, 0).
digit(3128, 0).
digit(54552, 1).
digit(54259, 1).
digit(33902, 1).
digit(26665, 0).
digit(56338, 1).
digit(28186, 0).
digit(13475, 1).
digit(45204, 1).
digit(27070, 0).
digit(19094, 1).
digit(56046, 1).
digit(26055, 0).
digit(21498, 1).
digit(2440, 0).
digit(24209, 1).
digit(18581, 0).
digit(27202, 1).
digit(13492, 0).
digit(7821, 1).
digit(8019, 0).
digit(8633, 1).
digit(596, 0).
digit(33190, 1).
digit(15329, 0).
digit(50979, 1).
digit(44040, 0).
digit(34282, 1).
digit(30624, 0).
digit(18837, 0).
digit(20481, 0).
digit(44198, 1).
digit(21097, 0).
digit(25987, 0).
digit(15353, 1).
digit(36451, 1).
digit(19346, 1).
digit(45683, 1).
digit(59521, 1).
digit(56352, 0).
digit(20672, 1).
digit(11834, 0).
digit(41739, 1).
digit(13697, 1).
digit(34238, 0).
digit(587, 1).
digit(23940, 0).
digit(12765, 0).
digit(18870, 1).
digit(45557, 0).
digit(49894, 0).
digit(18799, 0).
digit(42655, 1).
digit(43829, 1).
digit(35313, 1).
digit(35252, 1).
digit(57714, 1).
digit(43142, 1).
digit(45962, 1).
digit(58408, 0).
digit(9136, 1).
digit(17477, 1).
digit(40, 1).
digit(4325, 0).
digit(48439, 0).
digit(20095, 0).
digit(38464, 1).
digit(25676, 0).
digit(41376, 0).
digit(13307, 1).
digit(17449, 0).
digit(34388, 1).
digit(20174, 0).
digit(30163, 0).
digit(20456, 1).
digit(17877, 0).
digit(9055, 0).
digit(45444, 1).
digit(49311, 0).
digit(57120, 0).
digit(43561, 0).
digit(44606, 0).
digit(17090, 1).
digit(9930, 1).
digit(25181, 0).
digit(55355, 0).
digit(42795, 0).
digit(54143, 1).
digit(18293, 1).
digit(9823, 0).
digit(29668, 1).
digit(16714, 1).
digit(27170, 0).
digit(47294, 0).
digit(29033, 1).
digit(57057, 1).
digit(45303, 0).
digit(57049, 1).
digit(48335, 1).
digit(15694, 0).
digit(54922, 0).
digit(57906, 1).
digit(21658, 0).
digit(6568, 1).
digit(9251, 1).
digit(3234, 1).
digit(55130, 0).
digit(34508, 0).
digit(16224, 1).
digit(10067, 1).
digit(4310, 0).
digit(1052, 1).
digit(22419, 1).
digit(38303, 1).
digit(50256, 0).
digit(3593, 1).
digit(21029, 0).
digit(25045, 1).
digit(53353, 0).
digit(38232, 0).
digit(29233, 1).
digit(33254, 0).
digit(5488, 0).
digit(32727, 0).
digit(20875, 1).
digit(58830, 1).
digit(12078, 1).
digit(36107, 0).
digit(45385, 1).
digit(23616, 0).
digit(30678, 0).
digit(50561, 1).
digit(3295, 1).
digit(17834, 1).
digit(32520, 0).
digit(47551, 1).
digit(19645, 0).
digit(54804, 0).
digit(26408, 0).
digit(9035, 0).
digit(57162, 0).
digit(25118, 1).
digit(2988, 1).
digit(11622, 1).
digit(52393, 0).
digit(8351, 1).
digit(26182, 1).
digit(14929, 1).
digit(20108, 1).
digit(47079, 0).
digit(29732, 0).
digit(43872, 1).
digit(31534, 1).
digit(40086, 1).
digit(4744, 0).
digit(57384, 0).
digit(26167, 1).
digit(33874, 0).
digit(11855, 1).
digit(48180, 0).
digit(48613, 0).
digit(32605, 0).
digit(57115, 0).
digit(41810, 0).
digit(39484, 1).
digit(5516, 1).
digit(41638, 1).
digit(28286, 0).
digit(4239, 0).
digit(13547, 0).
digit(44449, 0).
digit(30179, 1).
digit(33214, 0).
digit(45083, 0).
digit(59872, 1).
digit(53195, 0).
digit(3734, 0).
digit(23113, 0).
digit(11715, 1).
digit(30940, 1).
digit(34757, 1).
digit(55380, 1).
digit(9101, 0).
digit(20036, 1).
digit(260, 0).
digit(59859, 0).
digit(22637, 0).
digit(39051, 1).
digit(53404, 0).
digit(52368, 1).
digit(39155, 0).
digit(43708, 1).
digit(17655, 0).
digit(16244, 0).
digit(29446, 0).
digit(44529, 1).
digit(23104, 0).
digit(41723, 1).
digit(14463, 1).
digit(17133, 0).
digit(55110, 0).
digit(19029, 1).
digit(55402, 0).
digit(20838, 1).
digit(18451, 1).
digit(56603, 1).
digit(7696, 1).
digit(45042, 0).
digit(16643, 1).
digit(42039, 1).
digit(23499, 0).
digit(11069, 1).
digit(56810, 1).
digit(2136, 1).
digit(36591, 1).
digit(58102, 0).
digit(23134, 1).
digit(23523, 1).
digit(31061, 1).
digit(11765, 1).
digit(55806, 0).
digit(35261, 0).
digit(33658, 1).
digit(10922, 1).
digit(36312, 0).
digit(18563, 0).
digit(9971, 0).
digit(43927, 1).
digit(41799, 1).
digit(54213, 1).
digit(3602, 1).
digit(19804, 1).
digit(44446, 1).
digit(40251, 0).
digit(57091, 0).
digit(58481, 1).
digit(21523, 0).
digit(59176, 1).
digit(13854, 0).
digit(37902, 0).
digit(23627, 0).
digit(25540, 1).
digit(54963, 1).
digit(30303, 1).
digit(19060, 0).
digit(24393, 0).
digit(44745, 1).
digit(18411, 1).
digit(1897, 0).
digit(21049, 0).
digit(27200, 0).
digit(48025, 0).
digit(14252, 1).
digit(23643, 0).
digit(51952, 1).
digit(45627, 1).
digit(24510, 0).
digit(20175, 0).
digit(44060, 0).
digit(52617, 0).
digit(14560, 1).
digit(31375, 1).
digit(37232, 1).
digit(39454, 1).
digit(6441, 0).
digit(15264, 1).
digit(35905, 0).
digit(55310, 1).
digit(5286, 0).
digit(24525, 0).
digit(39362, 1).
digit(2079, 0).
digit(51741, 1).
digit(5801, 0).
digit(7496, 1).
digit(25061, 1).
digit(7806, 1).
digit(10781, 0).
digit(11217, 0).
digit(30148, 1).
digit(14356, 0).
digit(54800, 0).
digit(24345, 0).
digit(28925, 1).
digit(43791, 1).
digit(37723, 0).
digit(42275, 0).
digit(8222, 1).
digit(37814, 0).
digit(47422, 0).
digit(24580, 1).
digit(9735, 1).
digit(50630, 0).
digit(39157, 1).
digit(38744, 0).
digit(16371, 1).
digit(18115, 1).
digit(49883, 0).
digit(40390, 0).
digit(53194, 1).
digit(11945, 0).
digit(40850, 0).
digit(56851, 1).
digit(17611, 0).
digit(52597, 0).
digit(11003, 0).
digit(57305, 1).
digit(9985, 0).
digit(3461, 0).
digit(59854, 1).
digit(5615, 0).
digit(24952, 0).
digit(24139, 1).
digit(28968, 0).
digit(14472, 0).
digit(40509, 1).
digit(51375, 0).
digit(44958, 0).
digit(234, 0).
digit(31679, 1).
digit(29320, 1).
digit(15505, 0).
digit(46235, 0).
digit(45722, 0).
digit(43653, 1).
digit(17064, 0).
digit(50310, 0).
digit(29570, 0).
digit(34241, 1).
digit(14948, 1).
digit(13487, 1).
digit(19454, 0).
digit(5429, 1).
digit(12371, 1).
digit(26687, 1).
digit(24693, 0).
digit(39077, 1).
digit(33253, 0).
digit(39295, 0).
digit(21744, 1).
digit(7454, 1).
digit(35175, 0).
digit(5698, 0).
digit(55765, 0).
digit(8387, 0).
digit(9550, 1).
digit(52766, 0).
digit(49623, 1).
digit(52738, 1).
digit(54608, 0).
digit(54873, 0).
digit(44093, 1).
digit(21699, 1).
digit(54755, 0).
digit(45966, 0).
digit(55594, 1).
digit(18259, 1).
digit(28671, 0).
digit(43981, 0).
digit(29364, 0).
digit(46275, 1).
digit(25694, 0).
digit(22330, 1).
digit(29903, 0).
digit(10458, 1).
digit(19884, 1).
digit(47434, 1).
digit(34582, 0).
digit(39817, 0).
digit(53687, 1).
digit(5978, 0).
digit(51895, 1).
digit(18284, 0).
digit(38126, 0).
digit(47697, 1).
digit(56836, 1).
digit(35642, 1).
digit(15637, 0).
digit(4861, 1).
digit(14314, 1).
digit(10026, 0).
digit(33648, 1).
digit(27454, 1).
digit(9717, 0).
digit(56801, 0).
digit(15559, 0).
digit(24136, 1).
digit(14612, 1).
digit(31764, 1).
digit(15790, 1).
digit(14000, 1).
digit(51980, 0).
digit(27035, 0).
digit(15065, 0).
digit(37499, 1).
digit(41291, 0).
digit(33228, 0).
digit(14631, 1).
digit(25421, 1).
digit(21533, 0).
digit(49538, 1).
digit(35303, 1).
digit(4884, 1).
digit(9544, 0).
digit(16308, 1).
digit(8895, 1).
digit(46093, 1).
digit(52675, 1).
digit(5708, 1).
digit(18286, 1).
digit(57822, 0).
digit(52395, 1).
digit(20475, 1).
digit(32541, 0).
digit(45598, 1).
digit(24222, 1).
digit(43162, 1).
digit(45167, 1).
digit(9103, 1).
digit(51671, 1).
digit(17430, 1).
digit(50541, 1).
digit(56099, 1).
digit(38102, 0).
digit(19176, 1).
digit(9175, 1).
digit(19009, 0).
digit(13177, 1).
digit(7025, 0).
digit(28819, 1).
digit(13617, 1).
digit(8625, 1).
digit(55931, 1).
digit(7807, 0).
digit(33368, 1).
digit(39570, 0).
digit(42911, 1).
digit(13726, 0).
digit(8250, 0).
digit(45941, 1).
digit(1483, 1).
digit(13830, 0).
digit(59958, 1).
digit(51739, 1).
digit(53080, 1).
digit(39503, 0).
digit(20911, 0).
digit(44582, 1).
digit(24895, 1).
digit(16651, 1).
digit(51539, 1).
digit(42803, 1).
digit(35660, 0).
digit(33647, 0).
digit(56704, 0).
digit(2747, 1).
digit(30301, 1).
digit(12793, 1).
digit(19543, 0).
digit(2473, 0).
digit(34077, 1).
digit(45515, 0).
digit(48834, 0).
digit(40471, 0).
digit(4089, 1).
digit(4444, 0).
digit(21051, 1).
digit(44698, 1).
digit(32568, 0).
digit(34708, 0).
digit(119, 0).
digit(57963, 0).
digit(49278, 0).
digit(18792, 1).
digit(52276, 0).
digit(34051, 0).
digit(10221, 0).
digit(53040, 1).
digit(22344, 0).
digit(443, 1).
digit(21766, 0).
digit(7363, 1).
digit(59681, 0).
digit(35634, 0).
digit(13799, 0).
digit(18921, 0).
digit(52530, 1).
digit(43300, 1).
digit(51110, 1).
digit(55503, 0).
digit(27913, 1).
digit(49888, 1).
digit(58479, 1).
digit(42731, 1).
digit(25666, 1).
digit(3107, 0).
digit(15240, 1).
digit(3540, 1).
digit(33963, 0).
digit(41658, 1).
digit(8258, 1).
digit(36141, 1).
digit(12799, 0).
digit(2495, 1).
digit(33332, 1).
digit(41643, 0).
digit(59595, 0).
digit(25304, 0).
digit(57473, 1).
digit(44474, 0).
digit(37286, 1).
digit(57364, 0).
digit(29143, 1).
digit(29088, 1).
digit(47004, 1).
digit(34369, 0).
digit(32365, 1).
digit(49392, 1).
digit(7589, 0).
digit(24480, 0).
digit(14158, 0).
digit(23293, 0).
digit(52710, 1).
digit(25386, 0).
digit(5443, 1).
digit(18028, 0).
digit(43443, 0).
digit(3379, 1).
digit(55277, 0).
digit(36794, 1).
digit(18407, 1).
digit(17151, 1).
digit(25701, 0).
digit(33904, 0).
digit(52799, 0).
digit(53219, 0).
digit(13947, 1).
digit(40150, 0).
digit(18324, 0).
digit(4500, 0).
digit(3328, 0).
digit(35473, 0).
digit(49603, 1).
digit(13625, 1).
digit(27759, 1).
digit(54371, 0).
digit(30080, 0).
digit(6575, 1).
digit(48742, 0).
digit(32743, 1).
digit(38473, 0).
digit(56101, 0).
digit(39217, 1).
digit(13725, 1).
digit(10069, 0).
digit(26298, 0).
digit(29586, 1).
digit(42769, 1).
digit(37201, 1).
digit(53524, 0).
digit(25853, 1).
digit(1372, 0).
digit(50063, 1).
digit(57647, 1).
digit(18321, 1).
digit(56686, 0).
digit(29517, 1).
digit(57356, 0).
digit(46398, 0).
digit(3897, 1).
digit(30742, 1).
digit(44413, 0).
digit(40586, 0).
digit(59331, 0).
digit(52552, 1).
digit(55879, 0).
digit(32951, 0).
digit(26241, 0).
digit(35681, 0).
digit(40327, 1).
digit(1084, 1).
digit(31297, 0).
digit(35516, 0).
digit(14749, 0).
digit(50930, 0).
digit(36110, 1).
digit(57715, 1).
digit(59, 1).
digit(53787, 1).
digit(37497, 0).
digit(42081, 0).
digit(7649, 0).
digit(52621, 0).
digit(29034, 0).
digit(46637, 1).
digit(21242, 0).
digit(31604, 0).
digit(10323, 0).
digit(450, 1).
digit(47435, 1).
digit(32765, 1).
digit(27489, 1).
digit(7954, 0).
digit(39960, 0).
digit(46113, 1).
digit(6822, 1).
digit(17050, 0).
digit(18443, 0).
digit(8527, 0).
digit(4159, 0).
digit(17932, 0).
digit(10604, 0).
digit(37697, 0).
digit(36031, 0).
digit(50850, 1).
digit(53030, 0).
digit(31389, 0).
digit(18958, 1).
digit(17394, 1).
digit(5020, 1).
digit(20279, 1).
digit(41554, 1).
digit(56025, 1).
digit(53162, 1).
digit(49437, 1).
digit(18725, 0).
digit(44633, 1).
digit(5560, 0).
digit(4872, 1).
digit(15504, 0).
digit(49860, 1).
digit(16761, 0).
digit(3394, 1).
digit(11541, 0).
digit(24399, 0).
digit(41606, 0).
digit(56870, 1).
digit(43808, 1).
digit(27582, 1).
digit(19652, 1).
digit(18572, 0).
digit(3132, 1).
digit(34457, 0).
digit(11911, 0).
digit(9505, 0).
digit(14293, 1).
digit(10257, 1).
digit(54221, 0).
digit(24628, 1).
digit(12584, 0).
digit(51035, 1).
digit(8233, 0).
digit(32824, 1).
digit(41012, 0).
digit(5886, 0).
digit(13397, 0).
digit(46144, 0).
digit(17987, 0).
digit(27663, 0).
digit(23953, 1).
digit(23882, 0).
digit(5327, 0).
digit(16483, 1).
digit(50344, 0).
digit(19308, 0).
digit(51415, 0).
digit(46225, 1).
digit(1242, 1).
digit(39735, 0).
digit(21166, 1).
digit(11851, 0).
digit(47267, 0).
digit(52554, 0).
digit(28746, 0).
digit(10378, 1).
digit(4942, 1).
digit(44159, 0).
digit(17305, 1).
digit(29883, 1).
digit(38525, 1).
digit(55115, 0).
digit(58913, 1).
digit(40877, 0).
digit(53339, 0).
digit(32979, 1).
digit(35798, 1).
digit(32034, 1).
digit(50729, 0).
digit(37728, 1).
digit(40240, 0).
digit(52619, 1).
digit(55991, 0).
digit(41360, 1).
digit(56489, 0).
digit(24762, 1).
digit(13631, 1).
digit(45952, 0).
digit(23067, 0).
digit(32273, 0).
digit(21555, 1).
digit(45237, 1).
digit(40746, 0).
digit(34889, 1).
digit(40888, 0).
digit(21767, 1).
digit(45955, 1).
digit(55366, 0).
digit(22857, 0).
digit(38388, 1).
digit(37113, 0).
digit(38042, 0).
digit(20483, 0).
digit(3797, 1).
digit(12057, 0).
digit(38123, 0).
digit(49919, 1).
digit(23362, 0).
digit(38231, 0).
digit(57659, 0).
digit(38264, 1).
digit(21308, 0).
digit(34086, 0).
digit(54620, 0).
digit(21721, 0).
digit(23030, 1).
digit(2872, 1).
digit(9884, 1).
digit(8070, 1).
digit(41156, 0).
digit(2914, 0).
digit(22144, 0).
digit(9106, 1).
digit(35039, 0).
digit(11639, 0).
digit(30200, 1).
digit(42045, 0).
digit(44829, 1).
digit(21735, 0).
digit(30605, 0).
digit(17968, 0).
digit(57614, 1).
digit(54290, 0).
digit(31129, 1).
digit(21573, 0).
digit(46111, 0).
digit(24541, 0).
digit(14991, 0).
digit(15105, 1).
digit(24563, 1).
digit(30827, 0).
digit(903, 0).
digit(17243, 1).
digit(53502, 0).
digit(20810, 1).
digit(43972, 1).
digit(2762, 1).
digit(37614, 1).
digit(3882, 0).
digit(25163, 1).
digit(35639, 0).
digit(41897, 1).
digit(45002, 1).
digit(35465, 0).
digit(53212, 0).
digit(24371, 0).
digit(44886, 1).
digit(484, 1).
digit(19058, 1).
digit(14595, 1).
digit(1930, 0).
digit(9687, 0).
digit(24988, 0).
digit(57775, 1).
digit(9773, 0).
digit(26027, 0).
digit(6062, 1).
digit(5690, 0).
digit(4343, 0).
digit(13454, 0).
digit(47236, 1).
digit(18095, 1).
digit(38543, 0).
digit(21635, 0).
digit(1742, 0).
digit(20357, 0).
digit(52126, 0).
digit(26103, 0).
digit(40605, 1).
digit(47734, 1).
digit(23809, 0).
digit(14466, 1).
digit(382, 1).
digit(7440, 1).
digit(46213, 1).
digit(31366, 1).
digit(29128, 0).
digit(28295, 0).
digit(9440, 0).
digit(49685, 0).
digit(27859, 0).
digit(16525, 1).
digit(15974, 1).
digit(34416, 0).
digit(4713, 0).
digit(8838, 0).
digit(37543, 0).
digit(30592, 1).
digit(18393, 1).
digit(20128, 0).
digit(27920, 1).
digit(22317, 0).
digit(29560, 0).
digit(3729, 1).
digit(20659, 1).
digit(37917, 0).
digit(31784, 1).
digit(24105, 0).
digit(57960, 1).
digit(11267, 0).
digit(11654, 0).
digit(28736, 1).
digit(16062, 1).
digit(12634, 1).
digit(34033, 1).
digit(40915, 0).
digit(39888, 1).
digit(11275, 1).
digit(30209, 1).
digit(9043, 1).
digit(2144, 0).
digit(35620, 0).
digit(32367, 1).
digit(48863, 0).
digit(8812, 0).
digit(50480, 0).
digit(21142, 0).
digit(32843, 0).
digit(46127, 1).
digit(16565, 1).
digit(48547, 0).
digit(20827, 0).
digit(28681, 1).
digit(49351, 0).
digit(38700, 1).
digit(26905, 1).
digit(20373, 1).
digit(25027, 1).
digit(23133, 1).
digit(30512, 1).
digit(39828, 1).
digit(27394, 0).
digit(55214, 1).
digit(29715, 1).
digit(31045, 1).
digit(13227, 1).
digit(35897, 1).
digit(14530, 0).
digit(40735, 1).
digit(8177, 1).
digit(25137, 0).
digit(23032, 0).
digit(44988, 1).
digit(5187, 0).
digit(47338, 1).
digit(7709, 0).
digit(20163, 1).
digit(22555, 1).
digit(53932, 0).
digit(40340, 0).
digit(56419, 0).
digit(35352, 1).
digit(28479, 1).
digit(4810, 1).
digit(52450, 1).
digit(22636, 0).
digit(50638, 0).
digit(56174, 0).
digit(25253, 1).
digit(42424, 1).
digit(3367, 0).
digit(27003, 1).
digit(16988, 1).
digit(24840, 0).
digit(11514, 0).
digit(3851, 1).
digit(48368, 1).
digit(11791, 0).
digit(19150, 1).
digit(51598, 0).
digit(55825, 1).
digit(13701, 1).
digit(10345, 1).
digit(40802, 1).
digit(36422, 1).
digit(49952, 0).
digit(14461, 0).
digit(15971, 0).
digit(40219, 1).
digit(10700, 0).
digit(34045, 0).
digit(13808, 0).
digit(27811, 0).
digit(45830, 0).
digit(28821, 0).
digit(12313, 0).
digit(17209, 1).
digit(53667, 1).
digit(20256, 0).
digit(30207, 0).
digit(13545, 0).
digit(1450, 1).
digit(51867, 1).
digit(41269, 1).
digit(19027, 0).
digit(45572, 1).
digit(43963, 0).
digit(20482, 0).
digit(36657, 1).
digit(12136, 1).
digit(42989, 0).
digit(20471, 0).
digit(67, 1).
digit(51210, 0).
digit(40262, 0).
digit(26369, 0).
digit(13730, 0).
digit(14767, 1).
digit(14722, 0).
digit(27978, 1).
digit(12575, 0).
digit(59704, 0).
digit(29288, 1).
digit(46039, 0).
digit(1831, 1).
digit(51813, 1).
digit(39639, 0).
digit(56750, 1).
digit(18737, 0).
digit(38062, 0).
digit(57400, 1).
digit(16619, 0).
digit(41424, 1).
digit(5622, 0).
digit(12360, 0).
digit(42028, 1).
digit(13175, 0).
digit(49395, 1).
digit(52392, 1).
digit(24776, 0).
digit(14131, 0).
digit(30123, 0).
digit(15017, 0).
digit(15189, 0).
digit(50390, 0).
digit(10844, 0).
digit(59979, 1).
digit(57931, 1).
digit(56506, 1).
digit(36618, 1).
digit(29750, 0).
digit(24634, 1).
digit(26818, 0).
digit(29746, 1).
digit(7221, 1).
digit(26713, 0).
digit(58858, 0).
digit(33149, 0).
digit(5675, 1).
digit(51936, 1).
digit(13411, 0).
digit(6917, 0).
digit(11228, 1).
digit(1680, 1).
digit(8788, 0).
digit(53487, 0).
digit(48960, 0).
digit(8083, 1).
digit(46919, 1).
digit(26040, 0).
digit(5340, 0).
digit(25325, 0).
digit(55963, 1).
digit(7691, 0).
digit(4412, 1).
digit(6700, 0).
digit(3431, 1).
digit(32834, 0).
digit(6043, 1).
digit(6320, 0).
digit(29515, 0).
digit(50646, 1).
digit(11607, 1).
digit(29677, 0).
digit(28993, 0).
digit(23340, 1).
digit(14853, 1).
digit(5601, 1).
digit(32708, 0).
digit(59671, 1).
digit(28489, 0).
digit(13822, 0).
digit(45858, 1).
digit(49076, 1).
digit(34152, 0).
digit(24888, 1).
digit(13164, 0).
digit(25395, 0).
digit(16975, 0).
digit(17048, 0).
digit(56755, 0).
digit(5563, 0).
digit(23852, 1).
digit(10248, 1).
digit(58691, 1).
digit(44542, 1).
digit(35792, 1).
digit(21737, 1).
digit(20349, 0).
digit(59105, 0).
digit(59363, 0).
digit(38939, 0).
digit(50842, 1).
digit(53657, 0).
digit(2227, 1).
digit(37915, 1).
digit(33665, 0).
digit(47902, 0).
digit(2094, 1).
digit(26812, 1).
digit(44919, 1).
digit(24555, 1).
digit(34426, 1).
digit(54059, 1).
digit(30127, 0).
digit(51509, 1).
digit(27246, 0).
digit(15330, 0).
digit(27822, 0).
digit(20392, 0).
digit(13260, 1).
digit(54108, 0).
digit(35278, 1).
digit(47340, 1).
digit(17518, 0).
digit(48414, 0).
digit(14919, 0).
digit(11268, 1).
digit(26009, 1).
digit(51059, 0).
digit(4070, 1).
digit(9922, 0).
digit(48600, 1).
digit(37598, 1).
digit(20327, 1).
digit(51377, 1).
digit(25518, 0).
digit(6296, 0).
digit(35823, 0).
digit(10007, 1).
digit(22444, 0).
digit(8379, 1).
digit(3135, 0).
digit(39232, 1).
digit(38903, 0).
digit(37619, 0).
digit(11518, 1).
digit(18715, 0).
digit(7506, 0).
digit(59142, 0).
digit(41315, 0).
digit(6958, 0).
digit(16904, 1).
digit(12907, 0).
digit(40563, 0).
digit(45905, 0).
digit(34656, 0).
digit(6341, 0).
digit(21019, 1).
digit(53916, 0).
digit(8108, 0).
digit(10168, 0).
digit(58586, 1).
digit(28488, 1).
digit(835, 1).
digit(11269, 0).
digit(10376, 1).
digit(40494, 0).
digit(17232, 1).
digit(8284, 0).
digit(24367, 0).
digit(18650, 0).
digit(12365, 1).
digit(14425, 0).
digit(24013, 1).
digit(24714, 1).
digit(54642, 0).
digit(25651, 1).
digit(40320, 0).
digit(3529, 0).
digit(39612, 0).
digit(18880, 1).
digit(25567, 0).
digit(19303, 0).
digit(44703, 0).
digit(11504, 1).
digit(14746, 0).
digit(33723, 0).
digit(35972, 0).
digit(28824, 0).
digit(37629, 1).
digit(15361, 1).
digit(50326, 1).
digit(33485, 0).
digit(58489, 0).
digit(12508, 1).
digit(26722, 0).
digit(34164, 1).
digit(56491, 1).
digit(10123, 1).
digit(38540, 0).
digit(43480, 1).
digit(55752, 0).
digit(30126, 1).
digit(50956, 1).
digit(6421, 0).
digit(47069, 0).
digit(48697, 1).
digit(55502, 1).
digit(36589, 0).
digit(14092, 0).
digit(20046, 1).
digit(32577, 1).
digit(16835, 1).
digit(31727, 1).
digit(49317, 1).
digit(14849, 1).
digit(34839, 0).
digit(31033, 1).
digit(13013, 0).
digit(5543, 1).
digit(13140, 1).
digit(50362, 1).
digit(4978, 1).
digit(5464, 0).
digit(4686, 0).
digit(51436, 1).
digit(54464, 0).
digit(26436, 0).
digit(10404, 1).
digit(20724, 0).
digit(33140, 1).
digit(14858, 1).
digit(3041, 1).
digit(14693, 0).
digit(1181, 1).
digit(58387, 1).
digit(57810, 0).
digit(4415, 0).
digit(59659, 0).
digit(39849, 1).
digit(31830, 0).
digit(1179, 0).
digit(7786, 0).
digit(39367, 1).
digit(15523, 1).
digit(20537, 0).
digit(47139, 0).
digit(51797, 0).
digit(54349, 0).
digit(24148, 0).
digit(26717, 1).
digit(27050, 0).
digit(29199, 0).
digit(22093, 1).
digit(45927, 1).
digit(41245, 0).
digit(11236, 1).
digit(39029, 0).
digit(1904, 0).
digit(39504, 0).
digit(23511, 1).
digit(47785, 1).
digit(54943, 0).
digit(10750, 0).
digit(3149, 1).
digit(8464, 1).
digit(58891, 1).
digit(42084, 0).
digit(12205, 0).
digit(38097, 1).
digit(47167, 0).
digit(7542, 1).
digit(21874, 0).
digit(10204, 0).
digit(14775, 0).
digit(9069, 1).
digit(8291, 1).
digit(18641, 0).
digit(53298, 0).
digit(29695, 0).
digit(14840, 0).
digit(31943, 0).
digit(34854, 0).
digit(46947, 0).
digit(53805, 1).
digit(30965, 0).
digit(42852, 1).
digit(11043, 1).
digit(55067, 1).
digit(54320, 0).
digit(32966, 0).
digit(4152, 1).
digit(55920, 1).
digit(40276, 1).
digit(42351, 1).
digit(59984, 1).
digit(48232, 1).
digit(41419, 1).
digit(46557, 0).
digit(45728, 0).
digit(38662, 0).
digit(37158, 0).
digit(7378, 0).
digit(54533, 1).
digit(12168, 0).
digit(28018, 1).
digit(9563, 1).
digit(15608, 1).
digit(10947, 1).
digit(47360, 1).
digit(55846, 1).
digit(11502, 1).
digit(24267, 1).
digit(41069, 0).
digit(55768, 1).
digit(55411, 1).
digit(27583, 0).
digit(15561, 1).
digit(36692, 0).
digit(47333, 1).
digit(12056, 1).
digit(50828, 0).
digit(2998, 1).
digit(31138, 0).
digit(55408, 0).
digit(19061, 0).
digit(56605, 1).
digit(54942, 0).
digit(46227, 0).
digit(38561, 1).
digit(30684, 1).
digit(30160, 0).
digit(32413, 1).
digit(31950, 1).
digit(31048, 1).
digit(27201, 1).
digit(5083, 0).
digit(12510, 1).
digit(59919, 1).
digit(10384, 0).
digit(1859, 1).
digit(29222, 1).
digit(25451, 1).
digit(11792, 0).
digit(56945, 1).
digit(56071, 1).
digit(26007, 0).
digit(30996, 1).
digit(3373, 1).
digit(48187, 0).
digit(6614, 0).
digit(26959, 1).
digit(23885, 1).
digit(40069, 1).
digit(23045, 1).
digit(32144, 0).
digit(52655, 0).
digit(40423, 1).
digit(47947, 1).
digit(38579, 0).
digit(33769, 1).
digit(3778, 0).
digit(24122, 1).
digit(6925, 1).
digit(2326, 1).
digit(24038, 1).
digit(36426, 1).
digit(3478, 1).
digit(18278, 0).
digit(22874, 0).
digit(31642, 1).
digit(5681, 1).
digit(27697, 0).
digit(38974, 1).
digit(6538, 0).
digit(32210, 0).
digit(21101, 1).
digit(41027, 0).
digit(1365, 1).
digit(25485, 1).
digit(51061, 0).
digit(17454, 0).
digit(21474, 1).
digit(25065, 0).
digit(14992, 1).
digit(16061, 1).
digit(23541, 0).
digit(49292, 1).
digit(59878, 0).
digit(169, 0).
digit(7127, 0).
digit(33844, 0).
digit(20852, 1).
digit(55027, 0).
digit(20953, 0).
digit(24, 1).
digit(5938, 0).
digit(3849, 0).
digit(16081, 0).
digit(2616, 1).
digit(4953, 1).
digit(21064, 0).
digit(24339, 1).
digit(24721, 1).
digit(8456, 1).
digit(21200, 1).
digit(14221, 1).
digit(18176, 1).
digit(3822, 1).
digit(19071, 1).
digit(36045, 1).
digit(37802, 1).
digit(16352, 0).
digit(13643, 1).
digit(23988, 0).
digit(47969, 1).
digit(6531, 1).
digit(42182, 1).
digit(25310, 1).
digit(27893, 0).
digit(41591, 1).
digit(53350, 0).
digit(59615, 0).
digit(32675, 1).
digit(25529, 0).
digit(36033, 1).
digit(59155, 0).
digit(8863, 0).
digit(42618, 1).
digit(8934, 1).
digit(21469, 0).
digit(44680, 1).
digit(6609, 1).
digit(22179, 1).
digit(35722, 1).
digit(52088, 1).
digit(56644, 1).
digit(45277, 1).
digit(30117, 1).
digit(5994, 1).
digit(18760, 1).
digit(57069, 1).
digit(7004, 0).
digit(53405, 1).
digit(40922, 0).
digit(38706, 0).
digit(44883, 0).
digit(29772, 0).
digit(49622, 0).
digit(23601, 0).
digit(46780, 1).
digit(52865, 0).
digit(56389, 0).
digit(26939, 1).
digit(38461, 0).
digit(33237, 0).
digit(49713, 0).
digit(29773, 1).
digit(36687, 1).
digit(59755, 0).
digit(25725, 0).
digit(39446, 0).
digit(14327, 1).
digit(790, 0).
digit(32086, 0).
digit(3675, 1).
digit(10346, 1).
digit(9975, 1).
digit(44344, 0).
digit(7272, 0).
digit(22135, 1).
digit(30214, 1).
digit(16868, 0).
digit(40833, 0).
digit(47791, 0).
digit(31427, 1).
digit(8388, 1).
digit(31937, 0).
digit(51603, 1).
digit(8369, 0).
digit(5844, 0).
digit(35275, 0).
digit(45284, 1).
digit(10975, 0).
digit(4903, 1).
digit(40103, 0).
digit(4067, 0).
digit(20274, 1).
digit(39364, 1).
digit(42968, 1).
digit(6104, 1).
digit(41534, 0).
digit(57154, 0).
digit(55892, 0).
digit(30225, 1).
digit(3806, 0).
digit(51108, 0).
digit(48488, 0).
digit(6946, 1).
digit(27289, 0).
digit(5132, 1).
digit(58466, 1).
digit(12929, 1).
digit(20052, 1).
digit(57430, 1).
digit(10849, 0).
digit(57785, 1).
digit(3963, 1).
digit(366, 1).
digit(47751, 1).
digit(45743, 1).
digit(26802, 1).
digit(10318, 1).
digit(12259, 0).
digit(50139, 1).
digit(9790, 0).
digit(41627, 0).
digit(25724, 1).
digit(33774, 1).
digit(18649, 1).
digit(20974, 1).
digit(48542, 0).
digit(51057, 0).
digit(9451, 0).
digit(23396, 1).
digit(47935, 1).
digit(58523, 0).
digit(56637, 1).
digit(35856, 1).
digit(11867, 1).
digit(38424, 1).
digit(29187, 0).
digit(31567, 0).
digit(43469, 1).
digit(41956, 1).
digit(18116, 1).
digit(11107, 1).
digit(12971, 1).
digit(21069, 0).
digit(54844, 0).
digit(58123, 1).
digit(51010, 1).
digit(41685, 1).
digit(22476, 1).
digit(44660, 0).
digit(38897, 1).
digit(58353, 0).
digit(48793, 1).
digit(56584, 1).
digit(27330, 1).
digit(57769, 0).
digit(51136, 1).
digit(36809, 0).
digit(33868, 1).
digit(5881, 0).
digit(25166, 0).
digit(9027, 0).
digit(54543, 1).
digit(38207, 1).
digit(32625, 0).
digit(25794, 1).
digit(9590, 1).
digit(32810, 0).
digit(32179, 1).
digit(8772, 1).
digit(39839, 0).
digit(6723, 0).
digit(13470, 1).
digit(52822, 1).
digit(57252, 0).
digit(35632, 0).
digit(41249, 1).
digit(48604, 0).
digit(35536, 0).
digit(28718, 1).
digit(39102, 1).
digit(52609, 0).
digit(43269, 1).
digit(33496, 0).
digit(13569, 1).
digit(8172, 0).
digit(2729, 0).
digit(8575, 1).
digit(15492, 0).
digit(47696, 0).
digit(12837, 0).
digit(22297, 0).
digit(48026, 0).
digit(14034, 0).
digit(37036, 1).
digit(7160, 1).
digit(11187, 0).
digit(5082, 0).
digit(45058, 0).
digit(24329, 1).
digit(13266, 1).
digit(16607, 0).
digit(19811, 0).
digit(1548, 1).
digit(55686, 1).
digit(46054, 0).
digit(26726, 1).
digit(55624, 0).
digit(25773, 0).
digit(58361, 1).
digit(59421, 0).
digit(21560, 0).
digit(15451, 1).
digit(58532, 0).
digit(15325, 0).
digit(31628, 0).
digit(20990, 1).
digit(40082, 0).
digit(52978, 0).
digit(52693, 1).
digit(16468, 1).
digit(24279, 1).
digit(51772, 1).
digit(43778, 0).
digit(48403, 0).
digit(38949, 0).
digit(15068, 1).
digit(57795, 0).
digit(37285, 1).
digit(31199, 0).
digit(12706, 1).
digit(35163, 1).
digit(12647, 1).
digit(14705, 0).
digit(24996, 1).
digit(20018, 1).
digit(5149, 1).
digit(18761, 1).
digit(9173, 1).
digit(23303, 1).
digit(22245, 0).
digit(37954, 0).
digit(56454, 0).
digit(46438, 0).
digit(7553, 1).
digit(37849, 1).
digit(6384, 1).
digit(16983, 1).
digit(33326, 1).
digit(9881, 0).
digit(55232, 1).
digit(276, 1).
digit(43232, 1).
digit(13913, 0).
digit(18878, 0).
digit(20225, 0).
digit(1607, 1).
digit(59460, 1).
digit(59784, 0).
digit(15776, 1).
digit(32886, 1).
digit(10230, 0).
digit(41374, 0).
digit(50235, 0).
digit(30535, 1).
digit(58829, 0).
digit(19749, 1).
digit(59605, 0).
digit(53085, 0).
digit(49118, 1).
digit(46236, 1).
digit(19937, 1).
digit(14325, 0).
digit(42205, 1).
digit(13160, 1).
digit(1839, 1).
digit(17627, 1).
digit(34731, 0).
digit(6640, 1).
digit(48284, 0).
digit(15789, 1).
digit(34923, 1).
digit(24793, 1).
digit(15557, 1).
digit(29499, 1).
digit(39935, 0).
digit(25324, 1).
digit(3410, 0).
digit(34423, 1).
digit(31307, 1).
digit(52624, 1).
digit(37970, 0).
digit(32632, 1).
digit(4319, 1).
digit(8040, 1).
digit(12716, 0).
digit(46273, 0).
digit(46276, 1).
digit(6001, 0).
digit(54351, 0).
digit(42632, 0).
digit(39474, 0).
digit(8513, 1).
digit(16045, 0).
digit(15095, 1).
digit(31490, 0).
digit(14137, 0).
digit(43406, 1).
digit(13855, 1).
digit(46822, 0).
digit(31280, 1).
digit(34667, 1).
digit(2619, 1).
digit(28906, 1).
digit(38463, 1).
digit(19902, 0).
digit(8096, 0).
digit(10010, 0).
digit(16640, 0).
digit(9018, 0).
digit(12680, 1).
digit(48167, 0).
digit(13433, 1).
digit(3723, 1).
digit(50534, 0).
digit(6331, 0).
digit(55029, 0).
digit(7941, 1).
digit(50990, 0).
digit(9067, 0).
digit(58120, 1).
digit(41556, 0).
digit(17797, 0).
digit(8028, 0).
digit(20731, 0).
digit(27712, 0).
digit(50839, 0).
digit(30978, 1).
digit(4496, 0).
digit(11968, 1).
digit(36789, 0).
digit(39856, 0).
digit(58767, 1).
digit(43685, 0).
digit(3070, 1).
digit(36851, 1).
digit(30222, 0).
digit(57507, 0).
digit(32092, 0).
digit(29551, 1).
digit(50731, 1).
digit(5520, 1).
digit(23088, 0).
digit(34513, 1).
digit(13892, 1).
digit(10691, 0).
digit(35973, 0).
digit(37354, 1).
digit(59944, 0).
digit(21601, 1).
digit(22224, 0).
digit(46430, 0).
digit(53448, 1).
digit(21768, 0).
digit(30955, 0).
digit(7469, 1).
digit(10523, 0).
digit(28108, 0).
digit(26581, 1).
digit(32778, 0).
digit(9890, 0).
digit(48326, 1).
digit(4289, 0).
digit(20870, 1).
digit(8335, 0).
digit(46703, 0).
digit(1472, 1).
digit(42342, 0).
digit(8087, 1).
digit(50566, 0).
digit(44862, 0).
digit(45165, 0).
digit(57339, 1).
digit(51048, 1).
digit(22065, 1).
digit(26696, 1).
digit(28101, 0).
digit(17334, 0).
digit(32842, 1).
digit(53469, 0).
digit(33024, 1).
digit(41729, 0).
digit(23280, 1).
digit(13197, 0).
digit(55275, 0).
digit(19628, 0).
digit(8813, 0).
digit(29902, 1).
digit(31755, 1).
digit(45115, 0).
digit(33370, 1).
digit(14345, 1).
digit(20463, 0).
digit(58461, 0).
digit(29190, 0).
digit(42393, 1).
digit(48299, 0).
digit(15300, 1).
digit(20584, 0).
digit(59010, 1).
digit(33654, 1).
digit(17169, 0).
digit(8540, 0).
digit(32097, 0).
digit(56007, 1).
digit(34355, 0).
digit(6292, 0).
digit(12664, 1).
digit(20231, 1).
digit(55907, 0).
digit(41884, 1).
digit(6554, 1).
digit(24322, 1).
digit(20654, 1).
digit(29414, 0).
digit(38764, 1).
digit(25153, 1).
digit(27693, 1).
digit(59427, 1).
digit(46433, 1).
digit(30484, 0).
digit(54411, 0).
digit(8806, 1).
digit(50338, 1).
digit(12337, 0).
digit(47830, 1).
digit(59249, 0).
digit(37225, 1).
digit(46445, 1).
digit(43468, 0).
digit(4682, 0).
digit(52777, 1).
digit(747, 1).
digit(50237, 1).
digit(8682, 0).
digit(52174, 1).
digit(41609, 0).
digit(2015, 1).
digit(49281, 0).
digit(52928, 0).
digit(10692, 1).
digit(39268, 1).
digit(6229, 1).
digit(56192, 1).
digit(49855, 1).
digit(38823, 1).
digit(16228, 1).
digit(29265, 1).
digit(36117, 1).
digit(33091, 1).
digit(57395, 0).
digit(29740, 1).
digit(42250, 0).
digit(1, 0).
digit(37844, 0).
digit(10834, 1).
digit(29315, 1).
digit(58307, 1).
digit(31718, 1).
digit(29651, 1).
digit(6458, 1).
digit(41770, 0).
digit(6117, 0).
digit(7253, 1).
digit(59517, 1).
digit(25757, 0).
digit(46204, 1).
digit(28000, 1).
digit(58570, 1).
digit(8679, 1).
digit(519, 0).
digit(11957, 0).
digit(16930, 1).
digit(50185, 0).
digit(32852, 1).
digit(1102, 0).
digit(18647, 1).
digit(57186, 0).
digit(49368, 1).
digit(40826, 1).
digit(17755, 0).
digit(5228, 0).
digit(35463, 0).
digit(22429, 0).
digit(49583, 0).
digit(44473, 1).
digit(47350, 1).
digit(2624, 0).
digit(6617, 1).
digit(54310, 1).
digit(38808, 0).
digit(27509, 1).
digit(57695, 0).
digit(47440, 1).
digit(38696, 0).
digit(10242, 0).
digit(25774, 1).
digit(35069, 1).
digit(28609, 1).
digit(25690, 0).
digit(34911, 1).
digit(52984, 1).
digit(27746, 1).
digit(3673, 0).
digit(58035, 0).
digit(7338, 0).
digit(51949, 1).
digit(11679, 1).
digit(51169, 1).
digit(8615, 1).
digit(20143, 1).
digit(37220, 0).
digit(10581, 1).
digit(21554, 1).
digit(36558, 1).
digit(13217, 0).
digit(40208, 0).
digit(54664, 1).
digit(9842, 1).
digit(27326, 0).
digit(1076, 0).
digit(49194, 1).
digit(43665, 0).
digit(5541, 0).
digit(16423, 0).
digit(5834, 1).
digit(40052, 1).
digit(57882, 0).
digit(53870, 1).
digit(50647, 1).
digit(315, 1).
digit(50130, 1).
digit(36037, 1).
digit(2794, 1).
digit(1596, 0).
digit(33943, 1).
digit(12106, 1).
digit(53468, 1).
digit(41412, 0).
digit(30430, 1).
digit(40803, 0).
digit(40897, 1).
digit(8162, 1).
digit(43258, 0).
digit(27092, 0).
digit(39992, 0).
digit(54886, 1).
digit(37027, 0).
digit(6859, 1).
digit(34723, 1).
digit(58897, 0).
digit(475, 1).
digit(42194, 1).
digit(57579, 1).
digit(55246, 0).
digit(37099, 0).
digit(27466, 0).
digit(25531, 1).
digit(44677, 1).
digit(59750, 1).
digit(45820, 1).
digit(7707, 0).
digit(25151, 1).
digit(19482, 1).
digit(16720, 1).
digit(16050, 1).
digit(12221, 1).
digit(58675, 1).
digit(55644, 1).
digit(46085, 1).
digit(51917, 0).
digit(51585, 0).
digit(44996, 1).
digit(55747, 1).
digit(39494, 0).
digit(951, 1).
digit(44136, 1).
digit(2448, 1).
digit(8932, 0).
digit(31386, 1).
digit(28160, 0).
digit(4642, 0).
digit(40725, 1).
digit(27931, 0).
digit(29968, 1).
digit(46751, 0).
digit(36711, 1).
digit(35657, 1).
digit(13961, 1).
digit(16218, 1).
digit(10139, 1).
digit(27638, 0).
digit(24631, 1).
digit(38446, 0).
digit(5760, 1).
digit(41248, 1).
digit(41559, 1).
digit(58015, 0).
digit(28586, 1).
digit(48510, 1).
digit(15864, 0).
digit(11579, 1).
digit(27290, 0).
digit(13196, 0).
digit(10897, 1).
digit(14900, 1).
digit(35430, 1).
digit(40007, 1).
digit(32557, 0).
digit(22270, 0).
digit(48303, 1).
digit(30299, 1).
digit(5237, 1).
digit(50465, 1).
digit(6972, 0).
digit(19240, 0).
digit(55911, 0).
digit(38751, 1).
digit(22952, 1).
digit(17539, 0).
digit(45817, 0).
digit(7495, 1).
digit(25243, 0).
digit(2730, 1).
digit(6179, 0).
digit(18712, 0).
digit(41907, 0).
digit(58008, 1).
digit(13272, 1).
digit(33345, 0).
digit(1502, 0).
digit(56351, 0).
digit(781, 0).
digit(33522, 0).
digit(8469, 0).
digit(44314, 0).
digit(46452, 1).
digit(224, 1).
digit(46178, 1).
digit(8877, 0).
digit(33087, 0).
digit(35008, 1).
digit(56123, 1).
digit(15825, 0).
digit(15632, 1).
digit(58866, 0).
digit(36501, 1).
digit(28771, 0).
digit(17360, 0).
digit(40024, 1).
digit(9269, 0).
digit(28466, 0).
digit(38590, 0).
digit(27501, 1).
digit(23970, 1).
digit(5115, 0).
digit(28612, 0).
digit(27206, 0).
digit(21942, 1).
digit(28973, 0).
digit(56349, 1).
digit(17538, 1).
digit(33465, 0).
digit(21534, 0).
digit(38276, 1).
digit(26310, 1).
digit(38310, 0).
digit(33378, 0).
digit(20066, 1).
digit(927, 0).
digit(38697, 1).
digit(29666, 1).
digit(30614, 1).
digit(47626, 0).
digit(45641, 1).
digit(6262, 1).
digit(28834, 1).
digit(8203, 1).
digit(52927, 1).
digit(42649, 0).
digit(53174, 0).
digit(7978, 0).
digit(27658, 0).
digit(12657, 1).
digit(25714, 0).
digit(50156, 0).
digit(33864, 0).
digit(22164, 1).
digit(27631, 0).
digit(46041, 0).
digit(51336, 0).
digit(53893, 0).
digit(11231, 0).
digit(59354, 1).
digit(17727, 1).
digit(50685, 1).
digit(57433, 1).
digit(57720, 0).
digit(53871, 1).
digit(8912, 0).
digit(36215, 1).
digit(55299, 1).
digit(15249, 0).
digit(45795, 1).
digit(14606, 1).
digit(17236, 0).
digit(24465, 1).
digit(53103, 1).
digit(49800, 0).
digit(57220, 1).
digit(11846, 0).
digit(7007, 1).
digit(19289, 1).
digit(15156, 0).
digit(38328, 1).
digit(34749, 1).
digit(44800, 1).
digit(26133, 1).
digit(1445, 1).
digit(56537, 1).
digit(48679, 1).
digit(42999, 1).
digit(21521, 0).
digit(43369, 1).
digit(41807, 0).
digit(52167, 1).
digit(7465, 0).
digit(23245, 1).
digit(6021, 1).
digit(55785, 0).
digit(113, 1).
digit(53662, 1).
digit(47072, 0).
digit(23579, 0).
digit(38749, 0).
digit(9204, 0).
digit(27855, 1).
digit(34162, 1).
digit(54019, 1).
digit(15048, 1).
digit(7391, 0).
digit(51975, 1).
digit(40823, 1).
digit(9039, 1).
digit(22247, 1).
digit(2047, 1).
digit(19787, 0).
digit(25312, 0).
digit(40551, 1).
digit(10874, 0).
digit(50313, 1).
digit(8098, 0).
digit(26836, 0).
digit(8273, 0).
digit(46579, 0).
digit(28223, 1).
digit(54999, 0).
digit(43879, 1).
digit(56803, 0).
digit(13106, 1).
digit(47129, 1).
digit(58306, 0).
digit(56241, 1).
digit(26376, 1).
digit(19049, 0).
digit(36362, 0).
digit(6864, 1).
digit(57642, 0).
digit(56283, 1).
digit(10223, 1).
digit(34826, 1).
digit(31639, 0).
digit(46347, 1).
digit(53746, 1).
digit(7934, 0).
digit(52850, 1).
digit(13296, 1).
digit(52245, 0).
digit(56117, 1).
digit(57041, 0).
digit(29448, 1).
digit(2811, 1).
digit(1521, 1).
digit(3516, 0).
digit(9142, 0).
digit(25970, 1).
digit(51779, 1).
digit(44023, 0).
digit(7816, 0).
digit(23315, 0).
digit(9914, 0).
digit(50542, 0).
digit(57064, 0).
digit(8032, 1).
digit(30293, 0).
digit(8354, 0).
digit(17605, 1).
digit(6002, 1).
digit(11754, 0).
digit(32007, 0).
digit(19117, 0).
digit(12556, 1).
digit(5552, 1).
digit(8323, 0).
digit(5174, 1).
digit(57798, 0).
digit(45193, 0).
digit(46627, 0).
digit(33512, 0).
digit(16455, 1).
digit(51546, 0).
digit(21575, 1).
digit(33596, 1).
digit(1060, 1).
digit(26144, 1).
digit(53330, 1).
digit(30370, 0).
digit(53611, 1).
digit(53664, 1).
digit(58865, 1).
digit(46114, 0).
digit(36269, 0).
digit(57014, 1).
digit(29937, 0).
digit(49167, 1).
digit(50911, 1).
digit(40101, 1).
digit(14725, 1).
digit(40352, 1).
digit(18072, 1).
digit(52887, 1).
digit(31753, 0).
digit(36765, 1).
digit(53896, 1).
digit(12261, 1).
digit(48371, 0).
digit(43941, 0).
digit(22601, 0).
digit(50094, 1).
digit(30616, 1).
digit(32564, 0).
digit(58985, 1).
digit(105, 1).
digit(19074, 0).
digit(42288, 1).
digit(54005, 1).
digit(16907, 1).
digit(14, 1).
digit(29250, 0).
digit(44648, 1).
digit(20591, 0).
digit(44121, 0).
digit(24536, 1).
digit(4486, 1).
digit(8445, 1).
digit(55484, 0).
digit(45129, 0).
digit(38582, 1).
digit(16355, 0).
digit(58062, 0).
digit(30441, 1).
digit(46142, 1).
digit(10542, 0).
digit(22956, 1).
digit(7119, 1).
digit(7055, 0).
digit(22043, 0).
digit(44366, 1).
digit(17331, 1).
digit(36519, 0).
digit(47621, 1).
digit(48111, 0).
digit(55290, 0).
digit(45703, 1).
digit(6802, 0).
digit(18564, 0).
digit(25549, 1).
digit(49463, 0).
digit(59028, 1).
digit(24463, 0).
digit(59571, 1).
digit(12134, 0).
digit(11330, 0).
digit(2374, 1).
digit(17761, 0).
digit(49310, 0).
digit(48350, 0).
digit(24466, 1).
digit(35826, 1).
digit(33343, 1).
digit(38750, 1).
digit(15920, 1).
digit(5862, 1).
digit(23980, 0).
digit(29585, 0).
digit(32534, 1).
digit(8000, 0).
digit(16888, 0).
digit(18165, 1).
digit(13117, 0).
digit(41103, 1).
digit(33441, 1).
digit(3532, 1).
digit(4926, 0).
digit(33374, 1).
digit(31647, 1).
digit(40108, 0).
digit(45677, 1).
digit(48601, 0).
digit(36408, 1).
digit(5202, 0).
digit(18295, 0).
digit(25513, 1).
digit(21715, 1).
digit(2920, 1).
digit(20132, 1).
digit(37579, 1).
digit(15921, 1).
digit(36371, 0).
digit(19969, 0).
digit(34007, 0).
digit(47869, 1).
digit(22120, 1).
digit(45669, 0).
digit(17706, 1).
digit(2718, 0).
digit(28484, 1).
digit(9459, 0).
digit(8823, 1).
digit(18656, 1).
digit(31685, 1).
digit(27544, 0).
digit(56308, 1).
digit(35302, 0).
digit(43429, 1).
digit(32085, 1).
digit(6365, 1).
digit(26295, 1).
digit(27318, 0).
digit(8472, 0).
digit(3809, 0).
digit(35525, 0).
digit(12578, 1).
digit(31509, 0).
digit(19850, 1).
digit(36047, 0).
digit(14366, 1).
digit(26323, 0).
digit(33416, 1).
digit(18768, 1).
digit(30695, 1).
digit(55837, 0).
digit(55786, 1).
digit(12266, 0).
digit(6701, 1).
digit(27567, 0).
digit(8987, 1).
digit(55271, 1).
digit(21898, 0).
digit(41252, 0).
digit(34246, 0).
digit(1816, 1).
digit(42744, 0).
digit(18124, 0).
digit(34639, 0).
digit(10343, 0).
digit(36549, 0).
digit(44281, 1).
digit(50423, 0).
digit(25189, 1).
digit(45384, 0).
digit(39560, 1).
digit(7188, 1).
digit(18182, 1).
digit(44205, 0).
digit(4420, 1).
digit(14133, 1).
digit(46090, 0).
digit(16759, 1).
digit(52336, 0).
digit(870, 1).
digit(7404, 1).
digit(52079, 0).
digit(39788, 0).
digit(56057, 1).
digit(9834, 0).
digit(39041, 1).
digit(37761, 1).
digit(48534, 1).
digit(14870, 1).
digit(38404, 0).
digit(7855, 0).
digit(40896, 1).
digit(10801, 0).
digit(47382, 1).
digit(19209, 0).
digit(30140, 1).
digit(26315, 0).
digit(50476, 0).
digit(30087, 0).
digit(6183, 1).
digit(28300, 1).
digit(30699, 0).
digit(13259, 1).
digit(39457, 0).
digit(18832, 0).
digit(39896, 1).
digit(35485, 0).
digit(41126, 0).
digit(20987, 1).
digit(943, 0).
digit(27885, 1).
digit(59391, 1).
digit(20322, 1).
digit(47527, 1).
digit(51228, 0).
digit(43610, 0).
digit(50678, 1).
digit(52955, 0).
digit(56208, 1).
digit(7183, 0).
digit(12008, 0).
digit(32800, 0).
digit(55358, 1).
digit(44524, 1).
digit(1012, 1).
digit(58012, 0).
digit(48848, 1).
digit(23240, 1).
digit(77, 1).
digit(32530, 1).
digit(59527, 1).
digit(17847, 1).
digit(27793, 1).
digit(38709, 1).
digit(6094, 0).
digit(13950, 1).
digit(31421, 1).
digit(29375, 1).
digit(54164, 1).
digit(42579, 0).
digit(37200, 1).
digit(28367, 1).
digit(22972, 1).
digit(53814, 0).
digit(41989, 0).
digit(22888, 1).
digit(23665, 1).
digit(51404, 0).
digit(49243, 0).
digit(6116, 1).
digit(58810, 1).
digit(19179, 0).
digit(25129, 1).
digit(53432, 0).
digit(54459, 0).
digit(34778, 1).
digit(54784, 1).
digit(32660, 1).
digit(59932, 1).
digit(16869, 1).
digit(53270, 0).
digit(21492, 1).
digit(56318, 0).
digit(7753, 0).
digit(13044, 0).
digit(57761, 1).
digit(16031, 0).
digit(43705, 1).
digit(16236, 1).
digit(59418, 0).
digit(6023, 0).
digit(44891, 1).
digit(36433, 1).
digit(12791, 1).
digit(10842, 1).
digit(45402, 0).
digit(37653, 0).
digit(9215, 0).
digit(28763, 0).
digit(4554, 1).
digit(54269, 1).
digit(9697, 1).
digit(14282, 0).
digit(16914, 1).
digit(23557, 0).
digit(33949, 1).
digit(55556, 0).
digit(28153, 1).
digit(58229, 0).
digit(2876, 1).
digit(34938, 1).
digit(51167, 0).
digit(59952, 0).
digit(55449, 1).
digit(40045, 1).
digit(32378, 1).
digit(3197, 1).
digit(34590, 1).
digit(59200, 1).
digit(18664, 1).
digit(59452, 0).
digit(32131, 0).
digit(30146, 1).
digit(11391, 0).
digit(25934, 1).
digit(11973, 0).
digit(54421, 0).
digit(6736, 1).
digit(7572, 1).
digit(54939, 0).
digit(35782, 0).
digit(52454, 1).
digit(58203, 1).
digit(31019, 1).
digit(57644, 1).
digit(6413, 1).
digit(57283, 1).
digit(37742, 1).
digit(11694, 0).
digit(30238, 1).
digit(32439, 1).
digit(50101, 0).
digit(44454, 1).
digit(7421, 1).
digit(4120, 1).
digit(31237, 1).
digit(37513, 1).
digit(37138, 1).
digit(35197, 1).
digit(4347, 0).
digit(50586, 0).
digit(14571, 1).
digit(47552, 0).
digit(10176, 0).
digit(14995, 1).
digit(50158, 1).
digit(50042, 0).
digit(6619, 0).
digit(55048, 0).
digit(4356, 0).
digit(35050, 0).
digit(49774, 0).
digit(5275, 0).
digit(48074, 1).
digit(44551, 0).
digit(34849, 1).
digit(33392, 0).
digit(20884, 1).
digit(44046, 1).
digit(6083, 1).
digit(48583, 1).
digit(45511, 0).
digit(18703, 0).
digit(12042, 1).
digit(14419, 0).
digit(37461, 1).
digit(48306, 1).
digit(8449, 0).
digit(26761, 0).
digit(14089, 0).
digit(1321, 1).
digit(44451, 0).
digit(41954, 1).
digit(44887, 1).
digit(849, 0).
digit(4115, 0).
digit(48390, 1).
digit(23821, 0).
digit(15266, 1).
digit(21809, 0).
digit(19376, 0).
digit(40908, 1).
digit(41492, 1).
digit(51767, 1).
digit(17581, 0).
digit(2435, 0).
digit(52362, 1).
digit(52673, 0).
digit(3111, 1).
digit(17058, 1).
digit(17554, 1).
digit(27911, 0).
digit(957, 0).
digit(56902, 1).
digit(13882, 1).
digit(57221, 0).
digit(17009, 0).
digit(54982, 0).
digit(56633, 1).
digit(16159, 1).
digit(39906, 0).
digit(46470, 1).
digit(23703, 1).
digit(14649, 0).
digit(21703, 0).
digit(32859, 1).
digit(43652, 1).
digit(14959, 0).
digit(26073, 0).
digit(41623, 1).
digit(10267, 1).
digit(53604, 1).
digit(38722, 0).
digit(28230, 0).
digit(40142, 1).
digit(18759, 0).
digit(5477, 1).
digit(42026, 1).
digit(35959, 0).
digit(49214, 0).
digit(11320, 0).
digit(6665, 1).
digit(57797, 1).
digit(51303, 0).
digit(41104, 1).
digit(6076, 0).
digit(48461, 1).
digit(57037, 1).
digit(59616, 1).
digit(41823, 1).
digit(57296, 0).
digit(52832, 0).
digit(55656, 0).
digit(45417, 0).
digit(2776, 1).
digit(22932, 0).
digit(49974, 0).
digit(23378, 0).
digit(34771, 0).
digit(20751, 0).
digit(52586, 1).
digit(19613, 1).
digit(7631, 1).
digit(9292, 1).
digit(7603, 1).
digit(23059, 1).
digit(21137, 0).
digit(14724, 1).
digit(37809, 1).
digit(6663, 0).
digit(2462, 1).
digit(11057, 1).
digit(44326, 0).
digit(19935, 0).
digit(55543, 1).
digit(28840, 0).
digit(55576, 0).
digit(27789, 1).
digit(23818, 1).
digit(46679, 1).
digit(21535, 1).
digit(34172, 0).
digit(50331, 0).
digit(42559, 0).
digit(38436, 1).
digit(6698, 1).
digit(31166, 1).
digit(10645, 0).
digit(27871, 1).
digit(31116, 0).
digit(22671, 1).
digit(12477, 0).
digit(28961, 1).
digit(19510, 1).
digit(14971, 0).
digit(24102, 1).
digit(58789, 1).
digit(12539, 1).
digit(42599, 0).
digit(30909, 1).
digit(24858, 0).
digit(30043, 1).
digit(1195, 0).
digit(41031, 1).
digit(33831, 0).
digit(56377, 0).
digit(52888, 1).
digit(16262, 1).
digit(14268, 1).
digit(37303, 0).
digit(40091, 0).
digit(33279, 1).
digit(47442, 1).
digit(39854, 0).
digit(27284, 1).
digit(42332, 1).
digit(15929, 1).
digit(6615, 0).
digit(17051, 0).
digit(52039, 0).
digit(7176, 1).
digit(34010, 1).
digit(55901, 1).
digit(24126, 1).
digit(35282, 0).
digit(15064, 0).
digit(2918, 1).
digit(8451, 1).
digit(24615, 0).
digit(59313, 0).
digit(4924, 1).
digit(43633, 1).
digit(8836, 1).
digit(26401, 1).
digit(19894, 1).
digit(19612, 0).
digit(58963, 1).
digit(39075, 0).
digit(16971, 1).
digit(47247, 0).
digit(3035, 0).
digit(32446, 1).
digit(39498, 0).
digit(34358, 0).
digit(14734, 1).
digit(51372, 1).
digit(15427, 1).
digit(50123, 0).
digit(13090, 1).
digit(26096, 1).
digit(28827, 1).
digit(29243, 0).
digit(56572, 1).
digit(36138, 0).
digit(20961, 1).
digit(53930, 1).
digit(16705, 0).
digit(40286, 0).
digit(33166, 1).
digit(121, 0).
digit(26507, 0).
digit(12158, 1).
digit(29658, 1).
digit(15133, 0).
digit(49304, 0).
digit(32554, 1).
digit(16153, 1).
digit(32295, 1).
digit(28030, 0).
digit(5144, 0).
digit(59148, 0).
digit(29323, 1).
digit(21299, 0).
digit(24075, 0).
digit(7877, 1).
digit(56707, 0).
digit(33841, 1).
digit(29039, 0).
digit(43020, 0).
digit(59902, 1).
digit(9354, 1).
digit(59230, 1).
digit(51447, 1).
digit(50909, 0).
digit(43050, 0).
digit(21818, 1).
digit(34166, 1).
digit(47850, 1).
digit(42690, 1).
digit(42865, 0).
digit(22087, 1).
digit(3067, 0).
digit(859, 0).
digit(33078, 0).
digit(38317, 1).
digit(52046, 1).
digit(39052, 1).
digit(4773, 0).
digit(57359, 0).
digit(48200, 0).
digit(42270, 0).
digit(17187, 0).
digit(42608, 0).
digit(35737, 1).
digit(41669, 1).
digit(553, 1).
digit(21094, 1).
digit(42630, 1).
digit(43811, 0).
digit(22510, 0).
digit(21618, 1).
digit(56834, 1).
digit(10537, 1).
digit(6443, 1).
digit(11152, 1).
digit(28875, 0).
digit(54272, 0).
digit(55242, 0).
digit(5298, 0).
digit(9877, 0).
digit(2873, 0).
digit(23751, 1).
digit(42859, 1).
digit(53058, 1).
digit(27684, 1).
digit(53535, 0).
digit(53994, 0).
digit(49559, 1).
digit(58260, 1).
digit(53615, 0).
digit(12927, 0).
digit(38415, 1).
digit(47837, 1).
digit(13973, 1).
digit(42658, 0).
digit(31945, 1).
digit(21191, 0).
digit(50057, 0).
digit(56016, 1).
digit(56081, 1).
digit(56444, 0).
digit(56639, 0).
digit(47644, 1).
digit(10684, 1).
digit(36646, 0).
digit(29544, 1).
digit(4270, 0).
digit(11615, 0).
digit(29615, 1).
digit(1277, 1).
digit(23728, 0).
digit(35404, 1).
digit(6400, 0).
digit(45131, 1).
digit(35669, 1).
digit(33428, 1).
digit(23176, 1).
digit(51629, 0).
digit(39776, 1).
digit(1709, 0).
digit(41506, 0).
digit(12407, 1).
digit(1798, 0).
digit(19897, 1).
digit(45290, 1).
digit(56423, 1).
digit(44359, 1).
digit(41038, 0).
digit(5285, 0).
digit(1331, 1).
digit(17828, 0).
digit(40468, 1).
digit(41368, 0).
digit(35006, 1).
digit(52648, 1).
digit(21612, 0).
digit(49943, 1).
digit(7687, 0).
digit(32187, 1).
digit(25278, 0).
digit(11071, 0).
digit(20658, 0).
digit(28971, 0).
digit(36484, 0).
digit(10914, 0).
digit(31348, 1).
digit(32808, 1).
digit(30427, 1).
digit(44363, 0).
digit(17650, 1).
digit(4852, 0).
digit(8093, 0).
digit(42893, 1).
digit(56010, 1).
digit(53833, 1).
digit(47042, 0).
digit(1530, 0).
digit(56129, 0).
digit(37482, 1).
digit(58170, 0).
digit(10259, 0).
digit(53436, 1).
digit(10838, 1).
digit(4960, 1).
digit(15669, 1).
digit(49534, 1).
digit(44966, 1).
digit(18540, 1).
digit(19008, 0).
digit(46183, 0).
digit(50572, 1).
digit(42974, 1).
digit(2654, 1).
digit(21078, 0).
digit(39631, 0).
digit(13129, 1).
digit(4865, 1).
digit(33921, 0).
digit(40015, 1).
digit(27411, 0).
digit(14986, 1).
digit(59266, 1).
digit(21985, 0).
digit(58658, 0).
digit(48897, 1).
digit(20012, 0).
digit(57412, 1).
digit(676, 1).
digit(6052, 1).
digit(10334, 0).
digit(46653, 0).
digit(48929, 0).
digit(23614, 1).
digit(18852, 1).
digit(8016, 0).
digit(21279, 0).
digit(57020, 0).
digit(57287, 1).
digit(23612, 1).
digit(47429, 1).
digit(44325, 0).
digit(48395, 1).
digit(56914, 0).
digit(9734, 1).
digit(30665, 0).
digit(10530, 0).
digit(13215, 0).
digit(57408, 0).
digit(552, 1).
digit(6757, 0).
digit(6103, 0).
digit(16674, 1).
digit(43500, 1).
digit(21256, 0).
digit(14402, 0).
digit(33012, 1).
digit(34852, 1).
digit(44399, 0).
digit(48438, 0).
digit(9942, 1).
digit(57977, 0).
digit(11360, 1).
digit(25864, 1).
digit(4225, 1).
digit(35552, 1).
digit(51499, 0).
digit(24863, 1).
digit(32133, 1).
digit(35912, 1).
digit(6607, 0).
digit(49817, 1).
digit(3800, 1).
digit(9455, 0).
digit(37189, 0).
digit(52691, 0).
digit(689, 0).
digit(38769, 1).
digit(13788, 1).
digit(7648, 0).
digit(37741, 1).
digit(47905, 1).
digit(29986, 1).
digit(16495, 1).
digit(53012, 1).
digit(46372, 1).
digit(15344, 0).
digit(55596, 1).
digit(42625, 1).
digit(18444, 1).
digit(39684, 1).
digit(9902, 1).
digit(6181, 1).
digit(52628, 0).
digit(42614, 0).
digit(22954, 0).
digit(28545, 1).
digit(47073, 0).
digit(22106, 1).
digit(39160, 1).
digit(58689, 1).
digit(52020, 1).
digit(25707, 0).
digit(18886, 1).
digit(58147, 1).
digit(20681, 0).
digit(42242, 1).
digit(13581, 1).
digit(28165, 0).
digit(37510, 0).
digit(4340, 0).
digit(55324, 1).
digit(11677, 1).
digit(53041, 1).
digit(19765, 1).
digit(49027, 0).
digit(39898, 0).
digit(45748, 0).
digit(47740, 0).
digit(53256, 0).
digit(24807, 0).
digit(11222, 0).
digit(6968, 0).
digit(54445, 0).
digit(9934, 0).
digit(54232, 0).
digit(36625, 1).
digit(43426, 1).
digit(43593, 0).
digit(81, 0).
digit(23118, 1).
digit(2663, 1).
digit(23991, 0).
digit(36811, 1).
digit(38413, 1).
digit(5271, 1).
digit(36373, 1).
digit(58083, 1).
digit(48851, 1).
digit(24879, 1).
digit(49557, 1).
digit(24090, 1).
digit(39861, 0).
digit(12178, 1).
digit(32296, 1).
digit(19145, 1).
digit(11200, 1).
digit(7185, 1).
digit(46785, 0).
digit(34418, 1).
digit(51623, 0).
digit(19798, 0).
digit(29052, 0).
digit(48554, 0).
digit(5425, 1).
digit(36947, 1).
digit(31392, 0).
digit(33172, 0).
digit(34950, 1).
digit(52657, 1).
digit(42867, 0).
digit(48196, 0).
digit(48908, 1).
digit(51882, 1).
digit(19818, 0).
digit(17459, 1).
digit(10732, 0).
digit(56965, 1).
digit(15684, 1).
digit(35435, 1).
digit(4653, 0).
digit(29471, 0).
digit(11982, 1).
digit(53185, 0).
digit(34168, 1).
digit(345, 1).
digit(51493, 1).
digit(39354, 1).
digit(4137, 1).
digit(35320, 0).
digit(58610, 1).
digit(6394, 0).
digit(39057, 1).
digit(15910, 0).
digit(33703, 0).
digit(52323, 0).
digit(22490, 1).
digit(19646, 0).
digit(43081, 0).
digit(59541, 0).
digit(10625, 0).
digit(7458, 0).
digit(36997, 1).
digit(41715, 1).
digit(49612, 1).
digit(56429, 0).
digit(21398, 1).
digit(9643, 1).
digit(25057, 1).
digit(1000, 0).
digit(20146, 1).
digit(48121, 0).
digit(14269, 0).
digit(11728, 1).
digit(40987, 1).
digit(23037, 1).
digit(14860, 1).
digit(51828, 0).
digit(303, 0).
digit(59023, 0).
digit(32953, 0).
digit(24143, 1).
digit(54692, 1).
digit(17759, 1).
digit(37571, 0).
digit(4775, 1).
digit(9083, 0).
digit(34926, 1).
digit(24086, 1).
digit(51809, 0).
digit(12071, 1).
digit(17977, 0).
digit(15042, 0).
digit(57684, 1).
digit(15275, 0).
digit(9264, 1).
digit(32449, 1).
digit(45556, 1).
digit(48664, 1).
digit(28414, 0).
digit(5321, 0).
digit(40969, 0).
digit(20829, 1).
digit(184, 1).
digit(38538, 1).
digit(41634, 0).
digit(52813, 0).
digit(2012, 1).
digit(2396, 0).
digit(6593, 0).
digit(27089, 0).
digit(21740, 1).
digit(7039, 1).
digit(59497, 0).
digit(21954, 0).
digit(56471, 1).
digit(19459, 0).
digit(51149, 1).
digit(22777, 0).
digit(23679, 0).
digit(48900, 1).
digit(13630, 1).
digit(34655, 1).
digit(19180, 0).
digit(43497, 0).
digit(5789, 1).
digit(23841, 1).
digit(21908, 1).
digit(447, 1).
digit(26923, 0).
digit(23044, 0).
digit(19133, 0).
digit(49113, 1).
digit(40218, 0).
digit(22095, 1).
digit(40336, 0).
digit(8180, 1).
digit(39531, 0).
digit(47097, 1).
digit(58017, 1).
digit(55767, 1).
digit(38951, 1).
digit(1452, 1).
digit(52428, 0).
digit(44826, 0).
digit(45029, 1).
digit(48916, 1).
digit(57997, 1).
digit(24208, 1).
digit(11335, 0).
digit(4749, 0).
digit(33937, 1).
digit(11229, 1).
digit(42462, 1).
digit(10885, 1).
digit(21958, 1).
digit(28090, 1).
digit(57201, 0).
digit(51290, 0).
digit(23290, 1).
digit(30787, 0).
digit(45540, 0).
digit(36414, 0).
digit(14892, 1).
digit(46303, 0).
digit(23369, 1).
digit(52498, 1).
digit(28835, 0).
digit(54728, 0).
digit(29498, 0).
digit(55501, 0).
digit(21990, 1).
digit(4023, 0).
digit(4748, 0).
digit(17411, 0).
digit(53685, 0).
digit(19724, 1).
digit(48411, 1).
digit(31895, 0).
digit(6302, 0).
digit(20627, 1).
digit(39483, 0).
digit(40796, 1).
digit(16415, 0).
digit(40971, 1).
digit(12809, 1).
digit(10399, 1).
digit(7303, 1).
digit(39819, 1).
digit(7597, 0).
digit(52289, 0).
digit(49731, 1).
digit(16508, 1).
digit(54708, 1).
digit(24355, 1).
digit(47976, 0).
digit(37725, 1).
digit(37102, 1).
digit(47784, 1).
digit(47816, 0).
digit(46342, 1).
digit(54434, 1).
digit(40303, 0).
digit(36919, 0).
digit(55986, 0).
digit(13399, 1).
digit(39554, 1).
digit(23775, 1).
digit(44317, 1).
digit(25397, 1).
digit(8763, 0).
digit(9903, 1).
digit(16348, 1).
digit(20772, 1).
digit(31084, 1).
digit(22168, 1).
digit(14877, 0).
digit(6586, 1).
digit(44692, 0).
digit(44038, 1).
digit(3898, 1).
digit(13974, 1).
digit(53273, 1).
digit(25981, 1).
digit(15868, 0).
digit(52133, 0).
digit(1517, 0).
digit(12690, 0).
digit(21765, 1).
digit(44725, 1).
digit(18906, 1).
digit(56737, 1).
digit(20801, 0).
digit(53849, 1).
digit(53563, 1).
digit(58718, 0).
digit(37012, 1).
digit(47957, 0).
digit(47118, 1).
digit(32130, 0).
digit(14481, 0).
digit(49739, 0).
digit(53804, 1).
digit(5194, 0).
digit(48183, 1).
digit(1734, 1).
digit(56490, 1).
digit(38398, 1).
digit(15692, 0).
digit(52431, 1).
digit(28203, 0).
digit(35168, 1).
digit(49601, 0).
digit(52119, 1).
digit(13126, 0).
digit(19682, 0).
digit(18509, 0).
digit(49419, 0).
digit(35527, 1).
digit(32869, 0).
digit(37647, 0).
digit(26587, 1).
digit(28457, 0).
digit(13798, 0).
digit(17122, 0).
digit(18462, 0).
digit(11439, 1).
digit(2858, 1).
digit(13459, 0).
digit(39850, 0).
digit(53341, 1).
digit(34373, 1).
digit(28510, 0).
digit(37816, 1).
digit(33861, 1).
digit(31498, 1).
digit(57830, 1).
digit(22378, 0).
digit(50652, 1).
digit(13029, 0).
digit(1868, 0).
digit(17106, 0).
digit(35299, 1).
digit(53562, 0).
digit(49706, 1).
digit(56328, 1).
digit(52934, 0).
digit(16803, 0).
digit(12314, 1).
digit(43570, 1).
digit(42771, 0).
digit(35680, 0).
digit(58376, 0).
digit(55635, 1).
digit(17022, 1).
digit(36241, 0).
digit(18337, 0).
digit(54235, 0).
digit(24626, 1).
digit(13583, 0).
digit(53533, 0).
digit(177, 1).
digit(22707, 0).
digit(25892, 0).
digit(34359, 1).
digit(28855, 0).
digit(51774, 0).
digit(41418, 1).
digit(8979, 0).
digit(40412, 0).
digit(11701, 0).
digit(29497, 0).
digit(41386, 1).
digit(47419, 0).
digit(9966, 0).
digit(7222, 1).
digit(28622, 1).
digit(56689, 1).
digit(46763, 0).
digit(33257, 0).
digit(36892, 1).
digit(33230, 0).
digit(43790, 1).
digit(39115, 0).
digit(37661, 1).
digit(35712, 0).
digit(32204, 1).
digit(17240, 1).
digit(33042, 0).
digit(40377, 1).
digit(57298, 0).
digit(28072, 0).
digit(3889, 1).
digit(2260, 1).
digit(36304, 0).
digit(37529, 0).
digit(10468, 1).
digit(30044, 0).
digit(58454, 0).
digit(51466, 0).
digit(49964, 1).
digit(59519, 1).
digit(53049, 1).
digit(52716, 0).
digit(22045, 1).
digit(14179, 0).
digit(30381, 1).
digit(46289, 0).
digit(11272, 1).
digit(251, 1).
digit(49420, 1).
digit(16012, 1).
digit(19769, 0).
digit(39805, 1).
digit(49075, 0).
digit(13390, 1).
digit(57865, 1).
digit(53593, 1).
digit(47997, 0).
digit(29114, 1).
digit(15786, 1).
digit(15015, 0).
digit(33310, 1).
digit(8660, 1).
digit(31365, 1).
digit(22628, 1).
digit(58995, 1).
digit(14278, 0).
digit(7622, 1).
digit(57948, 1).
digit(42404, 1).
digit(56347, 0).
digit(1681, 1).
digit(28828, 1).
digit(31081, 1).
digit(34179, 1).
digit(56217, 0).
digit(29755, 0).
digit(55127, 1).
digit(10602, 1).
digit(2166, 1).
digit(25473, 1).
digit(15885, 1).
digit(31511, 1).
digit(15536, 1).
digit(53670, 0).
digit(54716, 1).
digit(53957, 0).
digit(31502, 0).
digit(57693, 0).
digit(37972, 1).
digit(18270, 1).
digit(31447, 1).
digit(1346, 0).
digit(11543, 1).
digit(21424, 1).
digit(41030, 0).
digit(20760, 1).
digit(35697, 0).
digit(29173, 1).
digit(6819, 0).
digit(46516, 0).
digit(44994, 1).
digit(59730, 0).
digit(36620, 0).
digit(44951, 0).
digit(56291, 1).
digit(43948, 1).
digit(39555, 1).
digit(54941, 0).
digit(24376, 1).
digit(4389, 0).
digit(54438, 1).
digit(42111, 0).
digit(52244, 1).
digit(33535, 1).
digit(11060, 0).
digit(5691, 1).
digit(51784, 1).
digit(16204, 1).
digit(37511, 0).
digit(11422, 0).
digit(2058, 0).
digit(28738, 1).
digit(41971, 1).
digit(18093, 0).
digit(1069, 1).
digit(11835, 1).
digit(52698, 0).
digit(46905, 0).
digit(58355, 1).
digit(6165, 1).
digit(32863, 1).
digit(39944, 0).
digit(55175, 1).
digit(59477, 0).
digit(30749, 1).
digit(59768, 1).
digit(18019, 1).
digit(4216, 0).
digit(45823, 0).
digit(31288, 0).
digit(38850, 0).
digit(9166, 0).
digit(7052, 0).
digit(13790, 1).
digit(7993, 1).
digit(7630, 1).
digit(14613, 0).
digit(30261, 1).
digit(56576, 1).
digit(39846, 1).
digit(36608, 1).
digit(36273, 1).
digit(1200, 1).
digit(23308, 1).
digit(12666, 0).
digit(39707, 1).
digit(34329, 1).
digit(16536, 1).
digit(59653, 0).
digit(2151, 1).
digit(19369, 0).
digit(19261, 0).
digit(30939, 0).
digit(36469, 1).
digit(24892, 0).
digit(40778, 0).
digit(8430, 1).
digit(56533, 0).
digit(59694, 1).
digit(23120, 0).
digit(45434, 0).
digit(48267, 0).
digit(18247, 0).
digit(16259, 1).
digit(43595, 1).
digit(15407, 0).
digit(1828, 0).
digit(32096, 1).
digit(42881, 1).
digit(6468, 0).
digit(29939, 1).
digit(44199, 0).
digit(40312, 0).
digit(57738, 1).
digit(27396, 1).
digit(15777, 0).
digit(5342, 0).
digit(39533, 1).
digit(26038, 0).
digit(50952, 1).
digit(21201, 1).
digit(34096, 1).
digit(46158, 1).
digit(42189, 1).
digit(53360, 0).
digit(56984, 0).
digit(5249, 0).
digit(58665, 0).
digit(50667, 1).
digit(53665, 0).
digit(16439, 0).
digit(28357, 0).
digit(17586, 1).
digit(52221, 0).
digit(22632, 1).
digit(42349, 0).
digit(12166, 1).
digit(47977, 1).
digit(15934, 0).
digit(18359, 1).
digit(51758, 0).
digit(17101, 1).
digit(53982, 0).
digit(16634, 0).
digit(58820, 1).
digit(55787, 1).
digit(4082, 0).
digit(58567, 1).
digit(42510, 1).
digit(27665, 1).
digit(712, 0).
digit(8158, 0).
digit(1085, 1).
digit(33175, 1).
digit(41518, 1).
digit(50914, 1).
digit(24357, 1).
digit(10916, 0).
digit(24900, 1).
digit(45861, 1).
digit(55397, 1).
digit(16241, 1).
digit(26420, 1).
digit(13889, 0).
digit(52558, 0).
digit(47910, 1).
digit(35012, 0).
digit(23559, 1).
digit(42227, 0).
digit(12020, 0).
digit(40744, 1).
digit(25227, 1).
digit(3698, 0).
digit(41857, 0).
digit(37950, 0).
digit(4750, 1).
digit(15590, 0).
digit(27924, 1).
digit(41640, 1).
digit(26712, 0).
digit(31195, 0).
digit(38686, 0).
digit(37289, 1).
digit(21088, 1).
digit(19166, 0).
digit(44109, 1).
digit(36741, 0).
digit(10741, 1).
digit(17802, 1).
digit(1197, 1).
digit(556, 1).
digit(36995, 1).
digit(18222, 0).
digit(9404, 1).
digit(22306, 0).
digit(2192, 0).
digit(20369, 0).
digit(56709, 0).
digit(30255, 0).
digit(56835, 0).
digit(52882, 1).
digit(37883, 0).
digit(24636, 0).
digit(20824, 1).
digit(14123, 1).
digit(34091, 1).
digit(27242, 1).
digit(2257, 0).
digit(40132, 1).
digit(40098, 1).
digit(54141, 0).
digit(15316, 1).
digit(24978, 0).
digit(37110, 1).
digit(47327, 1).
digit(11178, 1).
digit(7532, 1).
digit(11661, 0).
digit(38179, 0).
digit(56625, 0).
digit(8855, 0).
digit(59781, 1).
digit(38155, 0).
digit(10773, 0).
digit(26343, 0).
digit(35001, 1).
digit(31393, 0).
digit(35945, 1).
digit(38595, 1).
digit(11794, 0).
digit(23914, 0).
digit(51685, 0).
digit(18442, 1).
digit(26317, 1).
digit(20645, 0).
digit(33238, 0).
digit(34874, 1).
digit(24625, 1).
digit(58180, 0).
digit(50385, 0).
digit(26070, 1).
digit(31526, 1).
digit(16519, 0).
digit(29378, 0).
digit(40801, 0).
digit(29377, 1).
digit(33163, 0).
digit(3693, 0).
digit(57849, 1).
digit(55479, 0).
digit(825, 0).
digit(55083, 1).
digit(41607, 1).
digit(38465, 1).
digit(24343, 0).
digit(7364, 0).
digit(17223, 0).
digit(52278, 1).
digit(42015, 1).
digit(30037, 0).
digit(50582, 1).
digit(26772, 1).
digit(58861, 0).
digit(36797, 0).
digit(28385, 0).
digit(6951, 1).
digit(36380, 1).
digit(19082, 0).
digit(43128, 1).
digit(14161, 0).
digit(35257, 1).
digit(28167, 1).
digit(2745, 0).
digit(6603, 0).
digit(36801, 1).
digit(14098, 1).
digit(1532, 0).
digit(7181, 0).
digit(37179, 1).
digit(17711, 1).
digit(54607, 1).
digit(12568, 0).
digit(1454, 0).
digit(44327, 1).
digit(33558, 1).
digit(34206, 1).
digit(6648, 0).
digit(52562, 1).
digit(51127, 0).
digit(50820, 0).
digit(28412, 0).
digit(41588, 0).
digit(19305, 1).
digit(35995, 1).
digit(4145, 0).
digit(9056, 0).
digit(48765, 1).
digit(39236, 0).
digit(44491, 0).
digit(17726, 0).
digit(16932, 1).
digit(59074, 1).
digit(10861, 0).
digit(57349, 0).
digit(43616, 1).
digit(32358, 1).
digit(21979, 1).
digit(56415, 0).
digit(1665, 1).
digit(22642, 1).
digit(53467, 1).
digit(18590, 0).
digit(56140, 0).
digit(50637, 0).
digit(45433, 1).
digit(58916, 0).
digit(27439, 1).
digit(52627, 1).
digit(16376, 1).
digit(33586, 1).
digit(12141, 1).
digit(17750, 1).
digit(51606, 1).
digit(58402, 0).
digit(41146, 1).
digit(6747, 1).
digit(51041, 0).
digit(14110, 0).
digit(50056, 1).
digit(50200, 0).
digit(41090, 1).
digit(28451, 0).
digit(51468, 1).
digit(40199, 0).
digit(45197, 1).
digit(52811, 1).
digit(23993, 1).
digit(37411, 0).
digit(22373, 0).
digit(38440, 0).
digit(42402, 0).
digit(47709, 1).
digit(30141, 0).
digit(48844, 1).
digit(42849, 1).
digit(59267, 0).
digit(33122, 1).
digit(21945, 0).
digit(57034, 0).
digit(59150, 1).
digit(6823, 0).
digit(15148, 1).
digit(30439, 0).
digit(32899, 0).
digit(19542, 0).
digit(32406, 1).
digit(22978, 0).
digit(12632, 1).
digit(26980, 0).
digit(55345, 0).
digit(33183, 1).
digit(58576, 1).
digit(35281, 1).
digit(50016, 1).
digit(31535, 0).
digit(24913, 0).
digit(57924, 0).
digit(9540, 1).
digit(59688, 1).
digit(31093, 1).
digit(51190, 1).
digit(13916, 0).
digit(7203, 0).
digit(43490, 1).
digit(26262, 0).
digit(20319, 1).
digit(12776, 0).
digit(41353, 0).
digit(4300, 1).
digit(33383, 1).
digit(10301, 1).
digit(3758, 1).
digit(27790, 1).
digit(25847, 1).
digit(6226, 1).
digit(10083, 0).
digit(18991, 0).
digit(6913, 1).
digit(22436, 0).
digit(47396, 0).
digit(41617, 0).
digit(30244, 1).
digit(55621, 1).
digit(47473, 0).
digit(29848, 0).
digit(23273, 0).
digit(47324, 0).
digit(32952, 0).
digit(13413, 1).
digit(8166, 1).
digit(13875, 1).
digit(32462, 1).
digit(47690, 0).
digit(22214, 1).
digit(25937, 1).
digit(5351, 1).
digit(13379, 1).
digit(41585, 1).
digit(46934, 1).
digit(19139, 1).
digit(59114, 1).
digit(31640, 1).
digit(1115, 1).
digit(38976, 1).
digit(24445, 0).
digit(58510, 0).
digit(11172, 0).
digit(45857, 0).
digit(16618, 0).
digit(13314, 0).
digit(249, 0).
digit(47405, 0).
digit(31857, 0).
digit(19760, 0).
digit(15044, 1).
digit(45051, 1).
digit(28245, 1).
digit(10938, 0).
digit(6406, 0).
digit(59929, 0).
digit(26139, 0).
digit(3443, 0).
digit(4162, 1).
digit(43265, 0).
digit(52058, 1).
digit(9439, 0).
digit(33092, 1).
digit(5612, 0).
digit(9705, 1).
digit(10169, 1).
digit(24993, 0).
digit(31661, 0).
digit(20886, 0).
digit(19171, 0).
digit(50434, 0).
digit(5012, 1).
digit(16545, 1).
digit(3380, 1).
digit(58546, 0).
digit(35399, 1).
digit(14311, 0).
digit(59585, 0).
digit(8247, 0).
digit(44675, 0).
digit(59680, 0).
digit(29041, 1).
digit(43887, 1).
digit(46944, 0).
digit(17378, 1).
digit(21196, 1).
digit(2932, 1).
digit(29609, 1).
digit(25928, 0).
digit(42324, 0).
digit(41445, 0).
digit(10541, 1).
digit(13873, 0).
digit(31529, 1).
digit(52073, 1).
digit(40399, 1).
digit(48229, 1).
digit(22848, 1).
digit(30320, 0).
digit(10006, 1).
digit(3012, 0).
digit(21576, 0).
digit(211, 1).
digit(37081, 1).
digit(15989, 1).
digit(14924, 1).
digit(30337, 1).
digit(10266, 1).
digit(34421, 0).
digit(42644, 1).
digit(24135, 1).
digit(4807, 1).
digit(32440, 1).
digit(47786, 0).
digit(53594, 1).
digit(27153, 0).
digit(52416, 1).
digit(51397, 1).
digit(41383, 0).
digit(57340, 1).
digit(12643, 1).
digit(59738, 1).
digit(30058, 0).
digit(41285, 0).
digit(1775, 0).
digit(58253, 1).
digit(53857, 0).
digit(39830, 1).
digit(4656, 0).
digit(54855, 1).
digit(15851, 1).
digit(4561, 1).
digit(1590, 0).
digit(55286, 1).
digit(34272, 1).
digit(20706, 1).
digit(19908, 0).
digit(55014, 1).
digit(5305, 1).
digit(35793, 0).
digit(43536, 0).
digit(36468, 0).
digit(44750, 0).
digit(6960, 0).
digit(45451, 1).
digit(26565, 0).
digit(11995, 1).
digit(4129, 1).
digit(24729, 1).
digit(47513, 0).
digit(35212, 1).
digit(41967, 1).
digit(35699, 1).
digit(32937, 1).
digit(18750, 0).
digit(12741, 0).
digit(40217, 0).
digit(37183, 0).
digit(5470, 0).
digit(12414, 0).
digit(30731, 1).
digit(58944, 1).
digit(26620, 0).
digit(40732, 0).
digit(3237, 1).
digit(18273, 0).
digit(48895, 0).
digit(47019, 1).
digit(29203, 0).
digit(2765, 0).
digit(9011, 1).
digit(47986, 1).
digit(21640, 0).
digit(21984, 1).
digit(37452, 0).
digit(39711, 1).
digit(20746, 1).
digit(30828, 0).
digit(16944, 1).
digit(40649, 0).
digit(19100, 1).
digit(19753, 1).
digit(2335, 1).
digit(24072, 0).
digit(38558, 1).
digit(58726, 0).
digit(206, 0).
digit(10134, 1).
digit(40280, 0).
digit(20912, 1).
digit(13168, 1).
digit(11395, 1).
digit(56470, 1).
digit(22603, 0).
digit(16642, 0).
digit(59187, 0).
digit(39012, 1).
digit(21314, 0).
digit(35414, 1).
digit(44846, 0).
digit(34263, 1).
digit(40817, 0).
digit(9875, 1).
digit(54808, 0).
digit(20548, 0).
digit(41474, 1).
digit(20391, 0).
digit(31314, 0).
digit(34595, 1).
digit(49443, 0).
digit(7317, 1).
digit(37415, 0).
digit(44368, 1).
digit(42945, 0).
digit(56950, 0).
digit(40162, 0).
digit(429, 0).
digit(47770, 1).
digit(12375, 1).
digit(29311, 0).
digit(31058, 1).
digit(40521, 0).
digit(56954, 0).
digit(34037, 0).
digit(40188, 0).
digit(7834, 1).
digit(37441, 1).
digit(22828, 1).
digit(32098, 0).
digit(32550, 0).
digit(47535, 1).
digit(35811, 1).
digit(36071, 1).
digit(3776, 0).
digit(52220, 1).
digit(3369, 0).
digit(27894, 0).
digit(50931, 1).
digit(41449, 1).
digit(41077, 0).
digit(59713, 1).
digit(39838, 1).
digit(20141, 0).
digit(34073, 0).
digit(13377, 0).
digit(35209, 0).
digit(36143, 0).
digit(20877, 0).
digit(33843, 0).
digit(42175, 1).
digit(52531, 1).
digit(51006, 0).
digit(6318, 1).
digit(27091, 1).
digit(50068, 1).
digit(2539, 0).
digit(33482, 0).
digit(5756, 1).
digit(28815, 1).
digit(27634, 1).
digit(50273, 1).
digit(31870, 1).
digit(29062, 1).
digit(8304, 0).
digit(3411, 1).
digit(16227, 0).
digit(10915, 1).
digit(20593, 1).
digit(46518, 1).
digit(56089, 1).
digit(9869, 0).
digit(48815, 0).
digit(25408, 0).
digit(54985, 0).
digit(9530, 0).
digit(1479, 0).
digit(17620, 0).
digit(2446, 1).
digit(32038, 1).
digit(52179, 0).
digit(22757, 0).
digit(38103, 0).
digit(33869, 1).
digit(31853, 1).
digit(57601, 0).
digit(36085, 1).
digit(47555, 0).
digit(37, 0).
digit(3019, 1).
digit(5570, 1).
digit(57022, 1).
digit(50291, 0).
digit(26406, 1).
digit(39843, 1).
digit(30292, 1).
digit(38100, 0).
digit(20843, 0).
digit(18800, 1).
digit(51352, 0).
digit(12796, 0).
digit(1989, 1).
digit(19516, 1).
digit(8760, 1).
digit(3900, 1).
digit(31505, 0).
digit(8491, 1).
digit(23845, 0).
digit(7658, 1).
digit(32062, 0).
digit(28964, 1).
digit(399, 0).
digit(51520, 0).
digit(14202, 0).
digit(6419, 1).
digit(41266, 1).
digit(30187, 0).
digit(29263, 0).
digit(57243, 1).
digit(21230, 1).
digit(56882, 1).
digit(11134, 1).
digit(23554, 1).
digit(5648, 1).
digit(17030, 1).
digit(43834, 0).
digit(10245, 0).
digit(29811, 1).
digit(45537, 1).
digit(43905, 1).
digit(16805, 1).
digit(3888, 0).
digit(14729, 0).
digit(45960, 0).
digit(6967, 0).
digit(52095, 0).
digit(25734, 1).
digit(56456, 1).
digit(22787, 0).
digit(34141, 1).
digit(33243, 1).
digit(53371, 1).
digit(13077, 0).
digit(37196, 1).
digit(51847, 1).
digit(31751, 1).
digit(24717, 0).
digit(54609, 0).
digit(39564, 0).
digit(57027, 0).
digit(15445, 0).
digit(39007, 1).
digit(16458, 1).
digit(52202, 0).
digit(33856, 1).
digit(42197, 1).
digit(17072, 1).
digit(22227, 1).
digit(10924, 0).
digit(6499, 1).
digit(47205, 0).
digit(44518, 0).
digit(59078, 0).
digit(3130, 1).
digit(46304, 1).
digit(52878, 1).
digit(48218, 0).
digit(23380, 0).
digit(26539, 1).
digit(39175, 0).
digit(51265, 0).
digit(33848, 0).
digit(21506, 0).
digit(5395, 1).
digit(9707, 0).
digit(2955, 0).
digit(4344, 0).
digit(31864, 1).
digit(41425, 0).
digit(4445, 1).
digit(44278, 0).
digit(14584, 1).
digit(59869, 0).
digit(49811, 1).
digit(17964, 1).
digit(28328, 0).
digit(11793, 1).
digit(20814, 0).
digit(50628, 0).
digit(58265, 0).
digit(18266, 1).
digit(17422, 1).
digit(43007, 1).
digit(34337, 1).
digit(41525, 1).
digit(3456, 1).
digit(50749, 0).
digit(32041, 1).
digit(14682, 1).
digit(12336, 1).
digit(14427, 1).
digit(28244, 1).
digit(25496, 1).
digit(38340, 1).
digit(42817, 0).
digit(38499, 1).
digit(33277, 0).
digit(50968, 0).
digit(26853, 0).
digit(25751, 1).
digit(35800, 0).
digit(45766, 0).
digit(2581, 0).
digit(32814, 1).
digit(22987, 0).
digit(23662, 0).
digit(1031, 1).
digit(58765, 0).
digit(51750, 1).
digit(42967, 1).
digit(13734, 0).
digit(23908, 1).
digit(29030, 1).
digit(38159, 1).
digit(29736, 0).
digit(52438, 0).
digit(45363, 1).
digit(57317, 1).
digit(33487, 1).
digit(47384, 1).
digit(52262, 1).
digit(32807, 1).
digit(5525, 0).
digit(14675, 0).
digit(21552, 1).
digit(54458, 1).
digit(44045, 0).
digit(3725, 1).
digit(16447, 1).
digit(4731, 1).
digit(9492, 1).
digit(57998, 1).
digit(49528, 0).
digit(11326, 1).
digit(30240, 0).
digit(17741, 0).
digit(25383, 0).
digit(28325, 0).
digit(33600, 1).
digit(37882, 1).
digit(59457, 0).
digit(42601, 1).
digit(9744, 0).
digit(45373, 1).
digit(13338, 1).
digit(47583, 1).
digit(20643, 1).
digit(48266, 1).
digit(6834, 1).
digit(25953, 0).
digit(5809, 0).
digit(26993, 1).
digit(14234, 1).
digit(55155, 1).
digit(18226, 1).
digit(49224, 0).
digit(13967, 1).
digit(17293, 0).
digit(31787, 1).
digit(18942, 1).
digit(17180, 1).
digit(10276, 0).
digit(47470, 0).
digit(56865, 1).
digit(45354, 0).
digit(43779, 1).
digit(3714, 1).
digit(51045, 0).
digit(9194, 0).
digit(52369, 1).
digit(2103, 1).
digit(4407, 1).
digit(28403, 0).
digit(50061, 0).
digit(28784, 0).
digit(36822, 1).
digit(38364, 0).
digit(34025, 0).
digit(52130, 0).
digit(37384, 1).
digit(5375, 1).
digit(49008, 1).
digit(16665, 1).
digit(27136, 1).
digit(45467, 0).
digit(57757, 0).
digit(43446, 1).
digit(10362, 1).
digit(36536, 1).
digit(21334, 1).
digit(13074, 0).
digit(18923, 0).
digit(21964, 1).
digit(12436, 1).
digit(961, 1).
digit(5514, 0).
digit(58741, 1).
digit(4193, 1).
digit(36095, 1).
digit(2432, 1).
digit(46685, 1).
digit(46520, 1).
digit(44629, 0).
digit(15581, 0).
digit(41862, 1).
digit(32313, 0).
digit(24444, 0).
digit(37936, 1).
digit(27407, 0).
digit(4467, 1).
digit(6678, 1).
digit(59586, 1).
digit(34401, 0).
digit(2831, 1).
digit(51285, 1).
digit(11634, 1).
digit(25524, 1).
digit(36183, 0).
digit(59043, 0).
digit(43770, 1).
digit(35612, 0).
digit(48532, 0).
digit(16562, 1).
digit(24348, 0).
digit(47174, 0).
digit(33673, 0).
digit(37343, 1).
digit(1495, 0).
digit(58247, 0).
digit(4804, 0).
digit(5850, 1).
digit(19584, 0).
digit(51749, 1).
digit(32109, 0).
digit(2897, 0).
digit(11019, 0).
digit(39417, 1).
digit(24368, 1).
digit(48727, 0).
digit(51548, 1).
digit(22689, 1).
digit(24389, 0).
digit(44389, 1).
digit(30938, 1).
digit(19297, 1).
digit(19714, 0).
digit(30073, 1).
digit(54549, 0).
digit(13992, 1).
digit(52967, 1).
digit(2110, 1).
digit(15380, 1).
digit(39407, 0).
digit(10783, 1).
digit(22013, 0).
digit(18102, 0).
digit(44227, 1).
digit(45320, 1).
digit(25327, 1).
digit(18953, 1).
digit(59536, 0).
digit(53587, 1).
digit(27821, 1).
digit(20888, 1).
digit(19622, 1).
digit(30218, 1).
digit(23404, 1).
digit(18174, 1).
digit(52867, 1).
digit(1243, 1).
digit(40414, 0).
digit(997, 0).
digit(45061, 0).
digit(41859, 1).
digit(15612, 1).
digit(4123, 0).
digit(15530, 0).
digit(56656, 1).
digit(50292, 1).
digit(37719, 1).
digit(47243, 1).
digit(56226, 1).
digit(38644, 0).
digit(99, 1).
digit(6738, 0).
digit(30496, 0).
digit(27549, 1).
digit(20281, 1).
digit(59658, 1).
digit(9804, 1).
digit(6558, 1).
digit(35436, 1).
digit(58926, 1).
digit(19604, 0).
digit(52002, 1).
digit(711, 1).
digit(3143, 0).
digit(24168, 1).
digit(23154, 0).
digit(21171, 1).
digit(59099, 0).
digit(11340, 0).
digit(5627, 0).
digit(6630, 0).
digit(27657, 1).
digit(6082, 1).
digit(46267, 1).
digit(3755, 1).
digit(6748, 1).
digit(53117, 0).
digit(2352, 0).
digit(3703, 1).
digit(22877, 0).
digit(26608, 0).
digit(5662, 0).
digit(56953, 1).
digit(57378, 1).
digit(17135, 1).
digit(29148, 0).
digit(13240, 0).
digit(9240, 1).
digit(27735, 0).
digit(55173, 1).
digit(35819, 1).
digit(20420, 0).
digit(53395, 1).
digit(13216, 0).
digit(50207, 1).
digit(33945, 0).
digit(638, 1).
digit(35862, 1).
digit(16148, 1).
digit(41341, 0).
digit(27659, 0).
digit(29345, 0).
digit(9796, 1).
digit(25569, 1).
digit(29840, 1).
digit(39328, 1).
digit(14154, 1).
digit(8610, 1).
digit(22267, 1).
digit(45006, 1).
digit(38682, 1).
digit(25230, 1).
digit(41487, 1).
digit(37678, 0).
digit(32846, 0).
digit(32785, 0).
digit(25628, 1).
digit(16699, 1).
digit(54239, 1).
digit(47386, 0).
digit(59339, 1).
digit(39646, 1).
digit(57203, 1).
digit(22412, 1).
digit(47119, 1).
digit(38007, 1).
digit(36621, 0).
digit(40373, 1).
digit(19856, 0).
digit(1654, 1).
digit(40003, 0).
digit(23754, 0).
digit(15662, 0).
digit(6142, 1).
digit(38707, 1).
digit(14752, 0).
digit(19168, 1).
digit(54825, 1).
digit(39912, 1).
digit(26268, 1).
digit(8051, 1).
digit(26457, 0).
digit(14965, 0).
digit(11278, 1).
digit(45383, 0).
digit(7650, 1).
digit(19772, 0).
digit(13280, 0).
digit(35286, 1).
digit(55745, 0).
digit(21143, 1).
digit(23813, 0).
digit(24080, 1).
digit(14097, 0).
digit(709, 0).
digit(19830, 1).
digit(47041, 1).
digit(24710, 1).
digit(47272, 1).
digit(49459, 0).
digit(45220, 1).
digit(53255, 0).
digit(56460, 0).
digit(53401, 0).
digit(50281, 0).
digit(24265, 0).
digit(10107, 0).
digit(39347, 0).
digit(37547, 1).
digit(56000, 1).
digit(1945, 1).
digit(42389, 0).
digit(44700, 0).
digit(49171, 0).
digit(24564, 1).
digit(26827, 1).
digit(36524, 0).
digit(58943, 0).
digit(29395, 0).
digit(41514, 1).
digit(4166, 1).
digit(4492, 1).
digit(56522, 0).
digit(41485, 0).
digit(25196, 1).
digit(22829, 0).
digit(5641, 0).
digit(12145, 1).
digit(44661, 1).
digit(46446, 0).
digit(10237, 0).
digit(52317, 1).
digit(35423, 0).
digit(4608, 0).
digit(15493, 1).
digit(10689, 0).
digit(4387, 1).
digit(31059, 0).
digit(44856, 1).
digit(134, 1).
digit(48458, 1).
digit(18645, 0).
digit(35719, 1).
digit(7402, 1).
digit(26029, 1).
digit(39890, 0).
digit(28323, 0).
digit(53919, 1).
digit(46801, 1).
digit(49680, 1).
digit(49595, 0).
digit(17337, 1).
digit(33577, 1).
digit(3418, 1).
digit(13853, 0).
digit(33295, 1).
digit(3966, 1).
digit(7075, 1).
digit(41350, 0).
digit(38573, 0).
digit(52599, 0).
digit(14491, 0).
digit(9310, 0).
digit(36098, 1).
digit(53147, 1).
digit(49150, 0).
digit(29405, 0).
digit(54580, 1).
digit(8533, 0).
digit(39326, 0).
digit(28387, 1).
digit(29416, 1).
digit(19096, 1).
digit(33219, 0).
digit(15761, 0).
digit(2154, 1).
digit(48716, 0).
digit(26190, 1).
digit(2318, 1).
digit(40582, 0).
digit(54597, 0).
digit(25696, 1).
digit(38909, 1).
digit(22046, 0).
digit(7169, 1).
digit(40829, 0).
digit(37467, 0).
digit(28857, 1).
digit(13449, 1).
digit(4971, 1).
digit(3, 1).
digit(3122, 1).
digit(10407, 0).
digit(27305, 0).
digit(23741, 0).
digit(34762, 1).
digit(31766, 1).
digit(58575, 0).
digit(35148, 0).
digit(978, 1).
digit(9313, 0).
digit(2083, 1).
digit(10863, 1).
digit(46865, 1).
digit(54313, 0).
digit(51865, 0).
digit(37565, 1).
digit(11047, 1).
digit(13206, 0).
digit(36537, 0).
digit(31113, 1).
digit(20928, 0).
digit(12357, 0).
digit(1701, 0).
digit(15356, 1).
digit(23984, 1).
digit(46385, 1).
digit(8992, 1).
digit(9750, 0).
digit(43961, 0).
digit(48518, 1).
digit(23749, 1).
digit(53714, 1).
digit(12573, 0).
digit(29957, 1).
digit(19744, 0).
digit(33003, 0).
digit(53750, 1).
digit(43827, 0).
digit(20611, 1).
digit(38057, 0).
digit(51331, 0).
digit(1425, 1).
digit(48592, 1).
digit(46842, 1).
digit(52248, 1).
digit(36968, 1).
digit(24483, 1).
digit(48720, 1).
digit(10121, 0).
digit(7131, 1).
digit(30409, 0).
digit(22452, 1).
digit(16105, 0).
digit(46726, 0).
digit(24711, 1).
digit(2429, 1).
digit(2890, 0).
digit(45837, 0).
digit(3024, 0).
digit(39937, 1).
digit(743, 0).
digit(36628, 0).
digit(49013, 0).
digit(37787, 0).
digit(43214, 1).
digit(9435, 0).
digit(13097, 0).
digit(6247, 1).
digit(15580, 0).
digit(9820, 0).
digit(5023, 1).
digit(58743, 0).
digit(40482, 1).
digit(12144, 0).
digit(41908, 1).
digit(5930, 1).
digit(51, 0).
digit(46683, 0).
digit(33112, 0).
digit(12642, 1).
digit(43350, 1).
digit(1040, 0).
digit(24212, 0).
digit(4003, 1).
digit(30047, 1).
digit(28904, 0).
digit(21682, 1).
digit(46931, 0).
digit(24293, 1).
digit(19742, 1).
digit(52097, 1).
digit(35646, 0).
digit(33095, 1).
digit(29056, 0).
digit(27895, 1).
digit(15034, 1).
digit(15734, 0).
digit(2736, 0).
digit(17725, 0).
digit(19486, 1).
digit(47310, 1).
digit(10667, 1).
digit(4002, 0).
digit(16779, 1).
digit(21542, 0).
digit(49594, 0).
digit(37269, 0).
digit(39485, 1).
digit(5842, 1).
digit(4821, 1).
digit(51742, 1).
digit(7315, 1).
digit(35432, 0).
digit(36929, 0).
digit(26176, 0).
digit(38225, 1).
digit(32160, 0).
digit(4029, 1).
digit(2870, 1).
digit(28102, 0).
digit(12367, 1).
digit(42581, 1).
digit(3763, 1).
digit(2725, 1).
digit(33850, 1).
digit(2008, 1).
digit(45254, 1).
digit(21260, 0).
digit(56608, 0).
digit(13636, 1).
digit(25910, 1).
digit(2919, 0).
digit(57000, 0).
digit(16267, 1).
digit(16591, 1).
digit(56512, 0).
digit(41837, 0).
digit(56626, 0).
digit(12040, 1).
digit(25042, 1).
digit(51305, 1).
digit(31356, 0).
digit(6488, 1).
digit(51669, 0).
digit(56020, 1).
digit(6870, 1).
digit(50485, 0).
digit(2615, 0).
digit(42262, 0).
digit(36564, 0).
digit(2301, 1).
digit(52537, 1).
digit(32207, 0).
digit(19966, 1).
digit(6973, 0).
digit(46603, 0).
digit(41228, 0).
digit(47553, 1).
digit(49667, 1).
digit(20263, 1).
digit(32231, 1).
digit(46010, 0).
digit(12807, 1).
digit(35896, 0).
digit(10886, 1).
digit(19974, 0).
digit(15231, 1).
digit(33110, 1).
digit(29386, 0).
digit(39716, 1).
digit(24103, 0).
digit(51188, 1).
digit(12230, 1).
digit(2162, 1).
digit(29697, 1).
digit(43971, 1).
digit(53001, 1).
digit(1936, 1).
digit(31377, 1).
digit(1017, 1).
digit(2298, 0).
digit(27528, 1).
digit(23399, 1).
digit(12952, 1).
digit(18830, 1).
digit(39437, 1).
digit(41073, 1).
digit(31420, 1).
digit(6745, 1).
digit(28524, 1).
digit(40982, 1).
digit(36094, 1).
digit(30352, 0).
digit(18400, 1).
digit(8272, 0).
digit(57464, 1).
digit(25448, 1).
digit(28235, 1).
digit(7271, 0).
digit(13989, 0).
digit(57436, 1).
digit(13923, 1).
digit(15389, 1).
digit(17540, 1).
digit(860, 0).
digit(30847, 1).
digit(24698, 0).
digit(4502, 1).
digit(42719, 1).
digit(2692, 1).
digit(44871, 0).
digit(42637, 0).
digit(21343, 0).
digit(51563, 1).
digit(31282, 1).
digit(11868, 1).
digit(16221, 0).
digit(58082, 1).
digit(50993, 1).
digit(58553, 0).
digit(10156, 1).
digit(52993, 0).
digit(21082, 0).
digit(31855, 0).
digit(43411, 1).
digit(57193, 1).
digit(18599, 0).
digit(55583, 1).
digit(41809, 1).
digit(35746, 0).
digit(50550, 1).
digit(3632, 0).
digit(58982, 0).
digit(40237, 0).
digit(59712, 0).
digit(32338, 1).
digit(55031, 1).
digit(53503, 0).
digit(8681, 0).
digit(55679, 0).
digit(4220, 0).
digit(18743, 0).
digit(28822, 1).
digit(56575, 1).
digit(52654, 1).
digit(17925, 1).
digit(17032, 1).
digit(53407, 0).
digit(41750, 1).
digit(59387, 1).
digit(28125, 0).
digit(12590, 1).
digit(25547, 0).
digit(13080, 1).
digit(51286, 0).
digit(44560, 0).
digit(35720, 0).
digit(32781, 0).
digit(19011, 1).
digit(53304, 1).
digit(14937, 0).
digit(7693, 1).
digit(9579, 1).
digit(37154, 1).
digit(53678, 0).
digit(23108, 0).
digit(13081, 0).
digit(44744, 0).
digit(14149, 1).
digit(49389, 0).
digit(44867, 0).
digit(16086, 1).
digit(44955, 0).
digit(2530, 1).
digit(17881, 0).
digit(16742, 1).
digit(7563, 1).
digit(13006, 1).
digit(44872, 0).
digit(9476, 0).
digit(54474, 0).
digit(14224, 1).
digit(42156, 1).
digit(38483, 0).
digit(48920, 0).
digit(57780, 1).
digit(16285, 0).
digit(30948, 0).
digit(22512, 1).
digit(7424, 1).
digit(54515, 0).
digit(24663, 1).
digit(6192, 1).
digit(43884, 0).
digit(59827, 1).
digit(31112, 0).
digit(30585, 0).
digit(16758, 0).
digit(18064, 0).
digit(28350, 1).
digit(53454, 0).
digit(34538, 0).
digit(6931, 1).
digit(27068, 1).
digit(18868, 0).
digit(36874, 0).
digit(42765, 1).
digit(14834, 1).
digit(54897, 1).
digit(48168, 0).
digit(5932, 0).
digit(50924, 0).
digit(25839, 0).
digit(36251, 0).
digit(31930, 1).
digit(39977, 1).
digit(29260, 1).
digit(57686, 0).
digit(55481, 0).
digit(33933, 0).
digit(53705, 0).
digit(9209, 0).
digit(36271, 1).
digit(33781, 1).
digit(52728, 0).
digit(51818, 0).
digit(11833, 0).
digit(57586, 0).
digit(37926, 1).
digit(35925, 1).
digit(41052, 0).
digit(49953, 0).
digit(36731, 1).
digit(17685, 1).
digit(34514, 0).
digit(48322, 0).
digit(41041, 0).
digit(9584, 1).
digit(34904, 0).
digit(35971, 0).
digit(28358, 0).
digit(17505, 1).
digit(18889, 0).
digit(17367, 1).
digit(42877, 0).
digit(2147, 0).
digit(39156, 0).
digit(8331, 1).
digit(49530, 0).
digit(27329, 1).
digit(46871, 0).
digit(44126, 1).
digit(39823, 0).
digit(59319, 0).
digit(22912, 0).
digit(38160, 1).
digit(27865, 0).
digit(21272, 1).
digit(56259, 0).
digit(16777, 0).
digit(30706, 0).
digit(46654, 0).
digit(31871, 1).
digit(16540, 0).
digit(58591, 1).
digit(57393, 0).
digit(39789, 0).
digit(3001, 0).
digit(15277, 1).
digit(25092, 0).
digit(55822, 0).
digit(18718, 0).
digit(30926, 0).
digit(4913, 1).
digit(28543, 0).
digit(42371, 1).
digit(3885, 1).
digit(33940, 0).
digit(33613, 1).
digit(30578, 0).
digit(891, 1).
digit(28221, 0).
digit(25067, 1).
digit(50696, 0).
digit(39224, 0).
digit(21680, 1).
digit(21127, 1).
digit(54177, 1).
digit(49035, 1).
digit(538, 1).
digit(34251, 0).
digit(19282, 1).
digit(33732, 0).
digit(18983, 1).
digit(56189, 1).
digit(55716, 1).
digit(25366, 0).
digit(59364, 1).
digit(52696, 0).
digit(36924, 1).
digit(355, 1).
digit(33192, 0).
digit(12691, 0).
digit(8368, 0).
digit(15959, 1).
digit(35774, 1).
digit(10265, 0).
digit(12992, 0).
digit(21811, 1).
digit(19664, 0).
digit(1078, 0).
digit(48059, 0).
digit(53206, 1).
digit(4027, 0).
digit(51033, 1).
digit(35345, 1).
digit(37518, 1).
digit(27257, 0).
digit(42657, 1).
digit(41289, 1).
digit(31695, 0).
digit(7789, 1).
digit(4049, 1).
digit(37191, 1).
digit(25743, 0).
digit(229, 0).
digit(27104, 1).
digit(4218, 0).
digit(231, 1).
digit(28166, 1).
digit(54967, 0).
digit(37839, 1).
digit(2716, 1).
digit(1822, 1).
digit(25235, 1).
digit(33623, 1).
digit(26250, 1).
digit(19491, 0).
digit(32167, 1).
digit(51005, 1).
digit(41158, 0).
digit(47815, 0).
digit(37854, 0).
digit(14958, 1).
digit(3876, 0).
digit(25805, 1).
digit(29974, 1).
digit(45066, 0).
digit(23277, 0).
digit(47776, 1).
digit(15154, 1).
digit(5245, 1).
digit(1174, 1).
digit(27609, 0).
digit(45001, 1).
digit(7372, 0).
digit(1942, 1).
digit(8189, 1).
digit(34992, 1).
digit(88, 0).
digit(18035, 1).
digit(16713, 1).
digit(36331, 0).
digit(17872, 1).
digit(18936, 0).
digit(31463, 0).
digit(7875, 0).
digit(51314, 0).
digit(19407, 0).
digit(47447, 0).
digit(9032, 0).
digit(51888, 1).
digit(50081, 0).
digit(21890, 0).
digit(42044, 1).
digit(34392, 0).
digit(44222, 1).
digit(30300, 0).
digit(1999, 0).
digit(7411, 1).
digit(7729, 1).
digit(17585, 0).
digit(42485, 1).
digit(12115, 0).
digit(18421, 1).
digit(46598, 1).
digit(35659, 0).
digit(13955, 0).
digit(52078, 1).
digit(39505, 1).
digit(4539, 0).
digit(15290, 1).
digit(51737, 0).
digit(20064, 1).
digit(33865, 1).
digit(31836, 0).
digit(54263, 0).
digit(41694, 1).
digit(35508, 1).
digit(48263, 0).
digit(11428, 0).
digit(45693, 1).
digit(44569, 1).
digit(19689, 0).
digit(47263, 0).
digit(10500, 1).
digit(39615, 0).
digit(45909, 1).
digit(33293, 0).
digit(31815, 1).
digit(24188, 0).
digit(38263, 1).
digit(24617, 1).
digit(31971, 1).
digit(51479, 0).
digit(57362, 1).
digit(12704, 1).
digit(33417, 1).
digit(27836, 1).
digit(55348, 0).
digit(30932, 0).
digit(30149, 0).
digit(9856, 1).
digit(49336, 0).
digit(710, 1).
digit(9965, 0).
digit(15383, 0).
digit(2906, 1).
digit(32677, 1).
digit(359, 0).
digit(19968, 1).
digit(50632, 1).
digit(43205, 0).
digit(49002, 1).
digit(5462, 0).
digit(29909, 1).
digit(9822, 0).
digit(43458, 1).
digit(53189, 0).
digit(12652, 0).
digit(48131, 1).
digit(50397, 1).
digit(17856, 1).
digit(2935, 0).
digit(54752, 1).
digit(45532, 1).
digit(26210, 0).
digit(9133, 1).
digit(12239, 0).
digit(32423, 0).
digit(20232, 1).
digit(25704, 1).
digit(32465, 1).
digit(52154, 0).
digit(54927, 0).
digit(37937, 0).
digit(47005, 0).
digit(42487, 0).
digit(9430, 1).
digit(18409, 1).
digit(35108, 1).
digit(24749, 0).
digit(52616, 1).
digit(57746, 0).
digit(8157, 0).
digit(59439, 0).
digit(5344, 0).
digit(10205, 1).
digit(36786, 1).
digit(39931, 1).
digit(12180, 0).
digit(982, 1).
digit(49575, 0).
digit(38941, 1).
digit(1137, 0).
digit(28695, 1).
digit(40537, 1).
digit(33789, 0).
digit(43965, 0).
digit(11954, 1).
digit(48963, 0).
digit(33985, 1).
digit(23786, 0).
digit(20848, 1).
digit(38027, 1).
digit(31677, 0).
digit(9987, 1).
digit(28452, 0).
digit(58181, 1).
digit(38425, 1).
digit(3850, 1).
digit(56176, 0).
digit(26202, 1).
digit(45426, 0).
digit(31904, 0).
digit(13155, 0).
digit(41806, 0).
digit(58493, 1).
digit(31430, 0).
digit(39270, 1).
digit(12323, 1).
digit(691, 1).
digit(55487, 0).
digit(47702, 1).
digit(18899, 1).
digit(47516, 1).
digit(41392, 1).
digit(44266, 0).
digit(37939, 1).
digit(35348, 0).
digit(15940, 1).
digit(10519, 1).
digit(23911, 1).
digit(20955, 0).
digit(5108, 0).
digit(17541, 1).
digit(1386, 0).
digit(15779, 0).
digit(6108, 1).
digit(46496, 1).
digit(17035, 1).
digit(42561, 1).
digit(2415, 1).
digit(48364, 0).
digit(36560, 0).
digit(32165, 0).
digit(39465, 1).
digit(49750, 0).
digit(5502, 0).
digit(19581, 0).
digit(8134, 0).
digit(48185, 0).
digit(26184, 0).
digit(4644, 1).
digit(35756, 1).
digit(38272, 1).
digit(1769, 0).
digit(36506, 0).
digit(13926, 0).
digit(26556, 1).
digit(46797, 0).
digit(16855, 1).
digit(29873, 0).
digit(40241, 0).
digit(37670, 1).
digit(19528, 1).
digit(2160, 0).
digit(27841, 0).
digit(30612, 1).
digit(9678, 1).
digit(47599, 0).
digit(50589, 1).
digit(29057, 1).
digit(52177, 1).
digit(43897, 1).
digit(29718, 0).
digit(54631, 1).
digit(11287, 1).
digit(54121, 0).
digit(27633, 0).
digit(47169, 1).
digit(50813, 0).
digit(52334, 1).
digit(21060, 1).
digit(47187, 1).
digit(3562, 0).
digit(2188, 1).
digit(46855, 1).
digit(29285, 1).
digit(23477, 1).
digit(3016, 0).
digit(13610, 1).
digit(12325, 0).
digit(5905, 0).
digit(7533, 1).
digit(4465, 0).
digit(19119, 0).
digit(56149, 1).
digit(21800, 0).
digit(58889, 1).
digit(34803, 0).
digit(43003, 1).
digit(32482, 1).
digit(19918, 1).
digit(3884, 0).
digit(28969, 1).
digit(42170, 0).
digit(23395, 0).
digit(50444, 0).
digit(33113, 1).
digit(45974, 1).
digit(42701, 1).
digit(25399, 0).
digit(28831, 1).
digit(19278, 1).
digit(39352, 1).
digit(20070, 1).
digit(108, 0).
digit(30389, 1).
digit(30031, 1).
digit(10129, 1).
digit(57381, 1).
digit(44268, 0).
digit(12241, 1).
digit(17286, 1).
digit(435, 0).
digit(36992, 1).
digit(40682, 1).
digit(38357, 1).
digit(20041, 1).
digit(19122, 1).
digit(54514, 0).
digit(20340, 1).
digit(43309, 0).
digit(9693, 1).
digit(59033, 1).
digit(47860, 0).
digit(46494, 0).
digit(59037, 0).
digit(15863, 0).
digit(19709, 0).
digit(12388, 1).
digit(20622, 0).
digit(58727, 1).
digit(25474, 1).
digit(18659, 1).
digit(37938, 1).
digit(50023, 1).
digit(45504, 0).
digit(23367, 0).
digit(50951, 1).
digit(55105, 0).
digit(5767, 0).
digit(38105, 1).
digit(1700, 1).
digit(13835, 1).
digit(50498, 1).
digit(16923, 1).
digit(47490, 1).
digit(27558, 1).
digit(46015, 1).
digit(45682, 0).
digit(39444, 1).
digit(33498, 1).
digit(21, 0).
digit(38683, 1).
digit(22398, 1).
digit(23576, 1).
digit(58909, 1).
digit(8296, 1).
digit(42040, 1).
digit(49519, 1).
digit(52056, 1).
digit(49198, 1).
digit(12479, 1).
digit(43276, 0).
digit(56367, 1).
digit(26685, 0).
digit(43349, 1).
digit(8935, 0).
digit(42483, 0).
digit(1363, 0).
digit(23060, 0).
digit(9456, 0).
digit(18178, 0).
digit(7925, 0).
digit(54502, 0).
digit(29579, 0).
digit(12532, 1).
digit(18912, 1).
digit(25460, 0).
digit(42326, 1).
digit(41944, 1).
digit(14574, 0).
digit(34523, 0).
digit(28809, 0).
digit(38055, 0).
digit(1600, 0).
digit(50577, 1).
digit(40921, 1).
digit(37101, 1).
digit(4563, 0).
digit(42153, 0).
digit(31020, 0).
digit(39714, 0).
digit(2307, 1).
digit(49280, 0).
digit(23660, 1).
digit(47407, 1).
digit(30425, 0).
digit(4294, 1).
digit(5957, 1).
digit(50769, 0).
digit(29849, 1).
digit(28069, 1).
digit(56261, 1).
digit(18261, 1).
digit(41557, 0).
digit(54557, 1).
digit(28076, 0).
digit(31869, 1).
digit(59832, 1).
digit(46604, 0).
digit(58991, 1).
digit(44269, 1).
digit(2403, 0).
digit(10508, 1).
digit(29394, 0).
digit(53484, 1).
digit(8110, 0).
digit(17184, 1).
digit(47202, 0).
digit(8249, 0).
digit(58694, 0).
digit(43332, 1).
digit(5154, 0).
digit(31165, 0).
digit(51992, 0).
digit(35934, 0).
digit(3628, 1).
digit(20506, 1).
digit(51540, 1).
digit(53387, 0).
digit(37506, 0).
digit(15770, 0).
digit(55279, 1).
digit(4190, 1).
digit(2366, 1).
digit(16656, 1).
digit(57961, 1).
digit(8724, 0).
digit(44210, 1).
digit(54649, 0).
digit(13880, 0).
digit(54703, 0).
digit(47581, 0).
digit(45665, 1).
digit(14589, 0).
digit(54123, 1).
digit(4793, 0).
digit(35289, 1).
digit(9523, 0).
digit(31683, 0).
digit(8003, 0).
digit(49197, 0).
digit(46575, 1).
digit(35820, 1).
digit(26390, 1).
digit(33231, 1).
digit(19492, 0).
digit(18352, 1).
digit(13918, 0).
digit(44207, 1).
digit(18013, 1).
digit(39237, 1).
digit(19630, 1).
digit(51187, 1).
digit(6530, 1).
digit(5317, 0).
digit(21497, 0).
digit(25427, 0).
digit(38492, 1).
digit(20409, 1).
digit(6676, 0).
digit(41055, 1).
digit(46560, 0).
digit(3175, 0).
digit(7485, 0).
digit(59197, 0).
digit(16107, 1).
digit(31537, 1).
digit(13903, 1).
digit(5010, 0).
digit(27182, 1).
digit(45205, 1).
digit(37092, 0).
digit(32293, 0).
digit(4565, 0).
digit(50264, 1).
digit(45989, 1).
digit(35311, 0).
digit(8961, 0).
digit(31578, 1).
digit(19292, 0).
digit(11317, 1).
digit(9994, 0).
digit(19731, 1).
digit(5330, 0).
digit(6337, 0).
digit(54605, 1).
digit(19355, 1).
digit(48012, 0).
digit(28334, 1).
digit(29532, 1).
digit(26044, 0).
digit(38008, 0).
digit(21358, 0).
digit(48085, 1).
digit(9618, 1).
digit(16110, 1).
digit(15103, 0).
digit(45035, 0).
digit(4597, 0).
digit(53098, 1).
digit(15242, 0).
digit(14052, 0).
digit(1148, 1).
digit(47796, 1).
digit(46975, 0).
digit(3487, 1).
digit(24308, 0).
digit(47868, 1).
digit(28094, 1).
digit(9393, 1).
digit(36869, 0).
digit(248, 1).
digit(7944, 0).
digit(25206, 0).
digit(25447, 1).
digit(6244, 1).
digit(9277, 1).
digit(9034, 0).
digit(35192, 0).
digit(45900, 1).
digit(53140, 1).
digit(30958, 1).
digit(41108, 0).
digit(53381, 0).
digit(42927, 1).
digit(5697, 0).
digit(20609, 0).
digit(50357, 0).
digit(25497, 0).
digit(17487, 1).
digit(30210, 1).
digit(21614, 0).
digit(45471, 1).
digit(42171, 1).
digit(46809, 1).
digit(59547, 0).
digit(49657, 0).
digit(3661, 0).
digit(53425, 1).
digit(47714, 0).
digit(4849, 0).
digit(35587, 1).
digit(58397, 1).
digit(16254, 1).
digit(41139, 1).
digit(6097, 0).
digit(54968, 1).
digit(6709, 1).
digit(14944, 0).
digit(39683, 0).
digit(29477, 0).
digit(45438, 1).
digit(5527, 1).
digit(4933, 1).
digit(39185, 1).
digit(10909, 0).
digit(59059, 0).
digit(38002, 1).
digit(24151, 0).
digit(36943, 1).
digit(38411, 0).
digit(40895, 0).
digit(15020, 1).
digit(40266, 0).
digit(19507, 1).
digit(38693, 0).
digit(19167, 1).
digit(39715, 0).
digit(21401, 1).
digit(18987, 1).
digit(13779, 0).
digit(33457, 1).
digit(4142, 0).
digit(57613, 1).
digit(45997, 0).
digit(30790, 0).
digit(15575, 1).
digit(45366, 1).
digit(50949, 0).
digit(53780, 1).
digit(21687, 1).
digit(41909, 1).
digit(28202, 0).
digit(59095, 0).
digit(31406, 1).
digit(10579, 0).
digit(26774, 1).
digit(27327, 0).
digit(6409, 0).
digit(12725, 1).
digit(47050, 1).
digit(12236, 1).
digit(50621, 1).
digit(20898, 1).
digit(12107, 0).
digit(20195, 1).
digit(2904, 0).
digit(41029, 1).
digit(53734, 0).
digit(44183, 1).
digit(31169, 1).
digit(15157, 1).
digit(46412, 0).
digit(28318, 1).
digit(15142, 0).
digit(10268, 1).
digit(52081, 1).
digit(53293, 0).
digit(22427, 1).
digit(42732, 0).
digit(30807, 0).
digit(8255, 1).
digit(42176, 1).
digit(5475, 0).
digit(18940, 1).
digit(43728, 0).
digit(16520, 0).
digit(59972, 0).
digit(22352, 0).
digit(35185, 0).
digit(20459, 1).
digit(46542, 1).
digit(56775, 0).
digit(36440, 0).
digit(38926, 0).
digit(15648, 0).
digit(24661, 0).
digit(48627, 0).
digit(16203, 1).
digit(28482, 1).
digit(50944, 0).
digit(46512, 1).
digit(26757, 1).
digit(18497, 1).
digit(26095, 0).
digit(15060, 0).
digit(15119, 0).
digit(35554, 1).
digit(46103, 1).
digit(26731, 0).
digit(14228, 1).
digit(53380, 0).
digit(41914, 1).
digit(29925, 0).
digit(11605, 0).
digit(5053, 0).
digit(21905, 1).
digit(15291, 0).
digit(17709, 0).
digit(57907, 0).
digit(43247, 0).
digit(41667, 0).
digit(5507, 1).
digit(28740, 0).
digit(21173, 1).
digit(14656, 1).
digit(33657, 0).
digit(40658, 1).
digit(29067, 0).
digit(50827, 1).
digit(21791, 1).
digit(28099, 1).
digit(993, 1).
digit(28979, 1).
digit(16906, 1).
digit(33440, 1).
digit(38380, 0).
digit(3896, 0).
digit(33173, 0).
digit(10352, 0).
digit(24814, 1).
digit(2604, 1).
digit(4557, 1).
digit(6277, 1).
digit(3331, 1).
digit(53348, 1).
digit(30966, 1).
digit(59450, 0).
digit(14354, 1).
digit(50026, 0).
digit(47279, 1).
digit(48947, 0).
digit(53427, 1).
digit(10290, 1).
digit(46499, 1).
digit(22288, 0).
digit(7971, 1).
digit(16076, 1).
digit(53403, 0).
digit(58036, 0).
digit(10821, 0).
digit(25595, 1).
digit(41819, 0).
digit(58056, 1).
digit(20895, 0).
digit(31966, 1).
digit(28056, 1).
digit(36277, 0).
digit(41432, 0).
digit(59559, 1).
digit(3829, 0).
digit(42136, 0).
digit(2362, 1).
digit(24645, 0).
digit(38660, 0).
digit(27309, 1).
digit(49504, 1).
digit(37207, 1).
digit(37997, 1).
digit(28019, 1).
digit(23255, 1).
digit(15271, 1).
digit(1458, 1).
digit(50197, 0).
digit(53038, 0).
digit(24973, 1).
digit(47035, 1).
digit(50453, 1).
digit(46963, 0).
digit(35370, 1).
digit(2599, 1).
digit(17351, 1).
digit(19361, 0).
digit(50750, 0).
digit(51259, 0).
digit(28629, 1).
digit(44031, 1).
digit(8176, 1).
digit(51544, 1).
digit(7752, 1).
digit(19623, 1).
digit(31762, 1).
digit(42133, 1).
digit(39685, 0).
digit(21960, 0).
digit(31384, 1).
digit(25542, 0).
digit(28439, 1).
digit(32795, 1).
digit(8542, 1).
digit(44030, 0).
digit(18389, 1).
digit(31778, 0).
digit(21281, 1).
digit(28569, 1).
digit(12140, 0).
digit(46810, 0).
digit(13764, 0).
digit(27628, 0).
digit(10455, 1).
digit(23857, 1).
digit(38572, 1).
digit(52359, 1).
digit(7830, 0).
digit(3644, 1).
digit(7038, 0).
digit(3464, 1).
digit(11807, 0).
digit(32746, 0).
digit(1471, 0).
digit(54817, 0).
digit(13826, 0).
digit(57979, 1).
digit(1049, 0).
digit(42460, 1).
digit(56273, 1).
digit(37079, 0).
digit(30101, 0).
digit(39321, 0).
digit(19189, 0).
digit(7911, 0).
digit(51215, 0).
digit(35698, 1).
digit(31012, 1).
digit(31657, 0).
digit(56949, 0).
digit(30618, 0).
digit(7651, 1).
digit(28208, 1).
digit(42277, 0).
digit(59901, 0).
digit(14712, 0).
digit(8194, 1).
digit(35400, 0).
digit(19441, 1).
digit(39393, 1).
digit(4994, 0).
digit(35329, 1).
digit(14212, 1).
digit(51815, 0).
digit(13483, 0).
digit(7219, 0).
digit(49154, 1).
digit(34054, 0).
digit(45767, 1).
digit(16163, 0).
digit(5019, 0).
digit(49099, 1).
digit(15704, 0).
digit(49263, 0).
digit(47501, 1).
digit(48677, 0).
digit(29239, 1).
digit(34016, 0).
digit(7981, 0).
digit(48039, 0).
digit(20348, 1).
digit(29086, 1).
digit(48808, 0).
digit(44113, 1).
digit(15962, 1).
digit(5239, 1).
digit(25252, 0).
digit(3120, 1).
digit(24629, 0).
digit(1132, 1).
digit(9458, 0).
digit(12049, 1).
digit(49876, 1).
digit(27785, 1).
digit(293, 0).
digit(39957, 1).
digit(37008, 1).
digit(9195, 0).
digit(36898, 1).
digit(59705, 1).
digit(31829, 1).
digit(6537, 1).
digit(6356, 1).
digit(7558, 1).
digit(50333, 1).
digit(13604, 1).
digit(11143, 1).
digit(5833, 1).
digit(15711, 0).
digit(56343, 1).
digit(32609, 0).
digit(11050, 1).
digit(36239, 1).
digit(26747, 1).
digit(17638, 1).
digit(13286, 0).
digit(51724, 1).
digit(38488, 0).
digit(31202, 0).
digit(4140, 1).
digit(38779, 1).
digit(454, 1).
digit(16238, 0).
digit(45994, 1).
digit(34989, 0).
digit(6444, 0).
digit(18148, 1).
digit(3195, 0).
digit(47075, 1).
digit(9478, 0).
digit(57190, 1).
digit(18833, 1).
digit(49987, 0).
digit(47681, 1).
digit(48780, 0).
digit(48337, 0).
digit(29669, 1).
digit(24808, 1).
digit(58876, 1).
digit(32459, 0).
digit(52930, 0).
digit(3226, 1).
digit(57101, 1).
digit(52625, 0).
digit(26118, 1).
digit(1287, 1).
digit(36562, 0).
digit(44720, 0).
digit(36955, 1).
digit(53891, 1).
digit(397, 1).
digit(46893, 1).
digit(20178, 1).
digit(8704, 1).
digit(36586, 0).
digit(38914, 0).
digit(5483, 1).
digit(8147, 0).
digit(44242, 1).
digit(2782, 0).
digit(11620, 0).
digit(5114, 1).
digit(25786, 1).
digit(56940, 0).
digit(53937, 0).
digit(1387, 0).
digit(49533, 0).
digit(19751, 1).
digit(41789, 0).
digit(16429, 0).
digit(12072, 1).
digit(58647, 0).
digit(8963, 1).
digit(46664, 1).
digit(28714, 0).
digit(44933, 0).
digit(30275, 0).
digit(14817, 0).
digit(35270, 1).
digit(6436, 1).
digit(16014, 1).
digit(27829, 0).
digit(21476, 1).
digit(32543, 1).
digit(16608, 1).
digit(46933, 1).
digit(11746, 0).
digit(56781, 0).
digit(16866, 0).
digit(5644, 1).
digit(34927, 1).
digit(35985, 1).
digit(4577, 1).
digit(32268, 0).
digit(9045, 1).
digit(48663, 1).
digit(24034, 0).
digit(55121, 1).
digit(38746, 0).
digit(20899, 1).
digit(10118, 1).
digit(47022, 1).
digit(53940, 0).
digit(55617, 0).
digit(4585, 1).
digit(55457, 0).
digit(52396, 0).
digit(34486, 0).
digit(41051, 0).
digit(33302, 0).
digit(7345, 1).
digit(59041, 1).
digit(5133, 0).
digit(28743, 1).
digit(38766, 0).
digit(31592, 1).
digit(9815, 1).
digit(2525, 0).
digit(16418, 1).
digit(48127, 1).
digit(30436, 1).
digit(37920, 1).
digit(40944, 1).
digit(3541, 0).
digit(23606, 0).
digit(6885, 1).
digit(27990, 0).
digit(54416, 1).
digit(53824, 1).
digit(2039, 1).
digit(25948, 1).
digit(20094, 1).
digit(6900, 1).
digit(2278, 0).
digit(20632, 0).
digit(41567, 1).
digit(49848, 0).
digit(35244, 1).
digit(20049, 0).
digit(53904, 1).
digit(45790, 1).
digit(18452, 1).
digit(24848, 1).
digit(45495, 1).
digit(51301, 0).
digit(16722, 1).
digit(25414, 1).
digit(35775, 0).
digit(51490, 0).
digit(2437, 1).
digit(34262, 0).
digit(47704, 0).
digit(510, 1).
digit(1368, 0).
digit(55666, 0).
digit(5883, 0).
digit(17534, 0).
digit(50350, 1).
digit(1696, 1).
digit(7882, 0).
digit(59723, 0).
digit(19732, 0).
digit(9742, 0).
digit(49111, 0).
digit(4321, 0).
digit(46192, 0).
digit(53173, 0).
digit(55671, 1).
digit(27465, 1).
digit(27409, 1).
digit(28519, 0).
digit(10027, 1).
digit(50400, 1).
digit(22621, 1).
digit(17134, 0).
digit(33319, 1).
digit(47616, 0).
digit(8343, 0).
digit(21975, 0).
digit(13112, 1).
digit(46782, 1).
digit(7158, 0).
digit(46705, 1).
digit(51491, 0).
digit(17606, 0).
digit(11671, 1).
digit(36353, 1).
digit(15566, 0).
digit(31369, 0).
digit(18989, 0).
digit(39093, 0).
digit(51432, 1).
digit(17782, 1).
digit(33118, 0).
digit(13254, 1).
digit(35943, 0).
digit(43249, 1).
digit(20686, 1).
digit(23301, 0).
digit(57223, 1).
digit(30866, 1).
digit(22325, 1).
digit(1819, 0).
digit(32054, 1).
digit(43129, 0).
digit(9181, 1).
digit(20114, 0).
digit(26219, 0).
digit(30567, 1).
digit(20531, 0).
digit(23302, 1).
digit(22464, 1).
digit(16474, 0).
digit(33367, 0).
digit(37841, 0).
digit(38993, 0).
digit(8218, 1).
digit(10892, 1).
digit(19861, 0).
digit(49832, 1).
digit(12554, 0).
digit(22586, 1).
digit(3391, 0).
digit(21010, 1).
digit(5749, 1).
digit(52906, 1).
digit(43527, 0).
digit(43283, 0).
digit(25584, 0).
digit(45147, 1).
digit(24461, 1).
digit(4958, 1).
digit(38831, 1).
digit(59806, 1).
digit(5468, 0).
digit(27969, 1).
digit(26475, 0).
digit(10806, 0).
digit(21355, 0).
digit(24923, 1).
digit(52411, 1).
digit(8897, 0).
digit(52105, 0).
digit(40284, 0).
digit(44656, 0).
digit(53016, 1).
digit(47739, 0).
digit(13157, 1).
digit(53101, 0).
digit(16485, 0).
digit(11969, 0).
digit(21273, 0).
digit(10058, 1).
digit(33224, 1).
digit(23739, 0).
digit(38731, 1).
digit(42797, 0).
digit(28292, 1).
digit(572, 1).
digit(18492, 1).
digit(53800, 0).
digit(57068, 0).
digit(43960, 0).
digit(44047, 1).
digit(54836, 0).
digit(32332, 1).
digit(4488, 0).
digit(44184, 0).
digit(17392, 0).
digit(9463, 0).
digit(45190, 0).
digit(7466, 0).
digit(34612, 1).
digit(35700, 0).
digit(12550, 0).
digit(18806, 1).
digit(40196, 1).
digit(54373, 1).
digit(59297, 1).
digit(33045, 1).
digit(21066, 0).
digit(56772, 1).
digit(13590, 1).
digit(55832, 1).
digit(29478, 0).
digit(45037, 1).
digit(24535, 1).
digit(41286, 0).
digit(54080, 0).
digit(15345, 1).
digit(12321, 0).
digit(41641, 1).
digit(39055, 0).
digit(6682, 1).
digit(36535, 1).
digit(35853, 0).
digit(45499, 1).
digit(59247, 1).
digit(10568, 1).
digit(43764, 1).
digit(28242, 0).
digit(46007, 0).
digit(10005, 0).
digit(43465, 0).
digit(31716, 0).
digit(1647, 1).
digit(50208, 0).
digit(23182, 1).
digit(29151, 0).
digit(52593, 0).
digit(40880, 1).
digit(8904, 1).
digit(45418, 0).
digit(23210, 0).
digit(45768, 1).
digit(10531, 0).
digit(50501, 1).
digit(2325, 0).
digit(2878, 1).
digit(20674, 1).
digit(19583, 0).
digit(11888, 0).
digit(49123, 1).
digit(16606, 0).
digit(25389, 0).
digit(50608, 1).
digit(48607, 0).
digit(42358, 0).
digit(29278, 1).
digit(59786, 0).
digit(2123, 1).
digit(54004, 0).
digit(7427, 0).
digit(6683, 1).
digit(4766, 1).
digit(2807, 1).
digit(38501, 0).
digit(25336, 1).
digit(39159, 1).
digit(10167, 0).
digit(2336, 1).
digit(13582, 0).
digit(40071, 1).
digit(47982, 1).
digit(31171, 0).
digit(9474, 1).
digit(12276, 0).
digit(52226, 0).
digit(33339, 1).
digit(57272, 1).
digit(8161, 1).
digit(8159, 0).
digit(7536, 1).
digit(11939, 1).
digit(51426, 1).
digit(42586, 1).
digit(55582, 0).
digit(10983, 1).
digit(57435, 0).
digit(31357, 1).
digit(11857, 1).
digit(27395, 0).
digit(15241, 1).
digit(12869, 0).
digit(38141, 1).
digit(24985, 0).
digit(29284, 1).
digit(3198, 0).
digit(45539, 0).
digit(16130, 1).
digit(869, 0).
digit(4047, 0).
digit(59696, 0).
digit(49954, 0).
digit(44768, 0).
digit(5262, 1).
digit(51478, 0).
digit(13184, 0).
digit(32083, 1).
digit(36213, 0).
digit(54614, 1).
digit(47642, 1).
digit(27345, 1).
digit(20088, 0).
digit(4010, 1).
digit(296, 0).
digit(21071, 1).
digit(37058, 1).
digit(57912, 0).
digit(49971, 0).
digit(10562, 1).
digit(21991, 1).
digit(58457, 0).
digit(270, 1).
digit(46535, 0).
digit(52804, 0).
digit(56764, 1).
digit(1209, 0).
digit(12004, 0).
digit(30051, 0).
digit(51060, 1).
digit(42422, 1).
digit(42837, 1).
digit(47572, 0).
digit(50447, 1).
digit(3603, 0).
digit(29684, 0).
digit(22478, 1).
digit(41961, 1).
digit(17661, 1).
digit(27569, 1).
digit(20719, 0).
digit(23891, 0).
digit(59102, 1).
digit(31575, 0).
digit(22455, 1).
digit(30550, 1).
digit(4487, 1).
digit(11522, 1).
digit(6977, 1).
digit(24436, 1).
digit(6279, 0).
digit(3592, 1).
digit(50855, 1).
digit(57003, 0).
digit(44372, 0).
digit(28015, 1).
digit(50379, 0).
digit(26840, 0).
digit(46437, 1).
digit(19544, 1).
digit(32189, 1).
digit(51618, 0).
digit(4014, 1).
digit(8970, 1).
digit(201, 1).
digit(13556, 1).
digit(40696, 1).
digit(30508, 1).
digit(19461, 1).
digit(7409, 0).
digit(36393, 0).
digit(21546, 0).
digit(32540, 1).
digit(45812, 0).
digit(56831, 1).
digit(50629, 0).
digit(21319, 0).
digit(48993, 1).
digit(2784, 1).
digit(13464, 0).
digit(3345, 1).
digit(46443, 0).
digit(33458, 1).
digit(50120, 1).
digit(36738, 1).
digit(34099, 0).
digit(51680, 0).
digit(40179, 0).
digit(20934, 0).
digit(738, 1).
digit(17930, 0).
digit(26514, 1).
digit(43344, 1).
digit(25152, 0).
digit(2547, 1).
digit(55006, 0).
digit(28131, 0).
digit(838, 1).
digit(17379, 0).
digit(44986, 1).
digit(19594, 0).
digit(29438, 0).
digit(51129, 1).
digit(56353, 0).
digit(43801, 1).
digit(2975, 0).
digit(36551, 1).
digit(6629, 0).
digit(16787, 0).
digit(51883, 0).
digit(42494, 1).
digit(25005, 0).
digit(39600, 0).
digit(42448, 0).
digit(29447, 1).
digit(55714, 1).
digit(43243, 0).
digit(19671, 1).
digit(47135, 0).
digit(37744, 0).
digit(3376, 0).
digit(5120, 0).
digit(3259, 0).
digit(35777, 1).
digit(35357, 1).
digit(41122, 0).
digit(17327, 0).
digit(46812, 0).
digit(491, 1).
digit(9599, 1).
digit(51417, 1).
digit(43273, 0).
digit(25800, 1).
digit(75, 0).
digit(43913, 0).
digit(43727, 0).
digit(22914, 1).
digit(56163, 0).
digit(40724, 1).
digit(22892, 1).
digit(4163, 1).
digit(42488, 0).
digit(49308, 1).
digit(32914, 1).
digit(23353, 1).
digit(59728, 1).
digit(52312, 0).
digit(7164, 0).
digit(45075, 1).
digit(12030, 1).
digit(33467, 1).
digit(40974, 1).
digit(20787, 1).
digit(16144, 0).
digit(45087, 1).
digit(47758, 1).
digit(57268, 1).
digit(22744, 0).
digit(57161, 0).
digit(4390, 1).
digit(7874, 0).
digit(1065, 1).
digit(38554, 1).
digit(58588, 1).
digit(2086, 1).
digit(37780, 0).
digit(19258, 0).
digit(15164, 1).
digit(51866, 0).
digit(24974, 1).
digit(25082, 0).
digit(2731, 1).
digit(36783, 0).
digit(42913, 0).
digit(43059, 0).
digit(24737, 1).
digit(37330, 1).
digit(8901, 0).
digit(5073, 1).
digit(47000, 0).
digit(58313, 0).
digit(11297, 1).
digit(35396, 1).
digit(43910, 1).
digit(37038, 1).
digit(10992, 0).
digit(26476, 1).
digit(3667, 0).
digit(7669, 0).
digit(13911, 0).
digit(43418, 0).
digit(8471, 1).
digit(52064, 0).
digit(24842, 0).
digit(4374, 0).
digit(58052, 1).
digit(14936, 1).
digit(13203, 0).
digit(1123, 1).
digit(50311, 0).
digit(28692, 0).
digit(5635, 1).
digit(2404, 0).
digit(58377, 1).
digit(23926, 0).
digit(31367, 0).
digit(27187, 1).
digit(33069, 1).
digit(51083, 1).
digit(57440, 1).
digit(55988, 1).
digit(47693, 0).
digit(17245, 0).
digit(21446, 1).
digit(37393, 1).
digit(45182, 1).
digit(39818, 1).
digit(41812, 1).
digit(23181, 1).
digit(48464, 1).
digit(27069, 0).
digit(31341, 0).
digit(28378, 1).
digit(17457, 1).
digit(27209, 0).
digit(4201, 0).
digit(37901, 0).
digit(1002, 1).
digit(43227, 1).
digit(25390, 0).
digit(9753, 0).
digit(44679, 1).
digit(18505, 0).
digit(53747, 0).
digit(21586, 0).
digit(44445, 0).
digit(8422, 1).
digit(10180, 1).
digit(51733, 0).
digit(44917, 0).
digit(20963, 1).
digit(50852, 0).
digit(58110, 0).
digit(50838, 1).
digit(21247, 1).
digit(3080, 1).
digit(1697, 1).
digit(45931, 1).
digit(12805, 1).
digit(23131, 0).
digit(51195, 1).
digit(6316, 0).
digit(46625, 0).
digit(23124, 0).
digit(8602, 1).
digit(13398, 1).
digit(28274, 0).
digit(26142, 1).
digit(23822, 1).
digit(34222, 1).
digit(15765, 0).
digit(14906, 1).
digit(27071, 1).
digit(40486, 0).
digit(38198, 0).
digit(38077, 1).
digit(32871, 1).
digit(19211, 1).
digit(19930, 0).
digit(25461, 0).
digit(33918, 1).
digit(2411, 0).
digit(26205, 1).
digit(32621, 0).
digit(4654, 1).
digit(30305, 1).
digit(43170, 0).
digit(47479, 0).
digit(1571, 0).
digit(21789, 0).
digit(38412, 0).
digit(6496, 0).
digit(56333, 1).
digit(48456, 1).
digit(54997, 0).
digit(19770, 0).
digit(32946, 0).
digit(48041, 1).
digit(526, 0).
digit(30331, 0).
digit(18316, 1).
digit(38177, 0).
digit(52596, 0).
digit(33363, 0).
digit(11004, 1).
digit(10555, 1).
digit(1259, 1).
digit(52204, 0).
digit(53310, 1).
digit(22937, 1).
digit(58719, 1).
digit(55371, 1).
digit(52921, 0).
digit(53460, 0).
digit(51390, 1).
digit(19304, 1).
digit(50539, 0).
digit(34985, 0).
digit(15520, 0).
digit(55634, 1).
digit(32892, 1).
digit(26225, 1).
digit(57094, 1).
digit(42369, 0).
digit(51186, 0).
digit(2583, 1).
digit(59036, 1).
digit(42518, 1).
digit(43761, 1).
digit(58396, 0).
digit(18741, 1).
digit(40464, 1).
digit(57730, 0).
digit(37708, 0).
digit(50654, 0).
digit(43071, 1).
digit(26174, 1).
digit(30129, 1).
digit(32447, 1).
digit(45288, 0).
digit(55593, 0).
digit(58285, 0).
digit(639, 0).
digit(56436, 0).
digit(38196, 1).
digit(10878, 1).
digit(20510, 0).
digit(16278, 1).
digit(33842, 1).
digit(44723, 0).
digit(4202, 1).
digit(40770, 1).
digit(27604, 1).
digit(23842, 0).
digit(51638, 0).
digit(51845, 0).
digit(45105, 1).
digit(20936, 0).
digit(51150, 0).
digit(23740, 1).
digit(59692, 1).
digit(46725, 1).
digit(22929, 0).
digit(889, 0).
digit(8754, 0).
digit(1044, 1).
digit(58230, 1).
digit(59510, 0).
digit(44089, 1).
digit(33266, 1).
digit(40147, 1).
digit(54404, 0).
digit(35262, 0).
digit(6928, 0).
digit(11371, 1).
digit(21380, 0).
digit(32129, 1).
digit(7850, 1).
digit(15753, 0).
digit(22798, 1).
digit(29215, 1).
digit(32535, 1).
digit(9552, 0).
digit(44598, 0).
digit(48523, 0).
digit(44063, 0).
digit(40760, 0).
digit(6923, 0).
digit(33684, 1).
digit(50808, 0).
digit(53180, 1).
digit(21698, 1).
digit(47797, 0).
digit(13098, 0).
digit(7831, 1).
digit(1535, 1).
digit(56779, 1).
digit(25305, 0).
digit(41331, 0).
digit(57962, 0).
digit(455, 1).
digit(427, 0).
digit(10933, 1).
digit(9733, 0).
digit(43033, 1).
digit(37782, 1).
digit(17473, 1).
digit(7135, 0).
digit(47991, 1).
digit(30363, 0).
digit(47672, 1).
digit(23892, 1).
digit(1349, 0).
digit(14384, 0).
digit(52668, 1).
digit(10488, 0).
digit(26775, 0).
digit(3177, 1).
digit(11051, 1).
digit(46268, 1).
digit(37752, 0).
digit(20478, 0).
digit(46617, 0).
digit(39878, 1).
digit(34531, 1).
digit(44418, 1).
digit(8550, 1).
digit(32974, 1).
digit(37843, 1).
digit(46017, 0).
digit(29235, 0).
digit(47917, 0).
digit(24606, 1).
digit(30388, 0).
digit(28365, 0).
digit(36908, 1).
digit(25049, 1).
digit(19991, 1).
digit(11974, 0).
digit(13649, 1).
digit(34919, 0).
digit(58702, 1).
digit(29598, 1).
digit(42768, 1).
digit(13777, 0).
digit(26445, 1).
digit(50083, 1).
digit(11558, 1).
digit(13238, 0).
digit(24904, 1).
digit(56509, 0).
digit(16943, 1).
digit(4569, 1).
digit(210, 0).
digit(49675, 0).
digit(3434, 0).
digit(55681, 0).
digit(48573, 1).
digit(29621, 0).
digit(35243, 0).
digit(4291, 1).
digit(37921, 0).
digit(59689, 0).
digit(31995, 0).
digit(23604, 1).
digit(2198, 1).
digit(22111, 0).
digit(16441, 1).
digit(30943, 0).
digit(17130, 1).
digit(15883, 0).
digit(36824, 0).
digit(24828, 0).
digit(54486, 0).
digit(32756, 1).
digit(21608, 0).
digit(37642, 0).
digit(50774, 0).
digit(12771, 1).
digit(118, 0).
digit(19127, 0).
digit(28322, 1).
digit(13537, 1).
digit(55288, 0).
digit(52320, 0).
digit(11606, 1).
digit(48295, 0).
digit(17648, 0).
digit(59651, 0).
digit(35409, 0).
digit(13003, 1).
digit(2353, 0).
digit(28878, 1).
digit(19419, 0).
digit(39023, 0).
digit(8985, 1).
digit(51997, 0).
digit(13833, 0).
digit(15392, 1).
digit(46796, 0).
digit(30734, 0).
digit(2327, 0).
digit(38873, 0).
digit(30969, 1).
digit(23003, 1).
digit(41467, 1).
digit(57202, 0).
digit(22173, 1).
digit(124, 1).
digit(29083, 1).
digit(21909, 0).
digit(13501, 1).
digit(33556, 1).
digit(9688, 0).
digit(8984, 1).
digit(41191, 0).
digit(47389, 0).
digit(5244, 0).
digit(23160, 1).
digit(41879, 1).
digit(55664, 0).
digit(49521, 1).
digit(46014, 0).
digit(45855, 1).
digit(45297, 1).
digit(24024, 0).
digit(27462, 0).
digit(11405, 1).
digit(23213, 1).
digit(398, 1).
digit(26960, 1).
digit(30145, 1).
digit(55805, 0).
digit(17984, 1).
digit(55045, 0).
digit(30475, 1).
digit(59613, 0).
digit(53726, 1).
digit(9441, 0).
digit(52047, 0).
digit(38874, 0).
digit(53051, 0).
digit(30270, 1).
digit(8355, 0).
digit(25825, 1).
digit(8689, 1).
digit(44628, 1).
digit(42875, 1).
digit(19480, 0).
digit(39664, 0).
digit(12038, 1).
digit(10040, 1).
digit(42894, 1).
digit(48602, 0).
digit(57471, 0).
digit(46562, 0).
digit(4351, 1).
digit(36431, 0).
digit(50463, 0).
digit(38928, 1).
digit(23498, 1).
digit(34036, 0).
digit(32493, 0).
digit(59175, 0).
digit(2356, 1).
digit(33641, 0).
digit(24429, 0).
digit(39512, 1).
digit(10565, 1).
digit(25952, 0).
digit(1676, 1).
digit(36356, 1).
digit(2882, 0).
digit(56912, 1).
digit(32063, 1).
digit(59783, 1).
digit(269, 1).
digit(59244, 0).
digit(47289, 0).
digit(16636, 1).
digit(23828, 0).
digit(18663, 1).
digit(10675, 1).
digit(12785, 0).
digit(47055, 1).
digit(24933, 1).
digit(31990, 1).
digit(37139, 1).
digit(21884, 1).
digit(3492, 0).
digit(23532, 1).
digit(30156, 0).
digit(23097, 1).
digit(11577, 0).
digit(44811, 0).
digit(38684, 1).
digit(50974, 0).
digit(37563, 1).
digit(39652, 1).
digit(7938, 1).
digit(4979, 1).
digit(57288, 0).
digit(69, 0).
digit(43030, 1).
digit(7921, 1).
digit(16122, 0).
digit(37405, 1).
digit(40909, 1).
digit(58462, 0).
digit(59428, 1).
digit(15713, 1).
digit(1093, 0).
digit(310, 1).
digit(39552, 1).
digit(31862, 1).
digit(41443, 0).
digit(34754, 1).
digit(16354, 0).
digit(13283, 1).
digit(1496, 1).
digit(58806, 1).
digit(3956, 1).
digit(54901, 1).
digit(27506, 1).
digit(50435, 0).
digit(45996, 1).
digit(26492, 1).
digit(58886, 1).
digit(19857, 0).
digit(38454, 1).
digit(45721, 0).
digit(27985, 0).
digit(49068, 1).
digit(17125, 1).
digit(10803, 1).
digit(38591, 1).
digit(49086, 0).
digit(37365, 1).
digit(13315, 1).
digit(46850, 1).
digit(34887, 0).
digit(31279, 1).
digit(25697, 1).
digit(16509, 1).
digit(8530, 1).
digit(11162, 0).
digit(57740, 1).
digit(39297, 1).
digit(37677, 0).
digit(16334, 1).
digit(25352, 1).
digit(45999, 1).
digit(26999, 0).
digit(17770, 0).
digit(3955, 1).
digit(34412, 1).
digit(17639, 1).
digit(22466, 1).
digit(12888, 0).
digit(17259, 1).
digit(7393, 1).
digit(47087, 0).
digit(15716, 1).
digit(25898, 1).
digit(56023, 0).
digit(30368, 1).
digit(2177, 1).
digit(51505, 0).
digit(58155, 1).
digit(20983, 0).
digit(34053, 1).
digit(28427, 0).
digit(6325, 0).
digit(44432, 1).
digit(34775, 0).
digit(5610, 1).
digit(30372, 1).
digit(41437, 1).
digit(7175, 1).
digit(1423, 0).
digit(19841, 1).
digit(41447, 0).
digit(27590, 1).
digit(13769, 0).
digit(58772, 1).
digit(56469, 0).
digit(53908, 1).
digit(21736, 0).
digit(12434, 1).
digit(27025, 1).
digit(15977, 1).
digit(8678, 1).
digit(33936, 1).
digit(48421, 0).
digit(19837, 1).
digit(38449, 1).
digit(38526, 1).
digit(54690, 0).
digit(39568, 1).
digit(25420, 0).
digit(48128, 0).
digit(55315, 0).
digit(59795, 0).
digit(23931, 1).
digit(3929, 0).
digit(20292, 0).
digit(14312, 1).
digit(51017, 0).
digit(26784, 0).
digit(24083, 1).
digit(29430, 1).
digit(56450, 0).
digit(22362, 0).
digit(32710, 1).
digit(24924, 1).
digit(44076, 0).
digit(53518, 1).
digit(15710, 1).
digit(23884, 0).
digit(2839, 0).
digit(40651, 1).
digit(5585, 0).
digit(21794, 1).
digit(15480, 0).
digit(45513, 0).
digit(72, 1).
digit(56778, 0).
digit(3906, 0).
digit(15237, 0).
digit(55478, 1).
digit(12732, 0).
digit(23793, 0).
digit(49232, 1).
digit(21928, 1).
digit(51327, 0).
digit(10136, 1).
digit(42764, 0).
digit(36780, 0).
digit(43229, 0).
digit(35488, 0).
digit(14119, 1).
digit(48629, 1).
digit(27281, 0).
digit(37732, 1).
digit(34440, 0).
digit(28818, 0).
digit(14346, 1).
digit(11883, 1).
digit(12533, 0).
digit(43712, 0).
digit(18468, 1).
digit(54853, 0).
digit(22596, 0).
digit(59454, 1).
digit(5107, 1).
digit(1725, 0).
digit(53627, 1).
digit(39050, 0).
digit(10615, 1).
digit(47216, 1).
digit(38664, 1).
digit(28405, 1).
digit(2306, 1).
digit(3269, 0).
digit(55047, 1).
digit(12880, 0).
digit(39279, 0).
digit(2695, 1).
digit(2937, 1).
digit(28673, 1).
digit(58447, 1).
digit(25823, 0).
digit(15696, 1).
digit(36476, 0).
digit(26961, 1).
digit(8612, 0).
digit(18163, 1).
digit(1028, 0).
digit(33462, 1).
digit(40690, 0).
digit(12119, 0).
digit(11404, 1).
digit(16234, 1).
digit(5048, 0).
digit(37432, 0).
digit(15933, 1).
digit(17779, 0).
digit(54946, 1).
digit(31194, 0).
digit(2767, 1).
digit(48641, 0).
digit(56968, 0).
digit(24893, 0).
digit(57903, 1).
digit(16876, 1).
digit(48661, 0).
digit(58320, 1).
digit(11917, 1).
digit(42261, 1).
digit(35367, 1).
digit(30493, 1).
digit(57092, 0).
digit(44324, 1).
digit(48079, 1).
digit(51120, 1).
digit(54392, 0).
digit(44161, 0).
digit(29541, 0).
digit(57542, 0).
digit(24567, 1).
digit(29274, 1).
digit(5128, 1).
digit(698, 1).
digit(50684, 0).
digit(10919, 0).
digit(24846, 0).
digit(47639, 0).
digit(13611, 1).
digit(5234, 1).
digit(5667, 1).
digit(7930, 0).
digit(47371, 0).
digit(45833, 0).
digit(22897, 0).
digit(23773, 0).
digit(20762, 0).
digit(49429, 0).
digit(38039, 1).
digit(8129, 0).
digit(16563, 0).
digit(24962, 0).
digit(25744, 1).
digit(27023, 0).
digit(47998, 1).
digit(33086, 1).
digit(44036, 1).
digit(30119, 0).
digit(8595, 1).
digit(31487, 1).
digit(54012, 0).
digit(48729, 0).
digit(22004, 1).
digit(35663, 0).
digit(21465, 1).
digit(55680, 0).
digit(39541, 1).
digit(39956, 0).
digit(25899, 1).
digit(24397, 1).
digit(52807, 1).
digit(26429, 1).
digit(15454, 0).
digit(48253, 1).
digit(56404, 0).
digit(43578, 1).
digit(35056, 0).
digit(51148, 0).
digit(26444, 1).
digit(12399, 1).
digit(14890, 0).
digit(51029, 0).
digit(18493, 1).
digit(35742, 0).
digit(13921, 0).
digit(42053, 0).
digit(32353, 1).
digit(38305, 1).
digit(14298, 1).
digit(29337, 1).
digit(44220, 1).
digit(31026, 0).
digit(50103, 1).
digit(40319, 1).
digit(40438, 1).
digit(36953, 0).
digit(33259, 1).
digit(49244, 0).
digit(44120, 1).
digit(20139, 0).
digit(18705, 1).
digit(31172, 0).
digit(25504, 1).
digit(49326, 1).
digit(30246, 0).
digit(783, 1).
digit(22752, 0).
digit(44696, 0).
digit(542, 0).
digit(34681, 1).
digit(27580, 0).
digit(58649, 0).
digit(37224, 0).
digit(40635, 1).
digit(34847, 0).
digit(33800, 1).
digit(43631, 0).
digit(9227, 1).
digit(39243, 0).
digit(33020, 0).
digit(55710, 1).
digit(17549, 0).
digit(10119, 0).
digit(30929, 0).
digit(28923, 0).
digit(51753, 0).
digit(34084, 0).
digit(41265, 0).
digit(24341, 1).
digit(5983, 1).
digit(34937, 1).
digit(17599, 1).
digit(48454, 1).
digit(2586, 0).
digit(25942, 0).
digit(58368, 1).
digit(30748, 0).
digit(5882, 1).
digit(22265, 0).
digit(47914, 0).
digit(12212, 1).
digit(4149, 1).
digit(58600, 1).
digit(35360, 1).
digit(39796, 1).
digit(31846, 1).
digit(43246, 1).
digit(23872, 1).
digit(16178, 0).
digit(13135, 0).
digit(2497, 1).
digit(43323, 1).
digit(45386, 0).
digit(8896, 0).
digit(26473, 1).
digit(55251, 0).
digit(30071, 0).
digit(14629, 0).
digit(17149, 0).
digit(22063, 0).
digit(42440, 1).
digit(35149, 1).
digit(43092, 0).
digit(32721, 1).
digit(4870, 0).
digit(56236, 0).
digit(2532, 0).
digit(25456, 0).
digit(24537, 0).
digit(30413, 0).
digit(23886, 1).
digit(51777, 0).
digit(40313, 0).
digit(50492, 0).
digit(59017, 0).
digit(43013, 0).
digit(18263, 0).
digit(26887, 1).
digit(45546, 1).
digit(16789, 1).
digit(15567, 0).
digit(54544, 1).
digit(45152, 0).
digit(7084, 1).
digit(40260, 0).
digit(44127, 0).
digit(22589, 0).
digit(44777, 0).
digit(58584, 0).
digit(10638, 1).
digit(5072, 0).
digit(49641, 1).
digit(6075, 1).
digit(22931, 1).
digit(46718, 1).
digit(47692, 1).
digit(8578, 1).
digit(34296, 0).
digit(59898, 0).
digit(45128, 0).
digit(32718, 1).
digit(57310, 1).
digit(58458, 0).
digit(36871, 0).
digit(40813, 0).
digit(53176, 1).
digit(53816, 1).
digit(21907, 0).
digit(29488, 0).
digit(8423, 1).
digit(19901, 1).
digit(2938, 1).
digit(36937, 0).
digit(54475, 0).
digit(33661, 0).
digit(26059, 1).
digit(20393, 1).
digit(8116, 0).
digit(55046, 1).
digit(22048, 0).
digit(47835, 0).
digit(26925, 1).
digit(31466, 0).
digit(12264, 0).
digit(25919, 0).
digit(16946, 1).
digit(57005, 1).
digit(23023, 0).
digit(49787, 1).
digit(9121, 1).
digit(55267, 1).
digit(3262, 0).
digit(47031, 0).
digit(48797, 1).
digit(19801, 0).
digit(45476, 0).
digit(28793, 1).
digit(57117, 0).
digit(56326, 1).
digit(40220, 1).
digit(40173, 0).
digit(27854, 0).
digit(25274, 1).
digit(51566, 1).
digit(29409, 0).
digit(44306, 1).
digit(18016, 1).
digit(35708, 0).
digit(57451, 0).
digit(5655, 1).
digit(11509, 1).
digit(36055, 0).
digit(12734, 1).
digit(14169, 1).
digit(19036, 1).
digit(27354, 1).
digit(16202, 0).
digit(26389, 0).
digit(6197, 1).
digit(44622, 1).
digit(20727, 0).
digit(30923, 0).
digit(24121, 0).
digit(38043, 0).
digit(35009, 1).
digit(40544, 0).
digit(50825, 0).
digit(59420, 0).
digit(53823, 1).
digit(46468, 1).
digit(9350, 1).
digit(15803, 1).
digit(22222, 1).
digit(45475, 0).
digit(39263, 0).
digit(27662, 1).
digit(45253, 1).
digit(47722, 0).
digit(7837, 1).
digit(13297, 1).
digit(25781, 1).
digit(44370, 1).
digit(28257, 0).
digit(59289, 1).
digit(38865, 0).
digit(59523, 1).
digit(9986, 0).
digit(16119, 0).
digit(48956, 1).
digit(18022, 0).
digit(25307, 1).
digit(18050, 1).
digit(10471, 0).
digit(18055, 0).
digit(667, 0).
digit(22928, 1).
digit(33667, 1).
digit(17839, 1).
digit(8543, 1).
digit(20438, 1).
digit(3045, 1).
digit(32867, 0).
digit(20177, 1).
digit(57067, 0).
digit(59645, 1).
digit(46772, 0).
digit(13392, 1).
digit(25945, 1).
digit(38713, 1).
digit(1369, 1).
digit(22462, 1).
digit(58002, 0).
digit(52797, 0).
digit(50864, 1).
digit(14897, 0).
digit(51002, 0).
digit(45032, 1).
digit(57321, 0).
digit(37024, 1).
digit(23727, 1).
digit(58673, 1).
digit(11352, 1).
digit(7969, 0).
digit(42093, 1).
digit(15838, 1).
digit(13521, 0).
digit(17043, 1).
digit(46391, 0).
digit(37862, 1).
digit(9699, 0).
digit(41406, 0).
digit(33025, 1).
digit(40615, 1).
digit(46479, 0).
digit(32383, 0).
digit(39955, 0).
digit(20798, 1).
digit(23389, 0).
digit(41446, 1).
digit(5484, 1).
digit(33369, 1).
digit(52175, 1).
digit(57517, 1).
digit(54833, 0).
digit(42401, 1).
digit(41119, 1).
digit(58108, 0).
digit(7074, 1).
digit(25596, 1).
digit(26623, 0).
digit(45230, 0).
digit(13840, 1).
digit(56955, 1).
digit(20047, 1).
digit(40924, 1).
digit(19265, 1).
digit(12947, 0).
digit(37455, 1).
digit(23351, 0).
digit(48578, 0).
digit(9332, 1).
digit(20194, 1).
digit(52031, 1).
digit(51834, 0).
digit(19535, 0).
digit(20321, 0).
digit(24225, 0).
digit(26508, 0).
digit(637, 1).
digit(50396, 1).
digit(22815, 1).
digit(39699, 1).
digit(50876, 0).
digit(2426, 1).
digit(23291, 1).
digit(46722, 1).
digit(2597, 0).
digit(17907, 1).
digit(36667, 0).
digit(30587, 1).
digit(4453, 0).
digit(28922, 1).
digit(4772, 1).
digit(30852, 1).
digit(29468, 0).
digit(36882, 0).
digit(47790, 1).
digit(11205, 1).
digit(23167, 1).
digit(27619, 1).
digit(19099, 1).
digit(9013, 1).
digit(56005, 0).
digit(23390, 1).
digit(4441, 1).
digit(49499, 1).
digit(208, 1).
digit(22938, 0).
digit(32298, 0).
digit(2957, 1).
digit(56538, 1).
digit(22700, 1).
digit(1977, 1).
digit(36198, 0).
digit(46291, 1).
digit(51824, 0).
digit(35052, 1).
digit(5604, 0).
digit(36509, 1).
digit(48177, 1).
digit(6511, 1).
digit(5336, 1).
digit(20418, 0).
digit(9399, 1).
digit(56777, 0).
digit(34634, 0).
digit(12150, 0).
digit(20840, 0).
digit(28411, 1).
digit(32793, 0).
digit(45921, 1).
digit(23036, 1).
digit(10719, 1).
digit(1601, 1).
digit(20608, 0).
digit(49972, 1).
digit(919, 1).
digit(3230, 1).
digit(40033, 1).
digit(28531, 1).
digit(14513, 0).
digit(54719, 1).
digit(26656, 0).
digit(9452, 0).
digit(30547, 0).
digit(21923, 1).
digit(36490, 1).
digit(41777, 1).
digit(26979, 0).
digit(30869, 1).
digit(25940, 1).
digit(48287, 0).
digit(52409, 0).
digit(51147, 0).
digit(32858, 0).
digit(57418, 1).
digit(4602, 1).
digit(17042, 1).
digit(47985, 1).
digit(44857, 1).
digit(24447, 1).
digit(28901, 1).
digit(18042, 0).
digit(45078, 1).
digit(25932, 0).
digit(3802, 1).
digit(41101, 0).
digit(31848, 1).
digit(35087, 0).
digit(24276, 0).
digit(42546, 1).
digit(47576, 1).
digit(56766, 0).
digit(54810, 0).
digit(11292, 1).
digit(4756, 0).
digit(9197, 0).
digit(26903, 0).
digit(27012, 1).
digit(8262, 0).
digit(18543, 0).
digit(19323, 1).
digit(45403, 1).
digit(12374, 0).
digit(19156, 0).
digit(11526, 0).
digit(34541, 0).
digit(49927, 1).
digit(47542, 1).
digit(46410, 0).
digit(3733, 1).
digit(1238, 1).
digit(28540, 0).
digit(43389, 1).
digit(14899, 1).
digit(13574, 1).
digit(47406, 0).
digit(27512, 0).
digit(18384, 1).
digit(52135, 0).
digit(25537, 1).
digit(24896, 0).
digit(29952, 1).
digit(49853, 0).
digit(8309, 0).
digit(6684, 0).
digit(32388, 0).
digit(35502, 0).
digit(3709, 0).
digit(30196, 0).
digit(28196, 1).
digit(51287, 1).
digit(31476, 0).
digit(30751, 0).
digit(13700, 1).
digit(32958, 1).
digit(48790, 0).
digit(56384, 1).
digit(44603, 0).
digit(24809, 0).
digit(52505, 1).
digit(59954, 1).
digit(46915, 1).
digit(18909, 0).
digit(31410, 0).
digit(36778, 1).
digit(18586, 0).
digit(1664, 0).
digit(14334, 0).
digit(39451, 1).
digit(55830, 0).
digit(18853, 1).
digit(57827, 0).
digit(8382, 1).
digit(40418, 0).
digit(1626, 0).
digit(45522, 0).
digit(11112, 1).
digit(19872, 0).
digit(47953, 0).
digit(56138, 1).
digit(19004, 0).
digit(23413, 1).
digit(35757, 0).
digit(11895, 1).
digit(51112, 1).
digit(27436, 0).
digit(26843, 0).
digit(8739, 0).
digit(19018, 1).
digit(24834, 1).
digit(3068, 1).
digit(29704, 0).
digit(55306, 1).
digit(22192, 1).
digit(1497, 1).
digit(56248, 0).
digit(26990, 0).
digit(669, 0).
digit(29869, 0).
digit(2853, 1).
digit(55961, 1).
digit(15198, 0).
digit(25822, 1).
digit(11943, 1).
digit(37182, 1).
digit(21054, 1).
digit(31015, 0).
digit(6254, 1).
digit(45926, 1).
digit(39463, 0).
digit(47735, 1).
digit(20302, 1).
digit(36297, 1).
digit(32489, 0).
digit(42874, 1).
digit(38139, 0).
digit(27283, 0).
digit(27034, 1).
digit(11337, 1).
digit(11315, 0).
digit(40716, 1).
digit(34038, 1).
digit(58386, 0).
digit(35671, 0).
digit(48902, 1).
digit(39193, 1).
digit(9677, 0).
digit(32055, 1).
digit(4970, 1).
digit(25818, 0).
digit(27772, 1).
digit(49124, 0).
digit(54433, 0).
digit(20275, 1).
digit(48543, 1).
digit(11666, 1).
digit(37237, 1).
digit(14717, 0).
digit(9322, 0).
digit(57169, 1).
digit(44704, 0).
digit(36461, 1).
digit(24915, 1).
digit(24405, 1).
digit(59545, 1).
digit(35865, 1).
digit(50797, 1).
digit(54368, 0).
digit(25857, 1).
digit(7985, 0).
digit(3953, 0).
digit(41612, 0).
digit(11845, 0).
digit(31043, 0).
digit(21944, 1).
digit(36517, 1).
digit(5268, 0).
digit(9615, 1).
digit(1528, 1).
digit(1969, 0).
digit(19776, 1).
digit(27681, 1).
digit(41927, 0).
digit(32274, 0).
digit(30034, 0).
digit(14229, 0).
digit(15082, 1).
digit(45235, 0).
digit(41743, 0).
digit(1268, 0).
digit(37716, 1).
digit(28002, 0).
digit(40210, 1).
digit(43706, 0).
digit(8919, 1).
digit(59051, 0).
digit(25687, 1).
digit(21292, 1).
digit(19915, 0).
digit(53785, 0).
digit(8916, 1).
digit(50354, 0).
digit(32682, 0).
digit(37828, 1).
digit(26415, 0).
digit(39376, 0).
digit(2572, 1).
digit(44558, 0).
digit(35538, 1).
digit(55651, 0).
digit(27816, 1).
digit(38828, 1).
digit(28038, 1).
digit(40860, 1).
digit(28690, 0).
digit(26930, 0).
digit(53409, 1).
digit(20678, 1).
digit(22328, 1).
digit(33992, 1).
digit(27936, 0).
digit(3514, 0).
digit(55817, 0).
digit(53177, 1).
digit(47545, 1).
digit(8891, 1).
digit(18207, 1).
digit(46896, 0).
digit(23204, 1).
digit(58773, 1).
digit(3473, 1).
digit(12926, 0).
digit(24227, 1).
digit(49588, 1).
digit(4109, 1).
digit(6228, 0).
digit(40983, 1).
digit(34798, 0).
digit(29985, 0).
digit(18319, 0).
digit(39300, 1).
digit(47880, 1).
digit(49720, 0).
digit(28893, 1).
digit(30194, 1).
digit(36928, 1).
digit(29374, 0).
digit(34586, 0).
digit(2944, 0).
digit(16749, 0).
digit(10203, 0).
digit(7205, 1).
digit(22832, 0).
digit(49518, 0).
digit(42980, 0).
digit(41880, 0).
digit(15255, 1).
digit(48670, 0).
digit(11279, 1).
digit(53344, 0).
digit(11014, 0).
digit(10712, 1).
digit(28992, 0).
digit(51631, 1).
digit(21751, 1).
digit(53383, 1).
digit(38297, 0).
digit(1371, 0).
digit(56908, 0).
digit(14060, 1).
digit(36532, 1).
digit(34004, 1).
digit(28794, 1).
digit(28565, 0).
digit(36894, 0).
digit(29262, 0).
digit(15583, 0).
digit(4696, 1).
digit(44096, 0).
digit(15508, 0).
digit(920, 1).
digit(38806, 1).
digit(50240, 0).
digit(20802, 1).
digit(23473, 1).
digit(45455, 0).
digit(21620, 0).
digit(59868, 0).
digit(17458, 0).
digit(9874, 0).
digit(27256, 1).
digit(52045, 0).
digit(58316, 0).
digit(49210, 1).
digit(39194, 0).
digit(50928, 1).
digit(16585, 0).
digit(8778, 1).
digit(22345, 1).
digit(57486, 0).
digit(16121, 1).
digit(13369, 1).
digit(8383, 1).
digit(11637, 1).
digit(48226, 1).
digit(33396, 1).
digit(40178, 1).
digit(1710, 1).
digit(50907, 1).
digit(57460, 1).
digit(44813, 1).
digit(20487, 0).
digit(7813, 0).
digit(14012, 0).
digit(14545, 0).
digit(56256, 0).
digit(38816, 0).
digit(40294, 1).
digit(33492, 0).
digit(38466, 1).
digit(42783, 0).
digit(58645, 0).
digit(12154, 1).
digit(23305, 0).
digit(27300, 1).
digit(48574, 0).
digit(44029, 0).
digit(10624, 1).
digit(40568, 1).
digit(26409, 1).
digit(34480, 0).
digit(14561, 1).
digit(5505, 0).
digit(38490, 1).
digit(29102, 0).
digit(13154, 1).
digit(47887, 0).
digit(33111, 0).
digit(57437, 1).
digit(4009, 0).
digit(34394, 0).
digit(22629, 0).
digit(36090, 0).
digit(47822, 0).
digit(15073, 1).
digit(11676, 1).
digit(1926, 0).
digit(48481, 1).
digit(53851, 0).
digit(29556, 1).
digit(39017, 1).
digit(45907, 1).
digit(32559, 1).
digit(32704, 1).
digit(23693, 1).
digit(51594, 1).
digit(27111, 1).
digit(44243, 0).
digit(21394, 1).
digit(30220, 0).
digit(38462, 1).
digit(34562, 0).
digit(57973, 1).
digit(28098, 1).
digit(21628, 0).
digit(33645, 0).
digit(11312, 1).
digit(30097, 0).
digit(47799, 1).
digit(54229, 0).
digit(23297, 0).
digit(52042, 1).
digit(22489, 1).
digit(26832, 0).
digit(17883, 1).
digit(39800, 0).
digit(1723, 0).
digit(43484, 1).
digit(14227, 1).
digit(42925, 0).
digit(27547, 0).
digit(40797, 0).
digit(43535, 1).
digit(55719, 1).
digit(59596, 1).
digit(53788, 0).
digit(10145, 1).
digit(25754, 0).
digit(18464, 0).
digit(53187, 1).
digit(6232, 1).
digit(52182, 0).
digit(40989, 0).
digit(47994, 1).
digit(49260, 1).
digit(42947, 1).
digit(8452, 1).
digit(11304, 0).
digit(5007, 0).
digit(49780, 0).
digit(46026, 1).
digit(37153, 0).
digit(28355, 1).
digit(33188, 1).
digit(45617, 0).
digit(16043, 0).
digit(14153, 0).
digit(508, 1).
digit(45741, 0).
digit(21568, 1).
digit(47923, 0).
digit(34080, 1).
digit(56660, 1).
digit(16368, 1).
digit(22656, 1).
digit(55415, 1).
digit(33823, 0).
digit(34381, 0).
digit(23, 1).
digit(28628, 0).
digit(41465, 0).
digit(14874, 1).
digit(46339, 0).
digit(22850, 1).
digit(49037, 1).
digit(29706, 0).
digit(22926, 1).
digit(48650, 1).
digit(29125, 1).
digit(8588, 1).
digit(1838, 1).
digit(14942, 1).
digit(48845, 1).
digit(55993, 0).
digit(37656, 0).
digit(51876, 1).
digit(17452, 0).
digit(48871, 1).
digit(42091, 0).
digit(18471, 0).
digit(49339, 1).
digit(15516, 0).
digit(6408, 1).
digit(10627, 1).
digit(10563, 0).
digit(33663, 1).
digit(25167, 0).
digit(15835, 1).
digit(30035, 1).
digit(52520, 0).
digit(2081, 0).
digit(42456, 0).
digit(24419, 0).
digit(2191, 0).
digit(42362, 0).
digit(8917, 0).
digit(22202, 0).
digit(9151, 0).
digit(38016, 1).
digit(3409, 0).
digit(50523, 0).
digit(1658, 1).
digit(2621, 0).
digit(39859, 0).
digit(8781, 1).
digit(41490, 0).
digit(4981, 0).
digit(1018, 0).
digit(8989, 1).
digit(47446, 1).
digit(19651, 1).
digit(29862, 0).
digit(49939, 0).
digit(38963, 0).
digit(10240, 0).
digit(54775, 1).
digit(52376, 1).
digit(19070, 1).
digit(12670, 0).
digit(25783, 0).
digit(2606, 0).
digit(58882, 0).
digit(28486, 0).
digit(8815, 1).
digit(35496, 1).
digit(787, 0).
digit(53537, 0).
digit(7130, 0).
digit(12324, 0).
digit(47904, 1).
digit(4245, 0).
digit(50789, 1).
digit(47200, 1).
digit(11532, 0).
digit(58784, 0).
digit(54071, 1).
digit(46582, 0).
digit(50771, 1).
digit(15099, 1).
digit(40819, 1).
digit(28522, 1).
digit(23382, 1).
digit(25302, 1).
digit(23921, 1).
digit(952, 0).
digit(13408, 0).
digit(43367, 0).
digit(36669, 1).
digit(17335, 0).
digit(49716, 0).
digit(12555, 0).
digit(53521, 1).
digit(44422, 1).
digit(32326, 1).
digit(50511, 0).
digit(47502, 1).
digit(19339, 1).
digit(28272, 0).
digit(4588, 0).
digit(24932, 1).
digit(10128, 0).
digit(1682, 0).
digit(43118, 1).
digit(29044, 0).
digit(52361, 1).
digit(20298, 1).
digit(55386, 1).
digit(42801, 0).
digit(15438, 1).
digit(10840, 0).
digit(40878, 1).
digit(53684, 0).
digit(23095, 0).
digit(20027, 1).
digit(54303, 0).
digit(34345, 0).
digit(29664, 0).
digit(27114, 0).
digit(16287, 1).
digit(47370, 0).
digit(29567, 1).
digit(17200, 0).
digit(30898, 1).
digit(29564, 0).
digit(27410, 1).
digit(43025, 1).
digit(15473, 1).
digit(2561, 0).
digit(44151, 1).
digit(5196, 0).
digit(53509, 0).
digit(12351, 0).
digit(11453, 0).
digit(45494, 0).
digit(36116, 0).
digit(58769, 0).
digit(48261, 1).
digit(49610, 1).
digit(43216, 0).
digit(54236, 1).
digit(28557, 1).
digit(56910, 1).
digit(12842, 1).
digit(9299, 1).
digit(8294, 0).
digit(35109, 1).
digit(33935, 1).
digit(39986, 1).
digit(7515, 0).
digit(36467, 0).
digit(14408, 1).
digit(54872, 1).
digit(29999, 1).
digit(37337, 1).
digit(38355, 0).
digit(2225, 1).
digit(32514, 0).
digit(27013, 1).
digit(26388, 1).
digit(48173, 0).
digit(22896, 1).
digit(27720, 0).
digit(42499, 0).
digit(57603, 1).
digit(49768, 1).
digit(55682, 0).
digit(23012, 1).
digit(39213, 0).
digit(26421, 1).
digit(416, 1).
digit(25441, 1).
digit(41816, 1).
digit(1293, 1).
digit(53959, 0).
digit(4108, 0).
digit(19880, 1).
digit(18863, 1).
digit(56717, 1).
digit(58609, 0).
digit(50032, 1).
digit(48625, 1).
digit(13825, 1).
digit(11588, 0).
digit(41193, 1).
digit(47579, 0).
digit(55840, 0).
digit(53116, 0).
digit(21342, 1).
digit(42339, 1).
digit(21199, 0).
digit(32237, 0).
digit(36842, 0).
digit(2630, 1).
digit(44589, 0).
digit(5730, 0).
digit(26937, 0).
digit(38918, 1).
digit(34588, 1).
digit(30556, 1).
digit(20749, 1).
digit(10601, 1).
digit(1579, 1).
digit(9698, 0).
digit(6989, 1).
digit(9691, 0).
digit(50036, 0).
digit(22812, 1).
digit(51908, 1).
digit(19072, 0).
digit(20058, 1).
digit(57760, 0).
digit(3212, 1).
digit(12456, 0).
digit(36569, 0).
digit(27205, 1).
digit(14758, 1).
digit(10823, 1).
digit(59199, 1).
digit(28660, 1).
digit(47476, 0).
digit(3430, 1).
digit(58238, 1).
digit(34759, 1).
digit(46926, 0).
digit(50232, 1).
digit(18683, 1).
digit(21068, 1).
digit(14412, 0).
digit(25244, 1).
digit(42752, 0).
digit(45422, 0).
digit(53880, 0).
digit(29559, 0).
digit(15574, 1).
digit(59987, 0).
digit(453, 0).
digit(4532, 0).
digit(6206, 1).
digit(7143, 0).
digit(44125, 1).
digit(20661, 0).
digit(21899, 1).
digit(27886, 1).
digit(22748, 1).
digit(8839, 1).
digit(30871, 0).
digit(10326, 0).
digit(33608, 0).
digit(38975, 1).
digit(10110, 1).
digit(42703, 0).
digit(30176, 1).
digit(13872, 0).
digit(43463, 0).
digit(13896, 0).
digit(40633, 0).
digit(36994, 0).
digit(19225, 0).
digit(51925, 0).
digit(49440, 1).
digit(28910, 0).
digit(50788, 1).
digit(4815, 1).
digit(37287, 0).
digit(22533, 1).
digit(11758, 0).
digit(2977, 1).
digit(45186, 1).
digit(44556, 1).
digit(49218, 0).
digit(37564, 1).
digit(35505, 0).
digit(18372, 1).
digit(44471, 0).
digit(15663, 0).
digit(10366, 0).
digit(12722, 1).
digit(49621, 0).
digit(32162, 0).
digit(102, 1).
digit(35159, 1).
digit(26531, 1).
digit(32562, 1).
digit(55317, 1).
digit(53894, 0).
digit(32427, 0).
digit(22445, 0).
digit(10907, 1).
digit(47638, 1).
digit(56728, 1).
digit(24627, 0).
digit(29098, 1).
digit(34283, 0).
digit(3926, 1).
digit(39169, 1).
digit(2528, 0).
digit(17806, 1).
digit(41226, 1).
digit(26789, 1).
digit(39542, 1).
digit(38774, 1).
digit(15150, 0).
digit(9331, 1).
digit(30555, 0).
digit(5814, 1).
digit(22949, 0).
digit(9038, 0).
digit(54094, 1).
digit(23034, 1).
digit(9158, 0).
digit(38315, 0).
digit(31238, 1).
digit(57929, 1).
digit(32666, 0).
digit(56734, 1).
digit(3551, 1).
digit(37323, 0).
digit(30809, 1).
digit(51564, 1).
digit(27982, 0).
digit(55917, 0).
digit(28110, 1).
digit(39801, 0).
digit(13268, 0).
digit(16484, 0).
digit(776, 0).
digit(34617, 1).
digit(27564, 1).
digit(11361, 0).
digit(19669, 0).
digit(7018, 0).
digit(24777, 0).
digit(15399, 0).
digit(41318, 0).
digit(12885, 1).
digit(54104, 0).
digit(39422, 0).
digit(34471, 0).
digit(2629, 0).
digit(11281, 0).
digit(34980, 1).
digit(58350, 0).
digit(19726, 0).
digit(27585, 1).
digit(27445, 0).
digit(8275, 1).
digit(18339, 1).
digit(6371, 1).
digit(31370, 1).
digit(15964, 1).
digit(10285, 1).
digit(33640, 0).
digit(10982, 1).
digit(38480, 1).
digit(28649, 1).
digit(21354, 0).
digit(55354, 0).
digit(27142, 1).
digit(22254, 0).
digit(33835, 0).
digit(25354, 1).
digit(13962, 0).
digit(59103, 0).
digit(12909, 1).
digit(29828, 1).
digit(58879, 0).
digit(33578, 0).
digit(41733, 1).
digit(46628, 0).
digit(13584, 1).
digit(51889, 0).
digit(50298, 0).
digit(30143, 1).
digit(59478, 1).
digit(51044, 0).
digit(21602, 0).
digit(45331, 1).
digit(33425, 1).
digit(26698, 1).
digit(27737, 1).
digit(15396, 1).
digit(6, 1).
digit(34305, 0).
digit(7078, 1).
digit(30607, 1).
digit(59994, 1).
digit(5713, 1).
digit(14160, 0).
digit(46417, 1).
digit(29205, 0).
digit(26859, 0).
digit(5530, 1).
digit(57829, 1).
digit(42986, 1).
digit(40459, 1).
digit(47452, 0).
digit(19588, 0).
digit(26820, 0).
digit(49471, 0).
digit(41856, 0).
digit(30373, 0).
digit(35028, 0).
digit(3306, 1).
digit(24470, 1).
digit(40479, 0).
digit(15108, 0).
digit(13201, 1).
digit(11742, 0).
digit(19626, 1).
digit(1678, 0).
digit(13985, 1).
digit(45502, 1).
digit(5630, 1).
digit(55215, 1).
digit(8969, 0).
digit(35089, 1).
digit(46913, 1).
digit(33914, 0).
digit(31663, 1).
digit(42008, 0).
digit(48517, 1).
digit(11418, 1).
digit(48236, 1).
digit(28024, 0).
digit(20803, 1).
digit(23506, 0).
digit(29227, 1).
digit(57109, 1).
digit(20755, 1).
digit(16938, 1).
digit(5711, 0).
digit(38311, 1).
digit(17191, 1).
digit(40441, 0).
digit(45437, 1).
digit(35063, 0).
digit(42063, 0).
digit(20361, 1).
digit(6065, 0).
digit(9161, 1).
digit(38226, 1).
digit(12824, 1).
digit(42702, 1).
digit(15882, 0).
digit(30480, 0).
digit(46917, 0).
digit(17005, 1).
digit(56768, 1).
digit(50958, 0).
digit(37607, 1).
digit(40039, 0).
digit(821, 1).
digit(6395, 1).
digit(27322, 0).
digit(34884, 1).
digit(30651, 0).
digit(56570, 0).
digit(19038, 0).
digit(40040, 0).
digit(26256, 0).
digit(57301, 1).
digit(19010, 1).
digit(46583, 1).
digit(1155, 1).
digit(26263, 1).
digit(58360, 0).
digit(466, 1).
digit(36814, 1).
digit(36163, 0).
digit(55030, 0).
digit(54786, 1).
digit(57705, 0).
digit(33084, 1).
digit(42664, 0).
digit(21357, 1).
digit(33108, 1).
digit(27124, 1).
digit(25428, 1).
digit(50194, 1).
digit(39018, 1).
digit(12459, 1).
digit(21002, 1).
digit(53308, 1).
digit(27427, 0).
digit(34542, 1).
digit(32528, 1).
digit(55524, 1).
digit(26604, 1).
digit(6357, 0).
digit(2009, 0).
digit(4624, 0).
digit(13467, 0).
digit(29518, 1).
digit(20642, 0).
digit(58256, 0).
digit(22023, 0).
digit(12635, 0).
digit(31234, 0).
digit(52185, 0).
digit(26152, 1).
digit(22627, 1).
digit(24268, 0).
digit(29575, 1).
digit(7373, 0).
digit(49697, 1).
digit(42444, 0).
digit(13689, 1).
digit(54254, 1).
digit(12897, 0).
digit(30653, 0).
digit(15507, 1).
digit(4288, 0).
digit(45701, 0).
digit(40421, 0).
digit(18229, 1).
digit(24476, 0).
digit(48695, 0).
digit(16694, 1).
digit(12184, 0).
digit(55740, 1).
digit(56757, 1).
digit(46046, 1).
digit(671, 1).
digit(51439, 1).
digit(18029, 0).
digit(6463, 1).
digit(34922, 1).
digit(52187, 1).
digit(9012, 1).
digit(35982, 0).
digit(40884, 1).
digit(6718, 1).
digit(24306, 1).
digit(1107, 0).
digit(48027, 1).
digit(49835, 0).
digit(54812, 0).
digit(7819, 1).
digit(46302, 1).
digit(21178, 1).
digit(34383, 1).
digit(4405, 1).
digit(56161, 0).
digit(6367, 0).
digit(41888, 1).
digit(2255, 1).
digit(45359, 0).
digit(18258, 1).
digit(29673, 1).
digit(27474, 1).
digit(47234, 1).
digit(5884, 0).
digit(57599, 1).
digit(25755, 0).
digit(55236, 1).
digit(21070, 1).
digit(37554, 0).
digit(54479, 1).
digit(10974, 0).
digit(57957, 1).
digit(56114, 1).
digit(58738, 0).
digit(49737, 0).
digit(5469, 0).
digit(26619, 0).
digit(16151, 0).
digit(55056, 0).
digit(4248, 1).
digit(25287, 1).
digit(58161, 1).
digit(1375, 1).
digit(34773, 0).
digit(35386, 1).
digit(48949, 1).
digit(51932, 1).
digit(19552, 1).
digit(57525, 0).
digit(34, 0).
digit(59203, 0).
digit(55890, 0).
digit(3509, 1).
digit(54822, 0).
digit(15555, 1).
digit(3534, 0).
digit(12382, 1).
digit(48061, 1).
digit(51122, 0).
digit(15763, 1).
digit(37290, 1).
digit(20427, 1).
digit(10085, 1).
digit(43602, 0).
digit(30524, 1).
digit(53394, 1).
digit(6207, 0).
digit(7761, 1).
digit(7609, 1).
digit(58116, 0).
digit(59085, 1).
digit(14668, 0).
digit(48126, 1).
digit(5235, 1).
digit(31173, 0).
digit(44578, 0).
digit(31911, 1).
digit(23254, 1).
digit(16885, 1).
digit(4972, 1).
digit(18412, 0).
digit(2823, 1).
digit(11045, 0).
digit(39758, 0).
digit(25914, 0).
digit(38035, 0).
digit(18474, 1).
digit(15697, 1).
digit(11675, 0).
digit(59288, 0).
digit(1717, 1).
digit(8652, 1).
digit(47046, 0).
digit(23856, 0).
digit(593, 1).
digit(15550, 1).
digit(3103, 1).
digit(11295, 0).
digit(11445, 1).
digit(52254, 1).
digit(58196, 1).
digit(8106, 1).
digit(7286, 0).
digit(37624, 1).
digit(35266, 1).
digit(42435, 1).
digit(16838, 1).
digit(20215, 0).
digit(37659, 1).
digit(42740, 1).
digit(7066, 0).
digit(3248, 0).
digit(48459, 0).
digit(18075, 1).
digit(36734, 0).
digit(31198, 1).
digit(42846, 1).
digit(18516, 1).
digit(6020, 1).
digit(40958, 0).
digit(54629, 0).
digit(30473, 0).
digit(24289, 1).
digit(52234, 1).
digit(22337, 0).
digit(3590, 0).
digit(7585, 0).
digit(49387, 0).
digit(46256, 1).
digit(39084, 1).
digit(52288, 0).
digit(46181, 1).
digit(27332, 0).
digit(15809, 1).
digit(29522, 1).
digit(5674, 0).
digit(48254, 1).
digit(12542, 0).
digit(15600, 0).
digit(3375, 1).
digit(44431, 1).
digit(48638, 0).
digit(45021, 0).
digit(9981, 1).
digit(36933, 1).
digit(39746, 0).
digit(4998, 1).
digit(39594, 1).
digit(45092, 1).
digit(8053, 1).
digit(2045, 1).
digit(29101, 0).
digit(26951, 1).
digit(36185, 1).
digit(25606, 1).
digit(53939, 1).
digit(3811, 1).
digit(941, 1).
digit(41542, 1).
digit(17560, 1).
digit(37714, 0).
digit(29490, 0).
digit(46461, 1).
digit(979, 0).
digit(12219, 0).
digit(4715, 1).
digit(46992, 0).
digit(29336, 0).
digit(20161, 0).
digit(10448, 1).
digit(42217, 0).
digit(33826, 1).
digit(50772, 1).
digit(10071, 0).
digit(9260, 1).
digit(17986, 1).
digit(44333, 0).
digit(54689, 0).
digit(57784, 0).
digit(46930, 1).
digit(15621, 0).
digit(42316, 1).
digit(25876, 1).
digit(39848, 0).
digit(26125, 1).
digit(40999, 1).
digit(38605, 0).
digit(40246, 0).
digit(34470, 1).
digit(30088, 0).
digit(35717, 0).
digit(21953, 0).
digit(15208, 0).
digit(23755, 1).
digit(55495, 1).
digit(6979, 0).
digit(25885, 1).
digit(15026, 1).
digit(12605, 0).
digit(17418, 1).
digit(11901, 1).
digit(58668, 0).
digit(28035, 1).
digit(6326, 0).
digit(46980, 0).
digit(18980, 1).
digit(54003, 0).
digit(11429, 0).
digit(4993, 1).
digit(57645, 0).
digit(7689, 1).
digit(57046, 1).
digit(49245, 1).
digit(4284, 0).
digit(48805, 1).
digit(36257, 1).
digit(37705, 0).
digit(283, 0).
digit(21513, 0).
digit(37955, 1).
digit(40287, 1).
digit(29068, 1).
digit(40097, 1).
digit(39637, 1).
digit(54328, 0).
digit(57631, 1).
digit(21350, 0).
digit(14462, 1).
digit(43472, 0).
digit(43704, 1).
digit(56365, 1).
digit(46661, 0).
digit(11412, 1).
digit(35328, 1).
digit(15293, 1).
digit(45240, 1).
digit(37837, 0).
digit(10254, 0).
digit(21052, 1).
digit(28134, 1).
digit(28408, 0).
digit(45162, 0).
digit(20865, 1).
digit(52114, 0).
digit(38095, 1).
digit(14359, 1).
digit(11809, 1).
digit(2899, 0).
digit(29949, 1).
digit(2952, 0).
digit(51247, 1).
digit(38162, 0).
digit(51627, 0).
digit(22646, 0).
digit(1807, 1).
digit(19102, 1).
digit(8982, 0).
digit(39527, 0).
digit(14181, 1).
digit(10564, 0).
digit(35431, 0).
digit(25064, 1).
digit(5203, 0).
digit(15967, 1).
digit(40780, 0).
digit(33965, 1).
digit(45544, 0).
digit(1995, 0).
digit(16516, 0).
digit(22152, 0).
digit(49889, 0).
digit(22764, 0).
digit(17279, 1).
digit(31887, 1).
digit(5479, 1).
digit(22816, 0).
digit(48676, 0).
digit(52048, 0).
digit(37052, 0).
digit(55663, 0).
digit(33947, 0).
digit(17871, 0).
digit(40190, 1).
digit(12103, 0).
digit(11061, 1).
digit(6227, 0).
digit(42183, 0).
digit(36948, 0).
digit(7720, 1).
digit(36571, 1).
digit(4553, 1).
digit(43131, 1).
digit(4504, 1).
digit(25812, 1).
digit(4008, 0).
digit(51051, 1).
digit(51710, 1).
digit(5225, 1).
digit(12457, 0).
digit(55855, 1).
digit(25785, 1).
digit(36829, 0).
digit(14430, 0).
digit(26335, 1).
digit(49355, 1).
digit(28122, 0).
digit(38497, 0).
digit(10220, 1).
digit(52849, 1).
digit(38110, 0).
digit(42279, 1).
digit(50727, 0).
digit(27307, 0).
digit(25767, 1).
digit(57634, 0).
digit(14949, 0).
digit(46125, 0).
digit(56696, 1).
digit(4485, 0).
digit(16211, 1).
digit(42307, 1).
digit(36410, 1).
digit(4237, 1).
digit(29040, 1).
digit(31581, 0).
digit(27896, 1).
digit(28133, 1).
digit(984, 0).
digit(19555, 1).
digit(26835, 1).
digit(34737, 0).
digit(17171, 1).
digit(32454, 0).
digit(21839, 1).
digit(4854, 0).
digit(18208, 0).
digit(38337, 1).
digit(5950, 1).
digit(31277, 0).
digit(48943, 1).
digit(27207, 1).
digit(14126, 1).
digit(40525, 1).
digit(41357, 0).
digit(43556, 0).
digit(10372, 0).
digit(15898, 1).
digit(39717, 1).
digit(51003, 1).
digit(55574, 1).
digit(33136, 0).
digit(44786, 1).
digit(44766, 0).
digit(6186, 1).
digit(41100, 0).
digit(14701, 1).
digit(26984, 1).
digit(25434, 1).
digit(14375, 1).
digit(17260, 1).
digit(15135, 1).
digit(30757, 1).
digit(19243, 1).
digit(9110, 1).
digit(19411, 1).
digit(31877, 0).
digit(11735, 1).
digit(31969, 0).
digit(33534, 0).
digit(18171, 0).
digit(5498, 0).
digit(25719, 1).
digit(22761, 0).
digit(42184, 1).
digit(56989, 0).
digit(29927, 1).
digit(25450, 1).
digit(6875, 0).
digit(46416, 1).
digit(23960, 1).
digit(3777, 0).
digit(38025, 0).
digit(21215, 0).
digit(12441, 1).
digit(36030, 1).
digit(51905, 1).
digit(51451, 1).
digit(23462, 1).
digit(33799, 1).
digit(16292, 1).
digit(16757, 1).
digit(11135, 1).
digit(17425, 1).
digit(51573, 0).
digit(57250, 1).
digit(42132, 1).
digit(24440, 0).
digit(35923, 0).
digit(56971, 1).
digit(26610, 1).
digit(33670, 0).
digit(1168, 0).
digit(6935, 0).
digit(49283, 1).
digit(25512, 0).
digit(14061, 1).
digit(19955, 1).
digit(45659, 1).
digit(41237, 1).
digit(45588, 1).
digit(21829, 1).
digit(35829, 1).
digit(49473, 1).
digit(12013, 1).
digit(14001, 1).
digit(50257, 0).
digit(59382, 1).
digit(32921, 1).
digit(48529, 0).
digit(27457, 1).
digit(35094, 1).
digit(33978, 0).
digit(41939, 1).
digit(4931, 0).
digit(57058, 1).
digit(23214, 1).
digit(5301, 1).
digit(16991, 1).
digit(46740, 0).
digit(35074, 0).
digit(47335, 0).
digit(55601, 1).
digit(43074, 0).
digit(34269, 1).
digit(28079, 0).
digit(12748, 0).
digit(9524, 0).
digit(39968, 0).
digit(29514, 1).
digit(46251, 1).
digit(46723, 0).
digit(11597, 0).
digit(582, 0).
digit(32612, 1).
digit(26645, 0).
digit(49221, 0).
digit(1927, 0).
digit(36617, 0).
digit(44705, 1).
digit(50190, 1).
digit(29407, 1).
digit(36412, 0).
digit(47141, 1).
digit(55587, 1).
digit(19900, 0).
digit(8903, 1).
digit(40181, 1).
digit(49331, 0).
digit(34990, 1).
digit(32221, 0).
digit(42294, 0).
digit(35880, 1).
digit(50751, 1).
digit(58073, 0).
digit(28151, 0).
digit(32531, 0).
digit(18790, 1).
digit(49569, 1).
digit(19188, 1).
digit(23219, 1).
digit(36264, 1).
digit(47108, 1).
digit(58580, 1).
digit(42185, 0).
digit(54685, 0).
digit(27145, 0).
digit(39044, 0).
digit(47960, 1).
digit(56735, 0).
digit(45372, 1).
digit(48513, 1).
digit(16941, 0).
digit(35449, 0).
digit(24824, 1).
digit(39811, 0).
digit(5583, 0).
digit(9329, 0).
digit(6645, 0).
digit(42612, 0).
digit(16660, 1).
digit(28016, 0).
digit(57214, 0).
digit(3604, 1).
digit(39076, 1).
digit(42100, 0).
digit(38677, 1).
digit(13137, 1).
digit(10045, 0).
digit(14019, 0).
digit(49596, 0).
digit(1578, 0).
digit(9695, 0).
digit(30382, 0).
digit(29347, 1).
digit(34792, 1).
digit(406, 1).
digit(4892, 0).
digit(21132, 0).
digit(1640, 1).
digit(11608, 1).
digit(7910, 1).
digit(15046, 0).
digit(14038, 1).
digit(12011, 0).
digit(7318, 0).
digit(18053, 1).
digit(49151, 0).
digit(17029, 1).
digit(55995, 1).
digit(51121, 0).
digit(15491, 0).
digit(26597, 0).
digit(11318, 0).
digit(40604, 1).
digit(43784, 1).
digit(55631, 0).
digit(22137, 1).
digit(18565, 1).
digit(7355, 0).
digit(47268, 1).
digit(32823, 1).
digit(33124, 1).
digit(58864, 0).
digit(8156, 1).
digit(27961, 1).
digit(37980, 0).
digit(17254, 1).
digit(8653, 0).
digit(49362, 1).
digit(50261, 1).
digit(3286, 0).
digit(34730, 0).
digit(39628, 0).
digit(39363, 0).
digit(28361, 1).
digit(55400, 0).
digit(32575, 0).
digit(12847, 1).
digit(27620, 0).
digit(20388, 0).
digit(31332, 1).
digit(42985, 0).
digit(37650, 1).
"""

In [111]:
happdigit = """
% ProbLog Inference Result：
happdigit(0,0).
happdigit(1,1).
happdigit(0,2).
happdigit(1,3).
happdigit(1,4).
happdigit(0,5).
happdigit(1,6).
happdigit(0,7).
happdigit(1,8).
happdigit(1,9).
happdigit(0,10).
happdigit(1,11).
happdigit(1,12).
happdigit(1,13).
happdigit(0,14).
happdigit(0,15).
happdigit(0,16).
happdigit(1,17).
happdigit(1,18).
happdigit(1,19).
happdigit(1,20).
happdigit(1,21).
happdigit(1,22).
happdigit(1,23).
happdigit(0,24).
happdigit(1,25).
happdigit(1,26).
happdigit(0,27).
happdigit(1,28).
happdigit(0,29).
happdigit(1,30).
happdigit(0,31).
happdigit(0,32).
happdigit(1,33).
happdigit(1,34).
happdigit(1,35).
happdigit(1,36).
happdigit(1,37).
happdigit(1,38).
happdigit(1,39).
happdigit(1,40).
happdigit(0,41).
happdigit(1,42).
happdigit(1,43).
happdigit(1,44).
happdigit(0,45).
happdigit(1,46).
happdigit(0,47).
happdigit(0,48).
happdigit(1,49).
happdigit(0,50).
happdigit(0,51).
happdigit(0,52).
happdigit(0,53).
happdigit(0,54).
happdigit(0,55).
happdigit(1,56).
happdigit(0,57).
happdigit(0,58).
happdigit(0,59).
happdigit(0,60).
happdigit(0,61).
happdigit(1,62).
happdigit(1,63).
happdigit(1,64).
happdigit(0,65).
happdigit(1,66).
happdigit(1,67).
happdigit(1,68).
happdigit(0,69).
happdigit(0,70).
happdigit(0,71).
happdigit(1,72).
happdigit(0,73).
happdigit(1,74).
happdigit(1,75).
happdigit(0,76).
happdigit(0,77).
happdigit(0,78).
happdigit(0,79).
happdigit(1,80).
"""
initiatedAt_output = """
% ProbLog Inference Result：
initAtDigit(sequence=0, 0).
initAtDigit(sequence=0, 1).
initAtDigit(sequence=0, 2).
initAtDigit(sequence=1, 3).
initAtDigit(sequence=1, 4).
initAtDigit(sequence=0, 5).
initAtDigit(sequence=1, 6).
initAtDigit(sequence=0, 7).
initAtDigit(sequence=1, 8).
initAtDigit(sequence=1, 9).
initAtDigit(sequence=0,10).
initAtDigit(sequence=1,11).
initAtDigit(sequence=1,12).
initAtDigit(sequence=1,13).
initAtDigit(sequence=0,14).
initAtDigit(sequence=0,15).
initAtDigit(sequence=0,16).
initAtDigit(sequence=0,17).
initAtDigit(sequence=1,18).
initAtDigit(sequence=1,19).
initAtDigit(sequence=1,20).
initAtDigit(sequence=1,21).
initAtDigit(sequence=1,22).
initAtDigit(sequence=1,23).
initAtDigit(sequence=0,24).
initAtDigit(sequence=1,25).
initAtDigit(sequence=1,26).
initAtDigit(sequence=0,27).
initAtDigit(sequence=1,28).
initAtDigit(sequence=0,29).
initAtDigit(sequence=1,30).
initAtDigit(sequence=0,31).
initAtDigit(sequence=0,32).
initAtDigit(sequence=0,33).
initAtDigit(sequence=1,34).
initAtDigit(sequence=1,35).
initAtDigit(sequence=1,36).
initAtDigit(sequence=1,37).
initAtDigit(sequence=1,38).
initAtDigit(sequence=1,39).
initAtDigit(sequence=1,40).
initAtDigit(sequence=0,41).
initAtDigit(sequence=1,42).
initAtDigit(sequence=1,43).
initAtDigit(sequence=1,44).
initAtDigit(sequence=0,45).
initAtDigit(sequence=1,46).
initAtDigit(sequence=0,47).
initAtDigit(sequence=0,48).
initAtDigit(sequence=0,49).
initAtDigit(sequence=0,50).
initAtDigit(sequence=0,51).
initAtDigit(sequence=0,52).
initAtDigit(sequence=0,53).
initAtDigit(sequence=0,54).
initAtDigit(sequence=0,55).
initAtDigit(sequence=0,56).
initAtDigit(sequence=0,57).
initAtDigit(sequence=0,58).
initAtDigit(sequence=0,59).
initAtDigit(sequence=0,60).
initAtDigit(sequence=0,61).
initAtDigit(sequence=0,62).
initAtDigit(sequence=1,63).
initAtDigit(sequence=1,64).
initAtDigit(sequence=0,65).
initAtDigit(sequence=1,66).
initAtDigit(sequence=1,67).
initAtDigit(sequence=1,68).
initAtDigit(sequence=0,69).
initAtDigit(sequence=0,70).
initAtDigit(sequence=0,71).
initAtDigit(sequence=0,72).
initAtDigit(sequence=0,73).
initAtDigit(sequence=1,74).
initAtDigit(sequence=1,75).
initAtDigit(sequence=0,76).
initAtDigit(sequence=0,77).
initAtDigit(sequence=0,78).
initAtDigit(sequence=0,79).
initAtDigit(sequence=0,80).
"""

In [139]:
prolog_reasoning(my_event_defs+query_list+'\n'+all_time_stamps+'\n'+happenat+'\n'+digit)

ProbLog Inference Result：
initAtEvent(sequence=1,0) = 0.0000
initAtEvent(sequence=1,1) = 0.0000
initAtEvent(sequence=1,2) = 0.0000
initAtEvent(sequence=1,3) = 0.0000
initAtEvent(sequence=1,4) = 0.0000
initAtEvent(sequence=1,5) = 0.0000
initAtEvent(sequence=1,6) = 1.0000
initAtEvent(sequence=1,7) = 0.0000
initAtEvent(sequence=1,8) = 0.0000
initAtEvent(sequence=1,9) = 1.0000
initAtEvent(sequence=1,10) = 0.0000
initAtEvent(sequence=1,11) = 1.0000
initAtEvent(sequence=1,12) = 1.0000
initAtEvent(sequence=1,13) = 1.0000
initAtEvent(sequence=1,14) = 1.0000
initAtEvent(sequence=1,15) = 0.0000
initAtEvent(sequence=1,16) = 0.0000
initAtEvent(sequence=1,17) = 0.0000
initAtEvent(sequence=1,18) = 0.0000
initAtEvent(sequence=1,19) = 1.0000
initAtEvent(sequence=1,20) = 1.0000
initAtEvent(sequence=1,21) = 1.0000
initAtEvent(sequence=1,22) = 1.0000
initAtEvent(sequence=1,23) = 1.0000
initAtEvent(sequence=1,24) = 1.0000
initAtEvent(sequence=1,25) = 1.0000
initAtEvent(sequence=1,26) = 1.0000
initAtEvent(

In [119]:
prolog_reasoning(prob_ec_cached+query_list+'\n'+all_time_stamps+'\n'+initiatedAt_output)

UnknownClause: No clauses found for 'initAtEvent/2' at 39:7.

In [ ]:
model = """
initiatedAt(sequence = Y, T) :-
    % at the current timestamp
    happensAt(X, T),
    digit(X, Y),

    % at the previous timestamp
    happensAt(Xprev, Tprev),
    digit(Xprev, Y),

    % timestamps
    allTimeStamps(Timestamps),
    previousTimeStamp(T, Timestamps, Tprev),
    Tprev >= 0.

happensAt(14564, 0).
happensAt(40314, 1).
happensAt(451, 2).
happensAt(56610, 3).
happensAt(49840, 4).
happensAt(27805, 5).
happensAt(42002, 6).
happensAt(51978, 7).
happensAt(18488, 8).
happensAt(44482, 9).
happensAt(53328, 10).

"""

In [80]:
from collections import deque, Counter
last_n = deque(maxlen=4)

for i,f in enumerate([1,2,3,1,2,2,2,5,4,4,3,4,2,2,1,1,2,1,3,1]):
    last_n.append(f"{f}")
    c_list = []
    counter = Counter(last_n)
    for c, key in enumerate(counter):
        if counter[key] > 1:
            c_list.append(key)

    print(last_n, c_list)


deque(['1'], maxlen=4) []
deque(['1', '2'], maxlen=4) []
deque(['1', '2', '3'], maxlen=4) []
deque(['1', '2', '3', '1'], maxlen=4) ['1']
deque(['2', '3', '1', '2'], maxlen=4) ['2']
deque(['3', '1', '2', '2'], maxlen=4) ['2']
deque(['1', '2', '2', '2'], maxlen=4) ['2']
deque(['2', '2', '2', '5'], maxlen=4) ['2']
deque(['2', '2', '5', '4'], maxlen=4) ['2']
deque(['2', '5', '4', '4'], maxlen=4) ['4']
deque(['5', '4', '4', '3'], maxlen=4) ['4']
deque(['4', '4', '3', '4'], maxlen=4) ['4']
deque(['4', '3', '4', '2'], maxlen=4) ['4']
deque(['3', '4', '2', '2'], maxlen=4) ['2']
deque(['4', '2', '2', '1'], maxlen=4) ['2']
deque(['2', '2', '1', '1'], maxlen=4) ['2', '1']
deque(['2', '1', '1', '2'], maxlen=4) ['2', '1']
deque(['1', '1', '2', '1'], maxlen=4) ['1']
deque(['1', '2', '1', '3'], maxlen=4) ['1']
deque(['2', '1', '3', '1'], maxlen=4) ['1']
